In [ ]:
## This file implements neural networks before and after lasso selection for p0006kpresabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0006kpresabsSTCC_qual.csv')
df.shape

(253, 22)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      1
1      0
2      1
3      1
4      1
      ..
248    1
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,...,GACAAACATGTATTAGCGTTATGTCGCGAACATCATAACCAGCAACATGCGATTGGCGTTAAGTCGTTTGATGATAAATATCACTTGCATGACTCGTGG,CTTTTTCACCTGT,CTTGTGAATTTAG,CTTATAACAATTACTATATTTGGCATTATATGTAGTATTATTTTCACGAG,CGCCATTATGTT,CAGAAAAGCGT,ACAATTACTATATTT,ST,CC,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,1,1,1,0,1,1,5,5,1
1,109,1,1,1,1,1,1,1,1,1,...,0,1,1,1,0,1,1,8,8,0
2,115,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,5,5,1
3,120335,1,1,1,1,1,1,1,1,1,...,0,1,1,1,0,1,1,5,5,1
4,120337,1,1,1,1,1,1,1,1,1,...,0,1,1,1,0,1,1,5,5,1


In [6]:
df['pheno'].value_counts()

0    129
1     85
2     39
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 21)

In [9]:
df_clean.head()

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,...,GACAAACATGTATTAGCGTTATGTCGCGAACATCATAACCAGCAACATGCGATTGGCGTTAAGTCGTTTGATGATAAATATCACTTGCATGACTCGTGG,CTTTTTCACCTGT,CTTGTGAATTTAG,CTTATAACAATTACTATATTTGGCATTATATGTAGTATTATTTTCACGAG,CGCCATTATGTT,CAGAAAAGCGT,ACAATTACTATATTT,ST,CC,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,0,1,1,5,5,1
1,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,0,1,1,8,8,0
2,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,5,5,1
3,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,0,1,1,5,5,1
4,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,0,1,1,5,5,1


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 21) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 129), (1, 129), (2, 129)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,NRS383,1
1,NRS254,1
2,NRS218,1
3,NRS215,0
4,BCH-SA-14,2
...,...,...
112,NRS227,1
113,NRS272,1
114,CFBRSa24,0
115,CFBRSa74,0


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 672us/step - loss: 4.0506 - accuracy: 0.3185 - val_loss: 2.3332 - val_accuracy: 0.4188
Epoch 2/1000
270/270 [==============================] - 0s 268us/step - loss: 1.7695 - accuracy: 0.3889 - val_loss: 1.5487 - val_accuracy: 0.3590
Epoch 3/1000
270/270 [==============================] - 0s 200us/step - loss: 1.5525 - accuracy: 0.4074 - val_loss: 1.2757 - val_accuracy: 0.3504
Epoch 4/1000
270/270 [==============================] - 0s 195us/step - loss: 1.1765 - accuracy: 0.4074 - val_loss: 1.1998 - val_accuracy: 0.4444
Epoch 5/1000
270/270 [==============================] - 0s 166us/step - loss: 1.1466 - accuracy: 0.5148 - val_loss: 1.1958 - val_accuracy: 0.4359
Epoch 6/1000
270/270 [==============================] - 0s 142us/step - loss: 1.1541 - accuracy: 0.4407 - val_loss: 1.1699 - val_accuracy: 0.4017
Epoch 7/1000
270/270 [==============================] - 0s 184us/step - loss: 

Epoch 112/1000
270/270 [==============================] - 0s 140us/step - loss: 0.8488 - accuracy: 0.6593 - val_loss: 1.0057 - val_accuracy: 0.6068
Epoch 113/1000
270/270 [==============================] - 0s 446us/step - loss: 0.8864 - accuracy: 0.6407 - val_loss: 0.9666 - val_accuracy: 0.5470
Epoch 114/1000
270/270 [==============================] - 0s 92us/step - loss: 0.9374 - accuracy: 0.6593 - val_loss: 0.9525 - val_accuracy: 0.5641
Epoch 115/1000
270/270 [==============================] - 0s 125us/step - loss: 0.9258 - accuracy: 0.6519 - val_loss: 0.9827 - val_accuracy: 0.5556
Epoch 116/1000
270/270 [==============================] - 0s 104us/step - loss: 1.0106 - accuracy: 0.6407 - val_loss: 0.9723 - val_accuracy: 0.5214
Epoch 117/1000
270/270 [==============================] - 0s 153us/step - loss: 0.8879 - accuracy: 0.6333 - val_loss: 0.9763 - val_accuracy: 0.5897
Epoch 118/1000
270/270 [==============================] - 0s 109us/step - loss: 0.8812 - accuracy: 0.6296 - val_l

270/270 [==============================] - 0s 267us/step - loss: 0.7513 - accuracy: 0.6963 - val_loss: 0.9952 - val_accuracy: 0.5470
Epoch 223/1000
270/270 [==============================] - 0s 364us/step - loss: 0.9982 - accuracy: 0.6556 - val_loss: 0.9672 - val_accuracy: 0.6325
Epoch 224/1000
270/270 [==============================] - 0s 232us/step - loss: 1.2467 - accuracy: 0.6926 - val_loss: 1.2764 - val_accuracy: 0.6154
Epoch 225/1000
270/270 [==============================] - 0s 338us/step - loss: 1.0759 - accuracy: 0.6741 - val_loss: 0.9707 - val_accuracy: 0.5299
Epoch 226/1000
270/270 [==============================] - 0s 175us/step - loss: 0.8428 - accuracy: 0.6704 - val_loss: 0.9134 - val_accuracy: 0.6154
Epoch 227/1000
270/270 [==============================] - 0s 252us/step - loss: 0.7414 - accuracy: 0.6741 - val_loss: 0.9701 - val_accuracy: 0.5470
Epoch 228/1000
270/270 [==============================] - 0s 378us/step - loss: 0.7916 - accuracy: 0.6889 - val_loss: 0.9363 - 

270/270 [==============================] - 0s 78us/step - loss: 0.7145 - accuracy: 0.6815 - val_loss: 0.9358 - val_accuracy: 0.5897
Epoch 333/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8767 - accuracy: 0.7037 - val_loss: 1.0904 - val_accuracy: 0.5983
Epoch 334/1000
270/270 [==============================] - 0s 89us/step - loss: 0.8377 - accuracy: 0.6963 - val_loss: 1.0983 - val_accuracy: 0.4701
Epoch 335/1000
270/270 [==============================] - 0s 97us/step - loss: 0.8629 - accuracy: 0.6667 - val_loss: 1.0094 - val_accuracy: 0.5897
Epoch 336/1000
270/270 [==============================] - 0s 101us/step - loss: 0.7583 - accuracy: 0.6926 - val_loss: 1.0467 - val_accuracy: 0.4786
Epoch 337/1000
270/270 [==============================] - 0s 272us/step - loss: 0.7775 - accuracy: 0.6852 - val_loss: 0.9648 - val_accuracy: 0.5812
Epoch 338/1000
270/270 [==============================] - 0s 128us/step - loss: 0.7638 - accuracy: 0.6778 - val_loss: 0.9784 - val_

270/270 [==============================] - 0s 72us/step - loss: 0.7966 - accuracy: 0.6889 - val_loss: 1.0082 - val_accuracy: 0.5812
Epoch 443/1000
270/270 [==============================] - 0s 70us/step - loss: 0.7287 - accuracy: 0.6815 - val_loss: 0.9789 - val_accuracy: 0.5726
Epoch 444/1000
270/270 [==============================] - 0s 81us/step - loss: 0.7083 - accuracy: 0.6963 - val_loss: 0.9899 - val_accuracy: 0.5385
Epoch 445/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6849 - accuracy: 0.6963 - val_loss: 0.9765 - val_accuracy: 0.5726
Epoch 446/1000
270/270 [==============================] - 0s 85us/step - loss: 0.6733 - accuracy: 0.7074 - val_loss: 0.9884 - val_accuracy: 0.5983
Epoch 447/1000
270/270 [==============================] - 0s 83us/step - loss: 0.6774 - accuracy: 0.7185 - val_loss: 0.9943 - val_accuracy: 0.5385
Epoch 448/1000
270/270 [==============================] - 0s 85us/step - loss: 0.6612 - accuracy: 0.7111 - val_loss: 0.9695 - val_acc

...

270/270 [==============================] - 0s 92us/step - loss: 0.6422 - accuracy: 0.7037 - val_loss: 1.0482 - val_accuracy: 0.5897
Epoch 609/1000
270/270 [==============================] - 0s 97us/step - loss: 0.6575 - accuracy: 0.7074 - val_loss: 1.0529 - val_accuracy: 0.6239
Epoch 610/1000
270/270 [==============================] - 0s 92us/step - loss: 0.6276 - accuracy: 0.7111 - val_loss: 1.0658 - val_accuracy: 0.5726
Epoch 611/1000
270/270 [==============================] - 0s 95us/step - loss: 0.6365 - accuracy: 0.7148 - val_loss: 1.0422 - val_accuracy: 0.6154
Epoch 612/1000
270/270 [==============================] - 0s 75us/step - loss: 0.6064 - accuracy: 0.7370 - val_loss: 1.0688 - val_accuracy: 0.5470
Epoch 613/1000
270/270 [==============================] - 0s 76us/step - loss: 0.6359 - accuracy: 0.7111 - val_loss: 1.0394 - val_accuracy: 0.6154
Epoch 614/1000
270/270 [==============================] - 0s 78us/step - loss: 0.6251 - accuracy: 0.7296 - val_loss: 1.0570 - val_acc

270/270 [==============================] - 0s 77us/step - loss: 0.5970 - accuracy: 0.7185 - val_loss: 1.0958 - val_accuracy: 0.6239
Epoch 719/1000
270/270 [==============================] - 0s 78us/step - loss: 0.5953 - accuracy: 0.7333 - val_loss: 1.0828 - val_accuracy: 0.6068
Epoch 720/1000
270/270 [==============================] - 0s 99us/step - loss: 0.5946 - accuracy: 0.7259 - val_loss: 1.0847 - val_accuracy: 0.6239
Epoch 721/1000
270/270 [==============================] - 0s 105us/step - loss: 0.6015 - accuracy: 0.7370 - val_loss: 1.0784 - val_accuracy: 0.6154
Epoch 722/1000
270/270 [==============================] - 0s 94us/step - loss: 0.6144 - accuracy: 0.7185 - val_loss: 1.0707 - val_accuracy: 0.6325
Epoch 723/1000
270/270 [==============================] - ETA: 0s - loss: 0.6870 - accuracy: 0.59 - 0s 107us/step - loss: 0.6015 - accuracy: 0.7222 - val_loss: 1.0982 - val_accuracy: 0.5897
Epoch 724/1000
270/270 [==============================] - 0s 103us/step - loss: 0.6003 - 

Epoch 829/1000
270/270 [==============================] - 0s 104us/step - loss: 0.6816 - accuracy: 0.7333 - val_loss: 1.1450 - val_accuracy: 0.5470
Epoch 830/1000
270/270 [==============================] - 0s 118us/step - loss: 0.7171 - accuracy: 0.7000 - val_loss: 1.1291 - val_accuracy: 0.6325
Epoch 831/1000
270/270 [==============================] - 0s 122us/step - loss: 0.7260 - accuracy: 0.7296 - val_loss: 1.1687 - val_accuracy: 0.6154
Epoch 832/1000
270/270 [==============================] - 0s 133us/step - loss: 0.6461 - accuracy: 0.7444 - val_loss: 1.1518 - val_accuracy: 0.5470
Epoch 833/1000
270/270 [==============================] - 0s 125us/step - loss: 0.6072 - accuracy: 0.7333 - val_loss: 1.1009 - val_accuracy: 0.5812
Epoch 834/1000
270/270 [==============================] - 0s 104us/step - loss: 0.5848 - accuracy: 0.7222 - val_loss: 1.1193 - val_accuracy: 0.5897
Epoch 835/1000
270/270 [==============================] - 0s 101us/step - loss: 0.5827 - accuracy: 0.7185 - val_

Epoch 940/1000
270/270 [==============================] - 0s 92us/step - loss: 0.5693 - accuracy: 0.7556 - val_loss: 1.1199 - val_accuracy: 0.6239
Epoch 941/1000
270/270 [==============================] - 0s 95us/step - loss: 0.5645 - accuracy: 0.7444 - val_loss: 1.1321 - val_accuracy: 0.5556
Epoch 942/1000
270/270 [==============================] - 0s 94us/step - loss: 0.5803 - accuracy: 0.7333 - val_loss: 1.1296 - val_accuracy: 0.6325
Epoch 943/1000
270/270 [==============================] - 0s 149us/step - loss: 0.5684 - accuracy: 0.7556 - val_loss: 1.1630 - val_accuracy: 0.5983
Epoch 944/1000
270/270 [==============================] - 0s 91us/step - loss: 0.5652 - accuracy: 0.7519 - val_loss: 1.1313 - val_accuracy: 0.6239
Epoch 945/1000
270/270 [==============================] - 0s 94us/step - loss: 0.5838 - accuracy: 0.7407 - val_loss: 1.1521 - val_accuracy: 0.5897
Epoch 946/1000
270/270 [==============================] - 0s 89us/step - loss: 0.5703 - accuracy: 0.7185 - val_loss: 

Epoch 996/1000
270/270 [==============================] - 0s 89us/step - loss: 0.5590 - accuracy: 0.7444 - val_loss: 1.1615 - val_accuracy: 0.6154
Epoch 997/1000
270/270 [==============================] - 0s 109us/step - loss: 0.5710 - accuracy: 0.7444 - val_loss: 1.1426 - val_accuracy: 0.5897
Epoch 998/1000
270/270 [==============================] - 0s 109us/step - loss: 0.5649 - accuracy: 0.7333 - val_loss: 1.1337 - val_accuracy: 0.6325
Epoch 999/1000
270/270 [==============================] - 0s 106us/step - loss: 0.5511 - accuracy: 0.7519 - val_loss: 1.1694 - val_accuracy: 0.5983
Epoch 1000/1000
270/270 [==============================] - 0s 89us/step - loss: 0.5679 - accuracy: 0.7444 - val_loss: 1.1570 - val_accuracy: 0.5983


In [33]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

117/117 [==============================] - 0s 68us/step
over-sampling test accuracy: 60.68%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([1, 1, 1, 0, 1, 1, 0, 2, 2, 2, 0, 2, 1, 0, 0, 1, 2, 2, 1, 2, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 2, 2, 0, 0, 2, 0, 0, 1, 2, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 2, 0, 0, 0, 2, 2, 0, 1, 0, 2, 2, 1, 2, 1, 0, 1, 2, 1, 2,
       1, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 1, 0, 0, 1, 1, 2, 2, 1, 1, 2, 1,
       0, 0, 2, 1, 1, 0, 2, 1, 1, 2, 1, 0, 2, 1, 1, 2, 0, 0, 2, 2, 1, 2,
       0, 0, 2, 2, 0, 1, 2])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,NRS383,1,1
1,NRS254,1,1
2,NRS218,1,1
3,NRS215,0,0
4,BCH-SA-14,2,1
...,...,...,...
112,NRS227,1,2
113,NRS272,1,2
114,CFBRSa24,0,0
115,CFBRSa74,0,1


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,0.134787,0.581356,0.283858
1,0.491630,0.497916,0.010454
2,0.324922,0.594283,0.080795
3,0.758227,0.169591,0.072182
4,0.388507,0.602946,0.008547
...,...,...,...
112,0.025057,0.046394,0.928549
113,0.053057,0.161249,0.785694
114,0.636068,0.360300,0.003632
115,0.248718,0.619990,0.131292


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p006ST.csv", index = False,
         header=None)

In [37]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 215us/step - loss: 0.7738 - accuracy: 0.7333 - val_loss: 1.1911 - val_accuracy: 0.6239
Epoch 2/1000
270/270 [==============================] - 0s 134us/step - loss: 0.7790 - accuracy: 0.7333 - val_loss: 1.2449 - val_accuracy: 0.6068
Epoch 3/1000
270/270 [==============================] - 0s 117us/step - loss: 0.9486 - accuracy: 0.7222 - val_loss: 1.2078 - val_accuracy: 0.5812
Epoch 4/1000
270/270 [==============================] - 0s 115us/step - loss: 1.2028 - accuracy: 0.7074 - val_loss: 1.7212 - val_accuracy: 0.5897
Epoch 5/1000
270/270 [==============================] - 0s 104us/step - loss: 1.1186 - accuracy: 0.7000 - val_loss: 1.7170 - val_accuracy: 0.4872
Epoch 6/1000
270/270 [==============================] - 0s 116us/step - loss: 0.9649 - accuracy: 0.7111 - val_loss: 1.3942 - val_accuracy: 0.6068
Epoch 7/1000
270/270 [==============================] - 0s 111us/step - loss: 

Epoch 57/1000
270/270 [==============================] - 0s 139us/step - loss: 0.5308 - accuracy: 0.7407 - val_loss: 1.1713 - val_accuracy: 0.6154
Epoch 58/1000
270/270 [==============================] - 0s 99us/step - loss: 0.5252 - accuracy: 0.7593 - val_loss: 1.1744 - val_accuracy: 0.6068
Epoch 59/1000
270/270 [==============================] - 0s 92us/step - loss: 0.5264 - accuracy: 0.7556 - val_loss: 1.1906 - val_accuracy: 0.6068
Epoch 60/1000
270/270 [==============================] - 0s 85us/step - loss: 0.5179 - accuracy: 0.7481 - val_loss: 1.2044 - val_accuracy: 0.6154
Epoch 61/1000
270/270 [==============================] - 0s 78us/step - loss: 0.5757 - accuracy: 0.7407 - val_loss: 1.2000 - val_accuracy: 0.5983
Epoch 62/1000
270/270 [==============================] - 0s 90us/step - loss: 0.5286 - accuracy: 0.7481 - val_loss: 1.1785 - val_accuracy: 0.6239
Epoch 63/1000
270/270 [==============================] - 0s 77us/step - loss: 0.5462 - accuracy: 0.7593 - val_loss: 1.1809 

Epoch 113/1000
270/270 [==============================] - 0s 81us/step - loss: 0.5196 - accuracy: 0.7593 - val_loss: 1.1911 - val_accuracy: 0.5812
Epoch 114/1000
270/270 [==============================] - 0s 93us/step - loss: 0.5148 - accuracy: 0.7481 - val_loss: 1.2062 - val_accuracy: 0.5897
Epoch 115/1000
270/270 [==============================] - 0s 195us/step - loss: 0.5369 - accuracy: 0.7407 - val_loss: 1.1839 - val_accuracy: 0.5983
Epoch 116/1000
270/270 [==============================] - 0s 153us/step - loss: 0.5274 - accuracy: 0.7519 - val_loss: 1.1864 - val_accuracy: 0.6068
Epoch 117/1000
270/270 [==============================] - 0s 159us/step - loss: 0.5323 - accuracy: 0.7556 - val_loss: 1.1909 - val_accuracy: 0.6068
Epoch 118/1000
270/270 [==============================] - 0s 150us/step - loss: 0.5507 - accuracy: 0.7593 - val_loss: 1.1961 - val_accuracy: 0.5983
Epoch 119/1000
270/270 [==============================] - 0s 114us/step - loss: 0.6028 - accuracy: 0.7556 - val_lo

Epoch 224/1000
270/270 [==============================] - 0s 101us/step - loss: 0.5282 - accuracy: 0.7630 - val_loss: 1.2016 - val_accuracy: 0.6068
Epoch 225/1000
270/270 [==============================] - 0s 97us/step - loss: 0.5378 - accuracy: 0.7556 - val_loss: 1.2160 - val_accuracy: 0.5897
Epoch 226/1000
270/270 [==============================] - 0s 94us/step - loss: 0.5270 - accuracy: 0.7481 - val_loss: 1.2147 - val_accuracy: 0.6154
Epoch 227/1000
270/270 [==============================] - 0s 92us/step - loss: 0.5258 - accuracy: 0.7630 - val_loss: 1.2028 - val_accuracy: 0.6154
Epoch 228/1000
270/270 [==============================] - 0s 94us/step - loss: 0.5208 - accuracy: 0.7704 - val_loss: 1.2221 - val_accuracy: 0.5983
Epoch 229/1000
270/270 [==============================] - 0s 109us/step - loss: 0.5343 - accuracy: 0.7556 - val_loss: 1.2192 - val_accuracy: 0.6068
Epoch 230/1000
270/270 [==============================] - 0s 116us/step - loss: 0.5523 - accuracy: 0.7593 - val_loss

270/270 [==============================] - 0s 114us/step - loss: 0.5218 - accuracy: 0.7593 - val_loss: 1.2362 - val_accuracy: 0.5726
Epoch 335/1000
270/270 [==============================] - 0s 147us/step - loss: 0.5595 - accuracy: 0.7556 - val_loss: 1.2101 - val_accuracy: 0.6154
Epoch 336/1000
270/270 [==============================] - 0s 197us/step - loss: 0.5292 - accuracy: 0.7667 - val_loss: 1.2167 - val_accuracy: 0.5983
Epoch 337/1000
270/270 [==============================] - 0s 143us/step - loss: 0.5282 - accuracy: 0.7593 - val_loss: 1.2196 - val_accuracy: 0.5983
Epoch 338/1000
270/270 [==============================] - 0s 190us/step - loss: 0.5201 - accuracy: 0.7593 - val_loss: 1.2152 - val_accuracy: 0.6154
Epoch 339/1000
270/270 [==============================] - 0s 162us/step - loss: 0.5160 - accuracy: 0.7519 - val_loss: 1.2159 - val_accuracy: 0.5983
Epoch 340/1000
270/270 [==============================] - 0s 273us/step - loss: 0.5135 - accuracy: 0.7630 - val_loss: 1.2181 - 

...

270/270 [==============================] - 0s 88us/step - loss: 0.4986 - accuracy: 0.7704 - val_loss: 1.2147 - val_accuracy: 0.6154
Epoch 555/1000
270/270 [==============================] - 0s 100us/step - loss: 0.5211 - accuracy: 0.7667 - val_loss: 1.2271 - val_accuracy: 0.5897
Epoch 556/1000
270/270 [==============================] - 0s 99us/step - loss: 0.5084 - accuracy: 0.7593 - val_loss: 1.2130 - val_accuracy: 0.6325
Epoch 557/1000
270/270 [==============================] - 0s 97us/step - loss: 0.5220 - accuracy: 0.7370 - val_loss: 1.2273 - val_accuracy: 0.6068
Epoch 558/1000
270/270 [==============================] - 0s 99us/step - loss: 0.5026 - accuracy: 0.7630 - val_loss: 1.2243 - val_accuracy: 0.5983
Epoch 559/1000
270/270 [==============================] - 0s 98us/step - loss: 0.5290 - accuracy: 0.7630 - val_loss: 1.2100 - val_accuracy: 0.6068
Epoch 560/1000
270/270 [==============================] - 0s 93us/step - loss: 0.5095 - accuracy: 0.7667 - val_loss: 1.2258 - val_ac

270/270 [==============================] - 0s 97us/step - loss: 0.5550 - accuracy: 0.7444 - val_loss: 1.2578 - val_accuracy: 0.5641
Epoch 665/1000
270/270 [==============================] - 0s 110us/step - loss: 0.5356 - accuracy: 0.7556 - val_loss: 1.2620 - val_accuracy: 0.6154
Epoch 666/1000
270/270 [==============================] - 0s 171us/step - loss: 0.4956 - accuracy: 0.7667 - val_loss: 1.2478 - val_accuracy: 0.6154
Epoch 667/1000
270/270 [==============================] - 0s 137us/step - loss: 0.5258 - accuracy: 0.7704 - val_loss: 1.2304 - val_accuracy: 0.6068
Epoch 668/1000
270/270 [==============================] - 0s 118us/step - loss: 0.5098 - accuracy: 0.7593 - val_loss: 1.2387 - val_accuracy: 0.6068
Epoch 669/1000
270/270 [==============================] - 0s 93us/step - loss: 0.5154 - accuracy: 0.7630 - val_loss: 1.2327 - val_accuracy: 0.5897
Epoch 670/1000
270/270 [==============================] - 0s 130us/step - loss: 0.5069 - accuracy: 0.7593 - val_loss: 1.2235 - va

270/270 [==============================] - 0s 83us/step - loss: 0.9391 - accuracy: 0.7185 - val_loss: 1.2429 - val_accuracy: 0.5641
Epoch 775/1000
270/270 [==============================] - 0s 81us/step - loss: 0.7174 - accuracy: 0.7556 - val_loss: 1.2967 - val_accuracy: 0.5812
Epoch 776/1000
270/270 [==============================] - 0s 104us/step - loss: 0.5000 - accuracy: 0.7778 - val_loss: 1.2464 - val_accuracy: 0.6239
Epoch 777/1000
270/270 [==============================] - 0s 111us/step - loss: 0.5068 - accuracy: 0.7704 - val_loss: 1.2209 - val_accuracy: 0.5983
Epoch 778/1000
270/270 [==============================] - 0s 101us/step - loss: 0.5333 - accuracy: 0.7593 - val_loss: 1.2139 - val_accuracy: 0.6154
Epoch 779/1000
270/270 [==============================] - 0s 129us/step - loss: 0.5020 - accuracy: 0.7593 - val_loss: 1.2161 - val_accuracy: 0.6154
Epoch 780/1000
270/270 [==============================] - 0s 106us/step - loss: 0.5001 - accuracy: 0.7704 - val_loss: 1.2062 - va

270/270 [==============================] - 0s 147us/step - loss: 0.5026 - accuracy: 0.7593 - val_loss: 1.2489 - val_accuracy: 0.5983
Epoch 885/1000
270/270 [==============================] - 0s 166us/step - loss: 0.4922 - accuracy: 0.7593 - val_loss: 1.2600 - val_accuracy: 0.6068
Epoch 886/1000
270/270 [==============================] - 0s 158us/step - loss: 0.5033 - accuracy: 0.7519 - val_loss: 1.2483 - val_accuracy: 0.5897
Epoch 887/1000
270/270 [==============================] - 0s 111us/step - loss: 0.5172 - accuracy: 0.7704 - val_loss: 1.2487 - val_accuracy: 0.5812
Epoch 888/1000
270/270 [==============================] - 0s 96us/step - loss: 0.5141 - accuracy: 0.7593 - val_loss: 1.2573 - val_accuracy: 0.5812
Epoch 889/1000
270/270 [==============================] - 0s 114us/step - loss: 0.5264 - accuracy: 0.7667 - val_loss: 1.2653 - val_accuracy: 0.5812
Epoch 890/1000
270/270 [==============================] - 0s 112us/step - loss: 0.5178 - accuracy: 0.7556 - val_loss: 1.2638 - v

270/270 [==============================] - 0s 115us/step - loss: 0.5011 - accuracy: 0.7667 - val_loss: 1.2222 - val_accuracy: 0.6068
Epoch 995/1000
270/270 [==============================] - 0s 158us/step - loss: 0.5024 - accuracy: 0.7667 - val_loss: 1.2146 - val_accuracy: 0.5897
Epoch 996/1000
270/270 [==============================] - 0s 126us/step - loss: 0.5038 - accuracy: 0.7667 - val_loss: 1.2274 - val_accuracy: 0.6068
Epoch 997/1000
270/270 [==============================] - 0s 119us/step - loss: 0.5108 - accuracy: 0.7444 - val_loss: 1.2241 - val_accuracy: 0.5983
Epoch 998/1000
270/270 [==============================] - 0s 130us/step - loss: 0.5029 - accuracy: 0.7667 - val_loss: 1.2309 - val_accuracy: 0.5897
Epoch 999/1000
270/270 [==============================] - 0s 116us/step - loss: 0.5050 - accuracy: 0.7593 - val_loss: 1.2258 - val_accuracy: 0.6239
Epoch 1000/1000
270/270 [==============================] - 0s 134us/step - loss: 0.5001 - accuracy: 0.7481 - val_loss: 1.2337 -

In [38]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 75.62%


In [19]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [20]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [21]:
y_prob = df_proba[df_proba['phage']=='p0006kpresabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[1.34786560e-01, 5.81355750e-01, 2.83857700e-01],
       [4.91629700e-01, 4.97916130e-01, 1.04541680e-02],
       [3.24921550e-01, 5.94283340e-01, 8.07951700e-02],
       [7.58227100e-01, 1.69591040e-01, 7.21818060e-02],
       [3.88507400e-01, 6.02945570e-01, 8.54699600e-03],
       [2.48717840e-01, 6.19990350e-01, 1.31291820e-01],
       [9.79144400e-01, 2.08521080e-02, 3.45880040e-06],
       [7.65052900e-03, 3.59286110e-03, 9.88756600e-01],
       [1.59875780e-04, 1.27847340e-04, 9.99712300e-01],
       [1.81753820e-05, 4.53804530e-04, 9.99528050e-01],
       [6.74196960e-01, 3.22848980e-01, 2.95411380e-03],
       [1.34261770e-05, 7.67990600e-05, 9.99909760e-01],
       [8.47269800e-02, 6.71719500e-01, 2.43553490e-01],
       [6.36068000e-01, 3.60300450e-01, 3.63160830e-03],
       [6.36068000e-01, 3.60300450e-01, 3.63160830e-03],
       [3.88507400e-01, 6.02945570e-01, 8.54699600e-03],
       [1.08321466e-01, 3.19265630e-01, 5.72412850e-01],
       [1.19599120e-01, 2.91120

In [15]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [23]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.7594784133245671

In [16]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [25]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.7594784133245671

In [26]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [27]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [28]:
dat2

,0,test
0,BCH-SA-11,2
1,NRS161,1
2,BCH-SA-14,2
3,BCH-SA-11,2
4,CFBRSa49,1
...,...,...
112,NRS064,2
113,NRS266,2
114,NRS222,0
115,GA27,2


In [29]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [30]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 480us/step - loss: 4.3294 - accuracy: 0.3148 - val_loss: 4.5637 - val_accuracy: 0.3077
Epoch 2/1000
270/270 [==============================] - 0s 90us/step - loss: 3.0343 - accuracy: 0.3444 - val_loss: 2.0698 - val_accuracy: 0.3590
Epoch 3/1000
270/270 [==============================] - 0s 69us/step - loss: 2.0185 - accuracy: 0.3333 - val_loss: 1.9301 - val_accuracy: 0.3590
Epoch 4/1000
270/270 [==============================] - 0s 76us/step - loss: 1.9052 - accuracy: 0.3370 - val_loss: 2.2624 - val_accuracy: 0.3590
Epoch 5/1000
270/270 [==============================] - 0s 117us/step - loss: 1.6080 - accuracy: 0.3778 - val_loss: 1.7821 - val_accuracy: 0.3761
Epoch 6/1000
270/270 [==============================] - 0s 74us/step - loss: 2.2293 - accuracy: 0.3704 - val_loss: 1.6243 - val_accuracy: 0.4103
Epoch 7/1000
270/270 [==============================] - 0s 62us/step - loss: 1.506

Epoch 57/1000
270/270 [==============================] - 0s 66us/step - loss: 0.9802 - accuracy: 0.5370 - val_loss: 0.9441 - val_accuracy: 0.5385
Epoch 58/1000
270/270 [==============================] - 0s 67us/step - loss: 1.1004 - accuracy: 0.5259 - val_loss: 1.2760 - val_accuracy: 0.5897
Epoch 59/1000
270/270 [==============================] - 0s 173us/step - loss: 1.0918 - accuracy: 0.5593 - val_loss: 0.9482 - val_accuracy: 0.6239
Epoch 60/1000
270/270 [==============================] - 0s 127us/step - loss: 1.1067 - accuracy: 0.5704 - val_loss: 1.3662 - val_accuracy: 0.6068
Epoch 61/1000
270/270 [==============================] - 0s 207us/step - loss: 1.3313 - accuracy: 0.5778 - val_loss: 1.3574 - val_accuracy: 0.5641
Epoch 62/1000
270/270 [==============================] - 0s 161us/step - loss: 1.3269 - accuracy: 0.5593 - val_loss: 1.3284 - val_accuracy: 0.6154
Epoch 63/1000
270/270 [==============================] - 0s 87us/step - loss: 1.6934 - accuracy: 0.5593 - val_loss: 2.36

Epoch 113/1000
270/270 [==============================] - 0s 50us/step - loss: 0.9274 - accuracy: 0.6185 - val_loss: 0.9003 - val_accuracy: 0.6154
Epoch 114/1000
270/270 [==============================] - 0s 54us/step - loss: 0.9464 - accuracy: 0.6000 - val_loss: 1.0817 - val_accuracy: 0.6496
Epoch 115/1000
270/270 [==============================] - 0s 52us/step - loss: 1.0565 - accuracy: 0.5852 - val_loss: 1.1267 - val_accuracy: 0.6239
Epoch 116/1000
270/270 [==============================] - 0s 63us/step - loss: 0.9718 - accuracy: 0.5963 - val_loss: 0.9276 - val_accuracy: 0.6068
Epoch 117/1000
270/270 [==============================] - 0s 53us/step - loss: 1.0020 - accuracy: 0.5815 - val_loss: 1.1635 - val_accuracy: 0.6325
Epoch 118/1000
270/270 [==============================] - 0s 70us/step - loss: 1.0556 - accuracy: 0.5889 - val_loss: 1.0026 - val_accuracy: 0.6496
Epoch 119/1000
270/270 [==============================] - 0s 61us/step - loss: 0.9580 - accuracy: 0.6037 - val_loss: 0

Epoch 224/1000
270/270 [==============================] - 0s 51us/step - loss: 1.1001 - accuracy: 0.6185 - val_loss: 1.0103 - val_accuracy: 0.5983
Epoch 225/1000
270/270 [==============================] - 0s 67us/step - loss: 1.1763 - accuracy: 0.5704 - val_loss: 1.1606 - val_accuracy: 0.6154
Epoch 226/1000
270/270 [==============================] - 0s 56us/step - loss: 1.4206 - accuracy: 0.6185 - val_loss: 2.4135 - val_accuracy: 0.6068
Epoch 227/1000
270/270 [==============================] - 0s 50us/step - loss: 1.8084 - accuracy: 0.5889 - val_loss: 2.2472 - val_accuracy: 0.5983
Epoch 228/1000
270/270 [==============================] - 0s 84us/step - loss: 1.6310 - accuracy: 0.5704 - val_loss: 1.6642 - val_accuracy: 0.6325
Epoch 229/1000
270/270 [==============================] - 0s 70us/step - loss: 1.1667 - accuracy: 0.5407 - val_loss: 1.1771 - val_accuracy: 0.4530
Epoch 230/1000
270/270 [==============================] - 0s 62us/step - loss: 1.1855 - accuracy: 0.5222 - val_loss: 1

Epoch 280/1000
270/270 [==============================] - 0s 47us/step - loss: 0.9485 - accuracy: 0.6259 - val_loss: 1.0103 - val_accuracy: 0.6496
Epoch 281/1000
270/270 [==============================] - 0s 42us/step - loss: 0.9275 - accuracy: 0.6370 - val_loss: 0.9178 - val_accuracy: 0.5983
Epoch 282/1000
270/270 [==============================] - 0s 48us/step - loss: 0.8940 - accuracy: 0.6296 - val_loss: 0.9559 - val_accuracy: 0.6410
Epoch 283/1000
270/270 [==============================] - 0s 45us/step - loss: 0.8812 - accuracy: 0.6074 - val_loss: 0.9847 - val_accuracy: 0.6581
Epoch 284/1000
270/270 [==============================] - 0s 47us/step - loss: 0.9947 - accuracy: 0.6222 - val_loss: 1.0298 - val_accuracy: 0.5983
Epoch 285/1000
270/270 [==============================] - 0s 45us/step - loss: 1.0506 - accuracy: 0.5852 - val_loss: 0.8858 - val_accuracy: 0.6581
Epoch 286/1000
270/270 [==============================] - 0s 45us/step - loss: 0.9346 - accuracy: 0.6148 - val_loss: 1

...

Epoch 669/1000
270/270 [==============================] - 0s 68us/step - loss: 1.1007 - accuracy: 0.5963 - val_loss: 1.5412 - val_accuracy: 0.5897
Epoch 670/1000
270/270 [==============================] - 0s 61us/step - loss: 1.1044 - accuracy: 0.6259 - val_loss: 1.0503 - val_accuracy: 0.6496
Epoch 671/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8258 - accuracy: 0.6778 - val_loss: 0.8835 - val_accuracy: 0.6325
Epoch 672/1000
270/270 [==============================] - 0s 67us/step - loss: 0.7993 - accuracy: 0.6741 - val_loss: 1.1149 - val_accuracy: 0.6325
Epoch 673/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8631 - accuracy: 0.6222 - val_loss: 0.9558 - val_accuracy: 0.6496
Epoch 674/1000
270/270 [==============================] - 0s 68us/step - loss: 0.8035 - accuracy: 0.6815 - val_loss: 0.9038 - val_accuracy: 0.5983
Epoch 675/1000
270/270 [==============================] - 0s 65us/step - loss: 0.8302 - accuracy: 0.6630 - val_loss: 0

Epoch 725/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7702 - accuracy: 0.6370 - val_loss: 0.8879 - val_accuracy: 0.6667
Epoch 726/1000
270/270 [==============================] - 0s 53us/step - loss: 0.7588 - accuracy: 0.6815 - val_loss: 0.8976 - val_accuracy: 0.6581
Epoch 727/1000
270/270 [==============================] - 0s 60us/step - loss: 0.7583 - accuracy: 0.6815 - val_loss: 0.8839 - val_accuracy: 0.6325
Epoch 728/1000
270/270 [==============================] - 0s 68us/step - loss: 0.7584 - accuracy: 0.6889 - val_loss: 0.9240 - val_accuracy: 0.6923
Epoch 729/1000
270/270 [==============================] - 0s 66us/step - loss: 0.7690 - accuracy: 0.6926 - val_loss: 0.8630 - val_accuracy: 0.6667
Epoch 730/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7610 - accuracy: 0.7000 - val_loss: 0.9541 - val_accuracy: 0.6838
Epoch 731/1000
270/270 [==============================] - 0s 235us/step - loss: 0.8032 - accuracy: 0.6963 - val_loss: 

Epoch 781/1000
270/270 [==============================] - 0s 118us/step - loss: 0.7547 - accuracy: 0.6852 - val_loss: 0.9189 - val_accuracy: 0.6752
Epoch 782/1000
270/270 [==============================] - 0s 57us/step - loss: 0.7583 - accuracy: 0.7111 - val_loss: 0.9348 - val_accuracy: 0.6581
Epoch 783/1000
270/270 [==============================] - 0s 58us/step - loss: 0.7538 - accuracy: 0.7037 - val_loss: 0.9065 - val_accuracy: 0.6667
Epoch 784/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7548 - accuracy: 0.7111 - val_loss: 0.9024 - val_accuracy: 0.6838
Epoch 785/1000
270/270 [==============================] - 0s 76us/step - loss: 0.7591 - accuracy: 0.7074 - val_loss: 0.9043 - val_accuracy: 0.6667
Epoch 786/1000
270/270 [==============================] - 0s 70us/step - loss: 0.7440 - accuracy: 0.7037 - val_loss: 0.9059 - val_accuracy: 0.6239
Epoch 787/1000
270/270 [==============================] - 0s 69us/step - loss: 0.7793 - accuracy: 0.6852 - val_loss: 

Epoch 837/1000
270/270 [==============================] - 0s 56us/step - loss: 0.7931 - accuracy: 0.6815 - val_loss: 0.9618 - val_accuracy: 0.6752
Epoch 838/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8499 - accuracy: 0.6741 - val_loss: 1.0293 - val_accuracy: 0.6496
Epoch 839/1000
270/270 [==============================] - 0s 76us/step - loss: 0.7913 - accuracy: 0.6815 - val_loss: 0.9419 - val_accuracy: 0.6239
Epoch 840/1000
270/270 [==============================] - 0s 70us/step - loss: 0.9028 - accuracy: 0.6667 - val_loss: 1.1022 - val_accuracy: 0.6325
Epoch 841/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8060 - accuracy: 0.6963 - val_loss: 0.9076 - val_accuracy: 0.5983
Epoch 842/1000
270/270 [==============================] - 0s 58us/step - loss: 0.7850 - accuracy: 0.6852 - val_loss: 1.0285 - val_accuracy: 0.6752
Epoch 843/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8243 - accuracy: 0.6963 - val_loss: 0

270/270 [==============================] - 0s 58us/step - loss: 0.8484 - accuracy: 0.6630 - val_loss: 1.1132 - val_accuracy: 0.6838
Epoch 948/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7950 - accuracy: 0.7037 - val_loss: 0.9372 - val_accuracy: 0.5812
Epoch 949/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8172 - accuracy: 0.6630 - val_loss: 1.1329 - val_accuracy: 0.6410
Epoch 950/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8622 - accuracy: 0.6741 - val_loss: 0.8725 - val_accuracy: 0.6410
Epoch 951/1000
270/270 [==============================] - 0s 59us/step - loss: 1.0373 - accuracy: 0.6741 - val_loss: 0.9533 - val_accuracy: 0.6752
Epoch 952/1000
270/270 [==============================] - 0s 66us/step - loss: 0.7895 - accuracy: 0.6815 - val_loss: 1.1521 - val_accuracy: 0.6410
Epoch 953/1000
270/270 [==============================] - 0s 66us/step - loss: 0.8439 - accuracy: 0.6815 - val_loss: 0.8741 - val_acc

In [59]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

117/117 [==============================] - 0s 93us/step
over-sampling test accuracy: 68.38%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([2, 2, 2, 2, 0, 2, 1, 1, 1, 1, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 1,
       0, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 2, 2, 0, 0, 0, 1,
       2, 0, 2, 2, 2, 1, 0, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 2, 2, 1, 1, 0, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1,
       1, 0, 2, 1, 2, 1, 2, 0, 2, 2, 2, 2, 2, 1, 0, 0, 1, 1, 0, 2, 1, 2,
       0, 1, 2, 2, 0, 0, 2])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,BCH-SA-11,2,2
1,NRS161,1,2
2,BCH-SA-14,2,2
3,BCH-SA-11,2,2
4,CFBRSa49,1,0
...,...,...,...
112,NRS064,2,2
113,NRS266,2,2
114,NRS222,0,0
115,GA27,2,0


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,0.023214,0.002530,0.974256
1,0.020274,0.005509,0.974217
2,0.244668,0.097915,0.657416
3,0.023214,0.002530,0.974256
4,0.393953,0.226959,0.379088
...,...,...,...
112,0.004061,0.005477,0.990461
113,0.127922,0.046466,0.825612
114,0.696836,0.255523,0.047641
115,0.393953,0.226959,0.379088


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p006ST.csv", index = False,
         header=None)

In [88]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 389us/step - loss: 22.6280 - accuracy: 0.3593 - val_loss: 14.0377 - val_accuracy: 0.3846
Epoch 2/1000
270/270 [==============================] - 0s 58us/step - loss: 18.6573 - accuracy: 0.3556 - val_loss: 11.4446 - val_accuracy: 0.3846
Epoch 3/1000
270/270 [==============================] - 0s 70us/step - loss: 14.8883 - accuracy: 0.3556 - val_loss: 8.9187 - val_accuracy: 0.3846
Epoch 4/1000
270/270 [==============================] - 0s 69us/step - loss: 10.6455 - accuracy: 0.3556 - val_loss: 6.4705 - val_accuracy: 0.3846
Epoch 5/1000
270/270 [==============================] - 0s 67us/step - loss: 6.8513 - accuracy: 0.3741 - val_loss: 3.9568 - val_accuracy: 0.4444
Epoch 6/1000
270/270 [==============================] - 0s 73us/step - loss: 2.8718 - accuracy: 0.3370 - val_loss: 3.0208 - val_accuracy: 0.4103
Epoch 7/1000
270/270 [==============================] - 0s 69us/step - loss: 

Epoch 57/1000
270/270 [==============================] - 0s 51us/step - loss: 0.9746 - accuracy: 0.5370 - val_loss: 0.9874 - val_accuracy: 0.5726
Epoch 58/1000
270/270 [==============================] - 0s 53us/step - loss: 0.9688 - accuracy: 0.5259 - val_loss: 1.0037 - val_accuracy: 0.5897
Epoch 59/1000
270/270 [==============================] - 0s 53us/step - loss: 0.9676 - accuracy: 0.5407 - val_loss: 0.9346 - val_accuracy: 0.5726
Epoch 60/1000
270/270 [==============================] - 0s 62us/step - loss: 0.9476 - accuracy: 0.5444 - val_loss: 0.9381 - val_accuracy: 0.5726
Epoch 61/1000
270/270 [==============================] - 0s 58us/step - loss: 0.9512 - accuracy: 0.5407 - val_loss: 0.9486 - val_accuracy: 0.5983
Epoch 62/1000
270/270 [==============================] - 0s 65us/step - loss: 0.9638 - accuracy: 0.5630 - val_loss: 0.9628 - val_accuracy: 0.5983
Epoch 63/1000
270/270 [==============================] - 0s 55us/step - loss: 0.9503 - accuracy: 0.5889 - val_loss: 0.9274 -

Epoch 113/1000
270/270 [==============================] - 0s 58us/step - loss: 1.0637 - accuracy: 0.5963 - val_loss: 1.0996 - val_accuracy: 0.5726
Epoch 114/1000
270/270 [==============================] - 0s 73us/step - loss: 0.9701 - accuracy: 0.5556 - val_loss: 0.9102 - val_accuracy: 0.5983
Epoch 115/1000
270/270 [==============================] - 0s 48us/step - loss: 0.9457 - accuracy: 0.5259 - val_loss: 1.2000 - val_accuracy: 0.6325
Epoch 116/1000
270/270 [==============================] - 0s 62us/step - loss: 1.0702 - accuracy: 0.6111 - val_loss: 1.1222 - val_accuracy: 0.6239
Epoch 117/1000
270/270 [==============================] - 0s 82us/step - loss: 0.9496 - accuracy: 0.5444 - val_loss: 0.8690 - val_accuracy: 0.5897
Epoch 118/1000
270/270 [==============================] - 0s 81us/step - loss: 0.9523 - accuracy: 0.5481 - val_loss: 1.0635 - val_accuracy: 0.5897
Epoch 119/1000
270/270 [==============================] - 0s 79us/step - loss: 0.9812 - accuracy: 0.5815 - val_loss: 0

Epoch 169/1000
270/270 [==============================] - 0s 58us/step - loss: 0.9130 - accuracy: 0.5963 - val_loss: 0.8504 - val_accuracy: 0.6154
Epoch 170/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8597 - accuracy: 0.6148 - val_loss: 0.9139 - val_accuracy: 0.6154
Epoch 171/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8704 - accuracy: 0.6185 - val_loss: 0.8491 - val_accuracy: 0.6325
Epoch 172/1000
270/270 [==============================] - 0s 55us/step - loss: 0.8617 - accuracy: 0.6111 - val_loss: 0.8346 - val_accuracy: 0.6325
Epoch 173/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8538 - accuracy: 0.6296 - val_loss: 0.8512 - val_accuracy: 0.6496
Epoch 174/1000
270/270 [==============================] - 0s 52us/step - loss: 0.8493 - accuracy: 0.6259 - val_loss: 0.8435 - val_accuracy: 0.6325
Epoch 175/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8503 - accuracy: 0.6148 - val_loss: 0

Epoch 225/1000
270/270 [==============================] - 0s 71us/step - loss: 0.8586 - accuracy: 0.6370 - val_loss: 0.9043 - val_accuracy: 0.6496
Epoch 226/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8558 - accuracy: 0.6519 - val_loss: 0.8692 - val_accuracy: 0.6667
Epoch 227/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8334 - accuracy: 0.6407 - val_loss: 0.8489 - val_accuracy: 0.6667
Epoch 228/1000
270/270 [==============================] - 0s 81us/step - loss: 0.8343 - accuracy: 0.6370 - val_loss: 0.8318 - val_accuracy: 0.6410
Epoch 229/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8238 - accuracy: 0.6148 - val_loss: 0.8868 - val_accuracy: 0.6667
Epoch 230/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8598 - accuracy: 0.6259 - val_loss: 0.8614 - val_accuracy: 0.6496
Epoch 231/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8379 - accuracy: 0.6185 - val_loss: 0

...

Epoch 671/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7428 - accuracy: 0.6852 - val_loss: 0.8054 - val_accuracy: 0.6581
Epoch 672/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7606 - accuracy: 0.6778 - val_loss: 0.8676 - val_accuracy: 0.6581
Epoch 673/1000
270/270 [==============================] - 0s 42us/step - loss: 0.7617 - accuracy: 0.6667 - val_loss: 0.8952 - val_accuracy: 0.6667
Epoch 674/1000
270/270 [==============================] - 0s 48us/step - loss: 0.7413 - accuracy: 0.6704 - val_loss: 0.8062 - val_accuracy: 0.6581
Epoch 675/1000
270/270 [==============================] - 0s 53us/step - loss: 0.7416 - accuracy: 0.6889 - val_loss: 0.8133 - val_accuracy: 0.6667
Epoch 676/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7283 - accuracy: 0.6889 - val_loss: 0.8134 - val_accuracy: 0.6667
Epoch 677/1000
270/270 [==============================] - 0s 52us/step - loss: 0.7361 - accuracy: 0.6963 - val_loss: 0

Epoch 727/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7456 - accuracy: 0.6963 - val_loss: 0.8134 - val_accuracy: 0.6667
Epoch 728/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7666 - accuracy: 0.6741 - val_loss: 0.9616 - val_accuracy: 0.6667
Epoch 729/1000
270/270 [==============================] - 0s 52us/step - loss: 0.7763 - accuracy: 0.6926 - val_loss: 0.8486 - val_accuracy: 0.6496
Epoch 730/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7221 - accuracy: 0.6963 - val_loss: 0.8087 - val_accuracy: 0.6667
Epoch 731/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7277 - accuracy: 0.6963 - val_loss: 0.8105 - val_accuracy: 0.6752
Epoch 732/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7562 - accuracy: 0.7000 - val_loss: 1.0337 - val_accuracy: 0.6667
Epoch 733/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8769 - accuracy: 0.6889 - val_loss: 1

Epoch 838/1000
270/270 [==============================] - 0s 57us/step - loss: 0.7410 - accuracy: 0.6556 - val_loss: 0.8515 - val_accuracy: 0.6838
Epoch 839/1000
270/270 [==============================] - 0s 56us/step - loss: 0.7356 - accuracy: 0.7037 - val_loss: 0.8302 - val_accuracy: 0.6752
Epoch 840/1000
270/270 [==============================] - 0s 54us/step - loss: 0.7299 - accuracy: 0.7074 - val_loss: 0.8176 - val_accuracy: 0.6752
Epoch 841/1000
270/270 [==============================] - 0s 58us/step - loss: 0.7142 - accuracy: 0.6963 - val_loss: 0.8448 - val_accuracy: 0.6923
Epoch 842/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7267 - accuracy: 0.6852 - val_loss: 0.8216 - val_accuracy: 0.6838
Epoch 843/1000
270/270 [==============================] - 0s 53us/step - loss: 0.7288 - accuracy: 0.6963 - val_loss: 0.8402 - val_accuracy: 0.6838
Epoch 844/1000
270/270 [==============================] - 0s 54us/step - loss: 0.7161 - accuracy: 0.6963 - val_loss: 0

Epoch 894/1000
270/270 [==============================] - 0s 70us/step - loss: 0.7431 - accuracy: 0.6889 - val_loss: 0.8135 - val_accuracy: 0.6667
Epoch 895/1000
270/270 [==============================] - 0s 73us/step - loss: 0.7301 - accuracy: 0.6852 - val_loss: 0.8911 - val_accuracy: 0.6752
Epoch 896/1000
270/270 [==============================] - 0s 70us/step - loss: 0.7676 - accuracy: 0.6815 - val_loss: 0.9697 - val_accuracy: 0.6752
Epoch 897/1000
270/270 [==============================] - 0s 78us/step - loss: 0.7188 - accuracy: 0.7037 - val_loss: 0.8373 - val_accuracy: 0.6581
Epoch 898/1000
270/270 [==============================] - 0s 88us/step - loss: 0.8146 - accuracy: 0.6778 - val_loss: 1.0455 - val_accuracy: 0.6667
Epoch 899/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8198 - accuracy: 0.6852 - val_loss: 0.9788 - val_accuracy: 0.6752
Epoch 900/1000
270/270 [==============================] - 0s 82us/step - loss: 0.7609 - accuracy: 0.6889 - val_loss: 0

Epoch 950/1000
270/270 [==============================] - 0s 67us/step - loss: 0.7545 - accuracy: 0.6741 - val_loss: 0.9480 - val_accuracy: 0.6667
Epoch 951/1000
270/270 [==============================] - 0s 71us/step - loss: 0.7741 - accuracy: 0.7037 - val_loss: 0.9562 - val_accuracy: 0.6752
Epoch 952/1000
270/270 [==============================] - 0s 50us/step - loss: 0.7344 - accuracy: 0.7074 - val_loss: 0.8076 - val_accuracy: 0.6752
Epoch 953/1000
270/270 [==============================] - 0s 53us/step - loss: 0.7321 - accuracy: 0.6667 - val_loss: 0.8094 - val_accuracy: 0.6667
Epoch 954/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7115 - accuracy: 0.7148 - val_loss: 0.8434 - val_accuracy: 0.6923
Epoch 955/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7128 - accuracy: 0.7074 - val_loss: 0.8432 - val_accuracy: 0.6667
Epoch 956/1000
270/270 [==============================] - 0s 68us/step - loss: 0.7115 - accuracy: 0.6889 - val_loss: 0

In [89]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 64.63%


In [31]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [32]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [33]:
y_prob2 = df_proba2[df_proba2['phage']=='p0006kpresabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[2.32143250e-02, 2.53005160e-03, 9.74255600e-01],
       [2.02739520e-02, 5.50900260e-03, 9.74217060e-01],
       [2.44668160e-01, 9.79154300e-02, 6.57416400e-01],
       [2.32143250e-02, 2.53005160e-03, 9.74255600e-01],
       [3.93953000e-01, 2.26959060e-01, 3.79087950e-01],
       [5.28821050e-02, 2.40460450e-02, 9.23071800e-01],
       [4.07823200e-01, 4.79629460e-01, 1.12547280e-01],
       [3.10971440e-01, 4.59986750e-01, 2.29041800e-01],
       [1.64014620e-01, 8.20671600e-01, 1.53138050e-02],
       [3.50327800e-01, 4.53829140e-01, 1.95843040e-01],
       [6.77101300e-04, 1.93885830e-08, 9.99322900e-01],
       [3.50327800e-01, 4.53829140e-01, 1.95843040e-01],
       [1.87493620e-01, 3.79260060e-01, 4.33246300e-01],
       [6.19027100e-01, 3.51751950e-01, 2.92209890e-02],
       [3.50327800e-01, 4.53829140e-01, 1.95843040e-01],
       [1.22721900e-03, 5.06774380e-05, 9.98722140e-01],
       [1.40099730e-01, 1.68918730e-02, 8.43008400e-01],
       [3.80423700e-01, 6.30313

In [34]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.7876397107166339

In [35]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.7876397107166339

In [36]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [37]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [38]:
dat3

,0,test
0,SR4187,0
1,NRS177,0
2,NRS109,2
3,CFBREBSa131,2
4,SR4152,1
...,...,...
112,NRS110,2
113,CFBRSa25,1
114,834N,0
115,CFBREBSa114,1


In [39]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [40]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 582us/step - loss: 5.9427 - accuracy: 0.3259 - val_loss: 7.4440 - val_accuracy: 0.2991
Epoch 2/1000
270/270 [==============================] - 0s 69us/step - loss: 4.4709 - accuracy: 0.3889 - val_loss: 5.3584 - val_accuracy: 0.3590
Epoch 3/1000
270/270 [==============================] - 0s 131us/step - loss: 3.5127 - accuracy: 0.3815 - val_loss: 4.1276 - val_accuracy: 0.3590
Epoch 4/1000
270/270 [==============================] - 0s 78us/step - loss: 2.7177 - accuracy: 0.3963 - val_loss: 2.2060 - val_accuracy: 0.3590
Epoch 5/1000
270/270 [==============================] - 0s 92us/step - loss: 1.7836 - accuracy: 0.3926 - val_loss: 1.4081 - val_accuracy: 0.3162
Epoch 6/1000
270/270 [==============================] - 0s 119us/step - loss: 1.4302 - accuracy: 0.4556 - val_loss: 1.7357 - val_accuracy: 0.4530
Epoch 7/1000
270/270 [==============================] - 0s 75us/step - loss: 1.71

Epoch 57/1000
270/270 [==============================] - 0s 68us/step - loss: 1.1445 - accuracy: 0.5630 - val_loss: 1.3148 - val_accuracy: 0.4701
Epoch 58/1000
270/270 [==============================] - 0s 95us/step - loss: 1.1840 - accuracy: 0.5074 - val_loss: 1.2057 - val_accuracy: 0.5299
Epoch 59/1000
270/270 [==============================] - 0s 134us/step - loss: 1.0909 - accuracy: 0.5704 - val_loss: 1.4146 - val_accuracy: 0.5214
Epoch 60/1000
270/270 [==============================] - 0s 118us/step - loss: 1.1455 - accuracy: 0.5333 - val_loss: 1.0767 - val_accuracy: 0.5641
Epoch 61/1000
270/270 [==============================] - 0s 152us/step - loss: 1.0899 - accuracy: 0.5519 - val_loss: 1.0178 - val_accuracy: 0.5470
Epoch 62/1000
270/270 [==============================] - 0s 123us/step - loss: 0.9568 - accuracy: 0.5630 - val_loss: 1.1766 - val_accuracy: 0.5385
Epoch 63/1000
270/270 [==============================] - 0s 142us/step - loss: 0.9665 - accuracy: 0.5519 - val_loss: 1.1

Epoch 168/1000
270/270 [==============================] - 0s 82us/step - loss: 0.9433 - accuracy: 0.6148 - val_loss: 0.9560 - val_accuracy: 0.5470
Epoch 169/1000
270/270 [==============================] - 0s 103us/step - loss: 0.8983 - accuracy: 0.6111 - val_loss: 1.1838 - val_accuracy: 0.5385
Epoch 170/1000
270/270 [==============================] - 0s 50us/step - loss: 0.9052 - accuracy: 0.6333 - val_loss: 0.9414 - val_accuracy: 0.5556
Epoch 171/1000
270/270 [==============================] - 0s 51us/step - loss: 0.8613 - accuracy: 0.6519 - val_loss: 1.3537 - val_accuracy: 0.5385
Epoch 172/1000
270/270 [==============================] - 0s 47us/step - loss: 1.7346 - accuracy: 0.6222 - val_loss: 3.2927 - val_accuracy: 0.5470
Epoch 173/1000
270/270 [==============================] - 0s 70us/step - loss: 2.3068 - accuracy: 0.6000 - val_loss: 3.4223 - val_accuracy: 0.5385
Epoch 174/1000
270/270 [==============================] - 0s 68us/step - loss: 2.2466 - accuracy: 0.6222 - val_loss: 

Epoch 224/1000
270/270 [==============================] - 0s 85us/step - loss: 0.9820 - accuracy: 0.6556 - val_loss: 1.2667 - val_accuracy: 0.5556
Epoch 225/1000
270/270 [==============================] - 0s 60us/step - loss: 0.9625 - accuracy: 0.6444 - val_loss: 0.9459 - val_accuracy: 0.5726
Epoch 226/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7781 - accuracy: 0.6556 - val_loss: 1.0378 - val_accuracy: 0.5726
Epoch 227/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8524 - accuracy: 0.6704 - val_loss: 0.9478 - val_accuracy: 0.5726
Epoch 228/1000
270/270 [==============================] - 0s 76us/step - loss: 0.7952 - accuracy: 0.6630 - val_loss: 0.9248 - val_accuracy: 0.5812
Epoch 229/1000
270/270 [==============================] - 0s 51us/step - loss: 0.7827 - accuracy: 0.6741 - val_loss: 0.8988 - val_accuracy: 0.5726
Epoch 230/1000
270/270 [==============================] - 0s 52us/step - loss: 0.7949 - accuracy: 0.6519 - val_loss: 0

Epoch 335/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8363 - accuracy: 0.6593 - val_loss: 1.0332 - val_accuracy: 0.5726
Epoch 336/1000
270/270 [==============================] - 0s 92us/step - loss: 0.8010 - accuracy: 0.6889 - val_loss: 0.9481 - val_accuracy: 0.5726
Epoch 337/1000
270/270 [==============================] - 0s 89us/step - loss: 0.7607 - accuracy: 0.6741 - val_loss: 0.8937 - val_accuracy: 0.5812
Epoch 338/1000
270/270 [==============================] - 0s 71us/step - loss: 0.7440 - accuracy: 0.6852 - val_loss: 0.9176 - val_accuracy: 0.5641
Epoch 339/1000
270/270 [==============================] - 0s 56us/step - loss: 0.7234 - accuracy: 0.6741 - val_loss: 0.9434 - val_accuracy: 0.5726
Epoch 340/1000
270/270 [==============================] - 0s 76us/step - loss: 0.7691 - accuracy: 0.6593 - val_loss: 0.9128 - val_accuracy: 0.6068
Epoch 341/1000
270/270 [==============================] - 0s 86us/step - loss: 0.7249 - accuracy: 0.6889 - val_loss: 0

...

Epoch 727/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7514 - accuracy: 0.6889 - val_loss: 1.0095 - val_accuracy: 0.5641
Epoch 728/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7405 - accuracy: 0.6889 - val_loss: 1.2199 - val_accuracy: 0.5983
Epoch 729/1000
270/270 [==============================] - 0s 55us/step - loss: 1.2580 - accuracy: 0.6963 - val_loss: 0.9412 - val_accuracy: 0.6154
Epoch 730/1000
270/270 [==============================] - 0s 56us/step - loss: 0.7087 - accuracy: 0.7000 - val_loss: 1.0272 - val_accuracy: 0.6068
Epoch 731/1000
270/270 [==============================] - 0s 63us/step - loss: 0.6722 - accuracy: 0.7148 - val_loss: 1.0102 - val_accuracy: 0.5726
Epoch 732/1000
270/270 [==============================] - 0s 67us/step - loss: 0.6672 - accuracy: 0.7370 - val_loss: 0.9006 - val_accuracy: 0.5897
Epoch 733/1000
270/270 [==============================] - 0s 62us/step - loss: 0.6470 - accuracy: 0.7333 - val_loss: 0

Epoch 783/1000
270/270 [==============================] - 0s 65us/step - loss: 0.6718 - accuracy: 0.7000 - val_loss: 0.9579 - val_accuracy: 0.6410
Epoch 784/1000
270/270 [==============================] - 0s 62us/step - loss: 0.6394 - accuracy: 0.7444 - val_loss: 0.9811 - val_accuracy: 0.6154
Epoch 785/1000
270/270 [==============================] - 0s 64us/step - loss: 0.6287 - accuracy: 0.7407 - val_loss: 0.9143 - val_accuracy: 0.5983
Epoch 786/1000
270/270 [==============================] - 0s 72us/step - loss: 0.6829 - accuracy: 0.7111 - val_loss: 0.9298 - val_accuracy: 0.5983
Epoch 787/1000
270/270 [==============================] - 0s 60us/step - loss: 0.6353 - accuracy: 0.7296 - val_loss: 1.0585 - val_accuracy: 0.5897
Epoch 788/1000
270/270 [==============================] - 0s 64us/step - loss: 0.6940 - accuracy: 0.7259 - val_loss: 0.9575 - val_accuracy: 0.5812
Epoch 789/1000
270/270 [==============================] - 0s 70us/step - loss: 0.7249 - accuracy: 0.7000 - val_loss: 0

Epoch 839/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7192 - accuracy: 0.7222 - val_loss: 0.9598 - val_accuracy: 0.5641
Epoch 840/1000
270/270 [==============================] - 0s 68us/step - loss: 0.7724 - accuracy: 0.6741 - val_loss: 0.9180 - val_accuracy: 0.5641
Epoch 841/1000
270/270 [==============================] - 0s 62us/step - loss: 0.6344 - accuracy: 0.7222 - val_loss: 1.1442 - val_accuracy: 0.6154
Epoch 842/1000
270/270 [==============================] - 0s 81us/step - loss: 0.6899 - accuracy: 0.7333 - val_loss: 0.9776 - val_accuracy: 0.6154
Epoch 843/1000
270/270 [==============================] - 0s 60us/step - loss: 0.6558 - accuracy: 0.7222 - val_loss: 0.9584 - val_accuracy: 0.5897
Epoch 844/1000
270/270 [==============================] - 0s 66us/step - loss: 0.6485 - accuracy: 0.7185 - val_loss: 0.9800 - val_accuracy: 0.5983
Epoch 845/1000
270/270 [==============================] - 0s 72us/step - loss: 0.6425 - accuracy: 0.7481 - val_loss: 1

Epoch 895/1000
270/270 [==============================] - 0s 67us/step - loss: 0.6361 - accuracy: 0.7444 - val_loss: 1.0030 - val_accuracy: 0.5983
Epoch 896/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6364 - accuracy: 0.7333 - val_loss: 0.9155 - val_accuracy: 0.6496
Epoch 897/1000
270/270 [==============================] - 0s 64us/step - loss: 0.6107 - accuracy: 0.7704 - val_loss: 0.9289 - val_accuracy: 0.5983
Epoch 898/1000
270/270 [==============================] - 0s 57us/step - loss: 0.6137 - accuracy: 0.7593 - val_loss: 0.9383 - val_accuracy: 0.5897
Epoch 899/1000
270/270 [==============================] - 0s 63us/step - loss: 0.6353 - accuracy: 0.7111 - val_loss: 0.9142 - val_accuracy: 0.5983
Epoch 900/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6205 - accuracy: 0.7370 - val_loss: 0.9820 - val_accuracy: 0.6154
Epoch 901/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6317 - accuracy: 0.7556 - val_loss: 0

Epoch 951/1000
270/270 [==============================] - 0s 63us/step - loss: 0.6217 - accuracy: 0.7593 - val_loss: 0.9865 - val_accuracy: 0.6410
Epoch 952/1000
270/270 [==============================] - 0s 61us/step - loss: 0.6561 - accuracy: 0.7481 - val_loss: 1.1739 - val_accuracy: 0.6239
Epoch 953/1000
270/270 [==============================] - 0s 68us/step - loss: 0.8718 - accuracy: 0.6889 - val_loss: 1.2659 - val_accuracy: 0.6325
Epoch 954/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8162 - accuracy: 0.7148 - val_loss: 0.9951 - val_accuracy: 0.6325
Epoch 955/1000
270/270 [==============================] - 0s 76us/step - loss: 0.6749 - accuracy: 0.7296 - val_loss: 0.9384 - val_accuracy: 0.6154
Epoch 956/1000
270/270 [==============================] - 0s 56us/step - loss: 0.6491 - accuracy: 0.7296 - val_loss: 0.9554 - val_accuracy: 0.6410
Epoch 957/1000
270/270 [==============================] - 0s 73us/step - loss: 0.6350 - accuracy: 0.7556 - val_loss: 0

In [98]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

117/117 [==============================] - 0s 35us/step
over-sampling test accuracy: 61.54%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([0, 0, 2, 2, 1, 2, 1, 2, 0, 2, 0, 1, 1, 0, 1, 0, 2, 1, 0, 0, 1, 1,
       2, 2, 0, 0, 1, 0, 1, 1, 0, 2, 0, 1, 1, 0, 0, 0, 2, 1, 1, 0, 1, 2,
       2, 2, 0, 2, 1, 0, 2, 0, 2, 2, 1, 1, 2, 0, 0, 1, 2, 0, 1, 0, 1, 2,
       0, 0, 0, 0, 1, 1, 1, 0, 2, 1, 2, 0, 2, 1, 1, 2, 0, 2, 0, 1, 2, 0,
       1, 1, 2, 0, 2, 1, 2, 2, 1, 2, 2, 1, 1, 0, 1, 2, 0, 1, 0, 1, 2, 1,
       1, 0, 1, 1, 0, 1, 1])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,SR4187,0,0
1,NRS177,0,0
2,NRS109,2,2
3,CFBREBSa131,2,2
4,SR4152,1,1
...,...,...,...
112,NRS110,2,1
113,CFBRSa25,1,1
114,834N,0,0
115,CFBREBSa114,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,1.000000,3.582708e-12,3.055194e-32
1,0.579007,4.162575e-01,4.735081e-03
2,0.007004,4.206216e-01,5.723745e-01
3,0.022045,9.653085e-02,8.814240e-01
4,0.218927,6.354200e-01,1.456533e-01
...,...,...,...
112,0.218927,6.354200e-01,1.456533e-01
113,0.218927,6.354200e-01,1.456533e-01
114,0.891823,9.339423e-02,1.478260e-02
115,0.403542,4.730577e-01,1.234005e-01


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p006ST.csv", index = False,
         header=None)

In [103]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 116us/step - loss: 0.7394 - accuracy: 0.7148 - val_loss: 0.9625 - val_accuracy: 0.6068
Epoch 2/1000
270/270 [==============================] - 0s 130us/step - loss: 0.7180 - accuracy: 0.7185 - val_loss: 0.9138 - val_accuracy: 0.5897
Epoch 3/1000
270/270 [==============================] - 0s 114us/step - loss: 0.7410 - accuracy: 0.7037 - val_loss: 0.9342 - val_accuracy: 0.6239
Epoch 4/1000
270/270 [==============================] - 0s 101us/step - loss: 0.7706 - accuracy: 0.6963 - val_loss: 0.9613 - val_accuracy: 0.6154
Epoch 5/1000
270/270 [==============================] - 0s 85us/step - loss: 0.7077 - accuracy: 0.7259 - val_loss: 0.9559 - val_accuracy: 0.5897
Epoch 6/1000
270/270 [==============================] - 0s 90us/step - loss: 0.7564 - accuracy: 0.7000 - val_loss: 1.0571 - val_accuracy: 0.6154
Epoch 7/1000
270/270 [==============================] - 0s 86us/step - loss: 0.8

Epoch 57/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7713 - accuracy: 0.7259 - val_loss: 0.9810 - val_accuracy: 0.6154
Epoch 58/1000
270/270 [==============================] - 0s 72us/step - loss: 0.7797 - accuracy: 0.7111 - val_loss: 0.8654 - val_accuracy: 0.6325
Epoch 59/1000
270/270 [==============================] - 0s 54us/step - loss: 0.8054 - accuracy: 0.7074 - val_loss: 1.1141 - val_accuracy: 0.6410
Epoch 60/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7947 - accuracy: 0.7037 - val_loss: 1.0632 - val_accuracy: 0.5983
Epoch 61/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7777 - accuracy: 0.6704 - val_loss: 1.1258 - val_accuracy: 0.6325
Epoch 62/1000
270/270 [==============================] - 0s 63us/step - loss: 0.8971 - accuracy: 0.7148 - val_loss: 1.1812 - val_accuracy: 0.5726
Epoch 63/1000
270/270 [==============================] - ETA: 0s - loss: 0.9777 - accuracy: 0.68 - 0s 73us/step - loss: 0.82

Epoch 168/1000
270/270 [==============================] - 0s 77us/step - loss: 0.7001 - accuracy: 0.7111 - val_loss: 0.8540 - val_accuracy: 0.5983
Epoch 169/1000
270/270 [==============================] - 0s 93us/step - loss: 0.6835 - accuracy: 0.7222 - val_loss: 0.8905 - val_accuracy: 0.5983
Epoch 170/1000
270/270 [==============================] - 0s 91us/step - loss: 0.6846 - accuracy: 0.7259 - val_loss: 0.8480 - val_accuracy: 0.6410
Epoch 171/1000
270/270 [==============================] - 0s 120us/step - loss: 0.6704 - accuracy: 0.7148 - val_loss: 0.8427 - val_accuracy: 0.6410
Epoch 172/1000
270/270 [==============================] - 0s 67us/step - loss: 0.7168 - accuracy: 0.7037 - val_loss: 0.8776 - val_accuracy: 0.6325
Epoch 173/1000
270/270 [==============================] - 0s 68us/step - loss: 0.6802 - accuracy: 0.7111 - val_loss: 0.9074 - val_accuracy: 0.5983
Epoch 174/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6820 - accuracy: 0.7370 - val_loss: 

Epoch 224/1000
270/270 [==============================] - 0s 66us/step - loss: 0.6710 - accuracy: 0.7407 - val_loss: 0.8901 - val_accuracy: 0.6410
Epoch 225/1000
270/270 [==============================] - 0s 82us/step - loss: 0.6781 - accuracy: 0.7148 - val_loss: 0.8490 - val_accuracy: 0.6154
Epoch 226/1000
270/270 [==============================] - 0s 65us/step - loss: 0.6588 - accuracy: 0.7556 - val_loss: 0.8628 - val_accuracy: 0.6410
Epoch 227/1000
270/270 [==============================] - 0s 74us/step - loss: 0.6620 - accuracy: 0.7370 - val_loss: 0.8562 - val_accuracy: 0.6496
Epoch 228/1000
270/270 [==============================] - 0s 67us/step - loss: 0.6677 - accuracy: 0.7296 - val_loss: 0.8846 - val_accuracy: 0.6239
Epoch 229/1000
270/270 [==============================] - 0s 70us/step - loss: 0.6712 - accuracy: 0.7259 - val_loss: 0.8836 - val_accuracy: 0.6325
Epoch 230/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8028 - accuracy: 0.7185 - val_loss: 1

Epoch 280/1000
270/270 [==============================] - 0s 78us/step - loss: 0.7235 - accuracy: 0.7370 - val_loss: 0.9291 - val_accuracy: 0.6068
Epoch 281/1000
270/270 [==============================] - ETA: 0s - loss: 0.6386 - accuracy: 0.75 - 0s 68us/step - loss: 0.6777 - accuracy: 0.7407 - val_loss: 0.8761 - val_accuracy: 0.6239
Epoch 282/1000
270/270 [==============================] - 0s 67us/step - loss: 0.7484 - accuracy: 0.6926 - val_loss: 0.8454 - val_accuracy: 0.6581
Epoch 283/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6967 - accuracy: 0.7259 - val_loss: 0.9924 - val_accuracy: 0.5641
Epoch 284/1000
270/270 [==============================] - 0s 63us/step - loss: 0.6831 - accuracy: 0.7370 - val_loss: 0.8694 - val_accuracy: 0.6154
Epoch 285/1000
270/270 [==============================] - 0s 74us/step - loss: 0.6982 - accuracy: 0.6963 - val_loss: 0.8930 - val_accuracy: 0.6154
Epoch 286/1000
270/270 [==============================] - 0s 68us/step - los

...

270/270 [==============================] - 0s 168us/step - loss: 0.6266 - accuracy: 0.7407 - val_loss: 0.8521 - val_accuracy: 0.6581
Epoch 669/1000
270/270 [==============================] - 0s 114us/step - loss: 0.6284 - accuracy: 0.7481 - val_loss: 0.8981 - val_accuracy: 0.6325
Epoch 670/1000
270/270 [==============================] - 0s 112us/step - loss: 0.6818 - accuracy: 0.7370 - val_loss: 1.1930 - val_accuracy: 0.6239
Epoch 671/1000
270/270 [==============================] - 0s 187us/step - loss: 0.7612 - accuracy: 0.7259 - val_loss: 0.9525 - val_accuracy: 0.6154
Epoch 672/1000
270/270 [==============================] - 0s 149us/step - loss: 0.7059 - accuracy: 0.7259 - val_loss: 0.8797 - val_accuracy: 0.6068
Epoch 673/1000
270/270 [==============================] - 0s 160us/step - loss: 0.6748 - accuracy: 0.7333 - val_loss: 0.9415 - val_accuracy: 0.6325
Epoch 674/1000
270/270 [==============================] - 0s 143us/step - loss: 0.6265 - accuracy: 0.7407 - val_loss: 0.9360 - 

270/270 [==============================] - 0s 74us/step - loss: 0.6347 - accuracy: 0.7444 - val_loss: 0.8745 - val_accuracy: 0.6325
Epoch 724/1000
270/270 [==============================] - 0s 101us/step - loss: 0.6297 - accuracy: 0.7444 - val_loss: 0.8736 - val_accuracy: 0.6068
Epoch 725/1000
270/270 [==============================] - 0s 159us/step - loss: 0.6331 - accuracy: 0.7259 - val_loss: 0.8811 - val_accuracy: 0.6325
Epoch 726/1000
270/270 [==============================] - 0s 128us/step - loss: 0.6233 - accuracy: 0.7481 - val_loss: 0.8897 - val_accuracy: 0.6068
Epoch 727/1000
270/270 [==============================] - 0s 137us/step - loss: 0.6174 - accuracy: 0.7519 - val_loss: 0.8744 - val_accuracy: 0.6410
Epoch 728/1000
270/270 [==============================] - 0s 272us/step - loss: 0.6285 - accuracy: 0.7444 - val_loss: 0.8656 - val_accuracy: 0.6410
Epoch 729/1000
270/270 [==============================] - 0s 130us/step - loss: 0.6362 - accuracy: 0.7407 - val_loss: 0.8955 - v

270/270 [==============================] - 0s 124us/step - loss: 0.6352 - accuracy: 0.7222 - val_loss: 0.9266 - val_accuracy: 0.6154
Epoch 834/1000
270/270 [==============================] - 0s 129us/step - loss: 0.6391 - accuracy: 0.7444 - val_loss: 0.9235 - val_accuracy: 0.6154
Epoch 835/1000
270/270 [==============================] - 0s 150us/step - loss: 0.6409 - accuracy: 0.7333 - val_loss: 0.8867 - val_accuracy: 0.6068
Epoch 836/1000
270/270 [==============================] - 0s 120us/step - loss: 0.6242 - accuracy: 0.7407 - val_loss: 0.8717 - val_accuracy: 0.6325
Epoch 837/1000
270/270 [==============================] - 0s 103us/step - loss: 0.6162 - accuracy: 0.7444 - val_loss: 0.8552 - val_accuracy: 0.6410
Epoch 838/1000
270/270 [==============================] - 0s 121us/step - loss: 0.6156 - accuracy: 0.7481 - val_loss: 0.8784 - val_accuracy: 0.6154
Epoch 839/1000
270/270 [==============================] - 0s 191us/step - loss: 0.6163 - accuracy: 0.7556 - val_loss: 0.8757 - 

270/270 [==============================] - 0s 124us/step - loss: 0.6485 - accuracy: 0.7296 - val_loss: 0.8910 - val_accuracy: 0.6325
Epoch 944/1000
270/270 [==============================] - 0s 118us/step - loss: 0.6502 - accuracy: 0.7481 - val_loss: 0.9034 - val_accuracy: 0.6154
Epoch 945/1000
270/270 [==============================] - 0s 169us/step - loss: 0.6194 - accuracy: 0.7370 - val_loss: 0.8923 - val_accuracy: 0.6154
Epoch 946/1000
270/270 [==============================] - 0s 313us/step - loss: 0.6294 - accuracy: 0.7556 - val_loss: 0.9974 - val_accuracy: 0.5812
Epoch 947/1000
270/270 [==============================] - 0s 251us/step - loss: 0.6284 - accuracy: 0.7667 - val_loss: 0.9237 - val_accuracy: 0.6239
Epoch 948/1000
270/270 [==============================] - 0s 80us/step - loss: 0.6806 - accuracy: 0.7296 - val_loss: 0.8804 - val_accuracy: 0.6325
Epoch 949/1000
270/270 [==============================] - 0s 67us/step - loss: 0.6972 - accuracy: 0.6926 - val_loss: 0.9099 - va

Epoch 999/1000
270/270 [==============================] - 0s 94us/step - loss: 0.6252 - accuracy: 0.7556 - val_loss: 0.9039 - val_accuracy: 0.6068
Epoch 1000/1000
270/270 [==============================] - 0s 82us/step - loss: 0.6146 - accuracy: 0.7444 - val_loss: 0.9147 - val_accuracy: 0.6154


In [104]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 72.40%


In [41]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [42]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [43]:
y_prob3 = df_proba3[df_proba3['phage']=='p0006kpresabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[1.00000000e+00, 3.58270770e-12, 3.05519420e-32],
       [5.79007450e-01, 4.16257500e-01, 4.73508150e-03],
       [7.00390250e-03, 4.20621600e-01, 5.72374500e-01],
       [2.20451350e-02, 9.65308500e-02, 8.81423950e-01],
       [2.18926730e-01, 6.35420000e-01, 1.45653280e-01],
       [2.20507850e-03, 3.14842800e-03, 9.94646500e-01],
       [2.18926730e-01, 6.35420000e-01, 1.45653280e-01],
       [2.20451350e-02, 9.65308500e-02, 8.81423950e-01],
       [7.01771260e-01, 2.03304290e-01, 9.49244500e-02],
       [2.20451350e-02, 9.65308500e-02, 8.81423950e-01],
       [6.13006000e-01, 2.22979130e-01, 1.64014820e-01],
       [4.34442550e-01, 5.58801900e-01, 6.75554400e-03],
       [1.28606190e-01, 4.65022770e-01, 4.06371030e-01],
       [5.73673840e-01, 3.75396500e-01, 5.09297030e-02],
       [7.22713300e-02, 9.27728350e-01, 3.11534250e-07],
       [7.70858940e-01, 1.99726660e-01, 2.94143670e-02],
       [1.00141570e-03, 8.93397300e-03, 9.90064700e-01],
       [4.03541770e-01, 4.73057

In [44]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.794214332675871

In [45]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.794214332675871

In [46]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [47]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [48]:
dat4

,0,test
0,CFBRSa25,1
1,CFBRSa07,0
2,NRS247,0
3,NY439,2
4,CFBREBSa110,1
...,...,...
112,SR1129,0
113,NRS172,0
114,NRS205,2
115,NY439,2


In [49]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [57]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 494us/step - loss: 20.6109 - accuracy: 0.3444 - val_loss: 10.8528 - val_accuracy: 0.3162
Epoch 2/1000
270/270 [==============================] - 0s 97us/step - loss: 16.2454 - accuracy: 0.3444 - val_loss: 8.5011 - val_accuracy: 0.3162
Epoch 3/1000
270/270 [==============================] - 0s 59us/step - loss: 12.1546 - accuracy: 0.3407 - val_loss: 6.7370 - val_accuracy: 0.3846
Epoch 4/1000
270/270 [==============================] - 0s 77us/step - loss: 9.6727 - accuracy: 0.3333 - val_loss: 6.9285 - val_accuracy: 0.4615
Epoch 5/1000
270/270 [==============================] - 0s 97us/step - loss: 9.3167 - accuracy: 0.4000 - val_loss: 6.4189 - val_accuracy: 0.4615
Epoch 6/1000
270/270 [==============================] - 0s 122us/step - loss: 8.4144 - accuracy: 0.4074 - val_loss: 5.5204 - val_accuracy: 0.4615
Epoch 7/1000
270/270 [==============================] - 0s 82us/step - loss: 7

Epoch 57/1000
270/270 [==============================] - 0s 69us/step - loss: 1.1689 - accuracy: 0.5889 - val_loss: 1.1299 - val_accuracy: 0.5385
Epoch 58/1000
270/270 [==============================] - 0s 59us/step - loss: 0.9860 - accuracy: 0.5852 - val_loss: 1.1243 - val_accuracy: 0.5214
Epoch 59/1000
270/270 [==============================] - 0s 103us/step - loss: 1.1285 - accuracy: 0.5222 - val_loss: 1.1082 - val_accuracy: 0.5470
Epoch 60/1000
270/270 [==============================] - 0s 78us/step - loss: 1.0950 - accuracy: 0.5815 - val_loss: 1.0600 - val_accuracy: 0.5128
Epoch 61/1000
270/270 [==============================] - 0s 93us/step - loss: 0.9796 - accuracy: 0.5407 - val_loss: 1.0755 - val_accuracy: 0.5128
Epoch 62/1000
270/270 [==============================] - 0s 105us/step - loss: 1.0639 - accuracy: 0.5481 - val_loss: 1.1134 - val_accuracy: 0.5470
Epoch 63/1000
270/270 [==============================] - 0s 57us/step - loss: 1.0536 - accuracy: 0.5889 - val_loss: 1.0247

Epoch 113/1000
270/270 [==============================] - 0s 68us/step - loss: 0.9274 - accuracy: 0.6074 - val_loss: 0.9881 - val_accuracy: 0.5641
Epoch 114/1000
270/270 [==============================] - 0s 53us/step - loss: 0.9210 - accuracy: 0.5963 - val_loss: 1.0149 - val_accuracy: 0.5470
Epoch 115/1000
270/270 [==============================] - 0s 48us/step - loss: 0.9234 - accuracy: 0.6000 - val_loss: 1.0112 - val_accuracy: 0.5726
Epoch 116/1000
270/270 [==============================] - 0s 54us/step - loss: 0.8977 - accuracy: 0.6074 - val_loss: 1.0047 - val_accuracy: 0.5641
Epoch 117/1000
270/270 [==============================] - 0s 74us/step - loss: 0.9228 - accuracy: 0.6222 - val_loss: 0.9996 - val_accuracy: 0.5641
Epoch 118/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8974 - accuracy: 0.6222 - val_loss: 0.9877 - val_accuracy: 0.5556
Epoch 119/1000
270/270 [==============================] - 0s 54us/step - loss: 0.8852 - accuracy: 0.6370 - val_loss: 0

Epoch 169/1000
270/270 [==============================] - 0s 102us/step - loss: 1.2215 - accuracy: 0.6037 - val_loss: 0.9846 - val_accuracy: 0.5641
Epoch 170/1000
270/270 [==============================] - 0s 172us/step - loss: 0.9233 - accuracy: 0.6222 - val_loss: 1.1958 - val_accuracy: 0.6239
Epoch 171/1000
270/270 [==============================] - 0s 89us/step - loss: 1.1523 - accuracy: 0.6185 - val_loss: 1.0909 - val_accuracy: 0.5470
Epoch 172/1000
270/270 [==============================] - 0s 168us/step - loss: 0.9485 - accuracy: 0.6037 - val_loss: 1.0260 - val_accuracy: 0.5214
Epoch 173/1000
270/270 [==============================] - 0s 103us/step - loss: 0.8661 - accuracy: 0.6370 - val_loss: 0.9958 - val_accuracy: 0.5641
Epoch 174/1000
270/270 [==============================] - 0s 88us/step - loss: 0.8711 - accuracy: 0.6222 - val_loss: 0.9983 - val_accuracy: 0.5641
Epoch 175/1000
270/270 [==============================] - 0s 60us/step - loss: 0.8629 - accuracy: 0.6333 - val_los

Epoch 280/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8086 - accuracy: 0.6704 - val_loss: 0.9778 - val_accuracy: 0.6154
Epoch 281/1000
270/270 [==============================] - 0s 82us/step - loss: 0.8481 - accuracy: 0.6519 - val_loss: 0.9807 - val_accuracy: 0.6068
Epoch 282/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8050 - accuracy: 0.6741 - val_loss: 0.9735 - val_accuracy: 0.5556
Epoch 283/1000
270/270 [==============================] - 0s 68us/step - loss: 0.7966 - accuracy: 0.6704 - val_loss: 0.9867 - val_accuracy: 0.6239
Epoch 284/1000
270/270 [==============================] - 0s 65us/step - loss: 0.8200 - accuracy: 0.6630 - val_loss: 0.9721 - val_accuracy: 0.5556
Epoch 285/1000
270/270 [==============================] - 0s 66us/step - loss: 0.8119 - accuracy: 0.6741 - val_loss: 0.9753 - val_accuracy: 0.5641
Epoch 286/1000
270/270 [==============================] - 0s 145us/step - loss: 0.8237 - accuracy: 0.6630 - val_loss: 

...

Epoch 672/1000
270/270 [==============================] - 0s 50us/step - loss: 0.9930 - accuracy: 0.6852 - val_loss: 1.0828 - val_accuracy: 0.6325
Epoch 673/1000
270/270 [==============================] - 0s 51us/step - loss: 0.8330 - accuracy: 0.6741 - val_loss: 1.0925 - val_accuracy: 0.5726
Epoch 674/1000
270/270 [==============================] - 0s 50us/step - loss: 0.7979 - accuracy: 0.6630 - val_loss: 1.1039 - val_accuracy: 0.6239
Epoch 675/1000
270/270 [==============================] - 0s 53us/step - loss: 0.8612 - accuracy: 0.6667 - val_loss: 1.0575 - val_accuracy: 0.6239
Epoch 676/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7751 - accuracy: 0.6889 - val_loss: 1.0677 - val_accuracy: 0.6068
Epoch 677/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7419 - accuracy: 0.6889 - val_loss: 1.0330 - val_accuracy: 0.5897
Epoch 678/1000
270/270 [==============================] - 0s 51us/step - loss: 0.7835 - accuracy: 0.6889 - val_loss: 1

Epoch 728/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7570 - accuracy: 0.6963 - val_loss: 1.0241 - val_accuracy: 0.5641
Epoch 729/1000
270/270 [==============================] - 0s 81us/step - loss: 0.7313 - accuracy: 0.7037 - val_loss: 1.0308 - val_accuracy: 0.5470
Epoch 730/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7235 - accuracy: 0.6963 - val_loss: 1.0254 - val_accuracy: 0.5812
Epoch 731/1000
270/270 [==============================] - 0s 75us/step - loss: 0.7303 - accuracy: 0.7037 - val_loss: 1.0206 - val_accuracy: 0.5726
Epoch 732/1000
270/270 [==============================] - 0s 94us/step - loss: 0.7254 - accuracy: 0.7074 - val_loss: 1.0242 - val_accuracy: 0.5641
Epoch 733/1000
270/270 [==============================] - 0s 81us/step - loss: 0.7262 - accuracy: 0.6926 - val_loss: 1.0259 - val_accuracy: 0.5726
Epoch 734/1000
270/270 [==============================] - 0s 69us/step - loss: 0.7227 - accuracy: 0.6963 - val_loss: 1

Epoch 839/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8201 - accuracy: 0.6815 - val_loss: 1.0933 - val_accuracy: 0.5983
Epoch 840/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7501 - accuracy: 0.6778 - val_loss: 1.0732 - val_accuracy: 0.6325
Epoch 841/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7841 - accuracy: 0.7000 - val_loss: 1.0606 - val_accuracy: 0.5983
Epoch 842/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7275 - accuracy: 0.7111 - val_loss: 1.0702 - val_accuracy: 0.5897
Epoch 843/1000
270/270 [==============================] - 0s 64us/step - loss: 0.7206 - accuracy: 0.7074 - val_loss: 1.0507 - val_accuracy: 0.6154
Epoch 844/1000
270/270 [==============================] - 0s 70us/step - loss: 0.7213 - accuracy: 0.7259 - val_loss: 1.0318 - val_accuracy: 0.5641
Epoch 845/1000
270/270 [==============================] - 0s 64us/step - loss: 0.7238 - accuracy: 0.7000 - val_loss: 1

Epoch 895/1000
270/270 [==============================] - 0s 53us/step - loss: 0.7075 - accuracy: 0.7185 - val_loss: 1.0262 - val_accuracy: 0.5812
Epoch 896/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7209 - accuracy: 0.7000 - val_loss: 1.0244 - val_accuracy: 0.5812
Epoch 897/1000
270/270 [==============================] - 0s 57us/step - loss: 0.7104 - accuracy: 0.7148 - val_loss: 1.0412 - val_accuracy: 0.5812
Epoch 898/1000
270/270 [==============================] - 0s 72us/step - loss: 0.7131 - accuracy: 0.7111 - val_loss: 1.0230 - val_accuracy: 0.6410
Epoch 899/1000
270/270 [==============================] - 0s 66us/step - loss: 0.7330 - accuracy: 0.6963 - val_loss: 1.0219 - val_accuracy: 0.5812
Epoch 900/1000
270/270 [==============================] - 0s 56us/step - loss: 0.7185 - accuracy: 0.7037 - val_loss: 1.0127 - val_accuracy: 0.5983
Epoch 901/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7141 - accuracy: 0.7296 - val_loss: 1

Epoch 951/1000
270/270 [==============================] - 0s 76us/step - loss: 0.7099 - accuracy: 0.7074 - val_loss: 1.0335 - val_accuracy: 0.5812
Epoch 952/1000
270/270 [==============================] - 0s 69us/step - loss: 0.7079 - accuracy: 0.7111 - val_loss: 1.0355 - val_accuracy: 0.6410
Epoch 953/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7190 - accuracy: 0.7148 - val_loss: 1.0369 - val_accuracy: 0.6496
Epoch 954/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7073 - accuracy: 0.7111 - val_loss: 1.0435 - val_accuracy: 0.5983
Epoch 955/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7168 - accuracy: 0.7000 - val_loss: 1.0231 - val_accuracy: 0.6410
Epoch 956/1000
270/270 [==============================] - 0s 72us/step - loss: 0.7106 - accuracy: 0.7074 - val_loss: 1.0233 - val_accuracy: 0.5726
Epoch 957/1000
270/270 [==============================] - 0s 67us/step - loss: 0.7113 - accuracy: 0.6963 - val_loss: 1

In [124]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

117/117 [==============================] - 0s 83us/step
over-sampling test accuracy: 60.68%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 1, 2, 2, 2, 1, 1, 2, 2,
       1, 1, 0, 1, 0, 2, 1, 2, 2, 1, 0, 2, 2, 2, 1, 1, 0, 1, 2, 2, 0, 1,
       1, 2, 0, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 0, 2, 1, 0, 1, 1, 2, 2, 1,
       2, 2, 1, 2, 0, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 0, 0, 2, 2, 0, 1,
       2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 1, 1, 2, 0, 1, 1, 2, 1, 1, 2, 2, 1,
       0, 1, 0, 2, 2, 2, 2])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,CFBRSa25,1,1
1,CFBRSa07,0,0
2,NRS247,0,0
3,NY439,2,2
4,CFBREBSa110,1,0
...,...,...,...
112,SR1129,0,0
113,NRS172,0,2
114,NRS205,2,2
115,NY439,2,2


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,0.273088,0.561901,0.165011
1,0.612003,0.292165,0.095832
2,0.703594,0.286382,0.010025
3,0.000912,0.002892,0.996195
4,0.901155,0.043834,0.055011
...,...,...,...
112,0.588236,0.231665,0.180099
113,0.349461,0.234379,0.416160
114,0.014172,0.000044,0.985784
115,0.000912,0.002892,0.996195


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p006ST.csv", index = False,
         header=None)

In [128]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 169us/step - loss: 0.7487 - accuracy: 0.6926 - val_loss: 1.0754 - val_accuracy: 0.5385
Epoch 2/1000
270/270 [==============================] - 0s 130us/step - loss: 0.8105 - accuracy: 0.6630 - val_loss: 1.1082 - val_accuracy: 0.6154
Epoch 3/1000
270/270 [==============================] - 0s 97us/step - loss: 0.9931 - accuracy: 0.6407 - val_loss: 1.1000 - val_accuracy: 0.5812
Epoch 4/1000
270/270 [==============================] - 0s 96us/step - loss: 0.7994 - accuracy: 0.6815 - val_loss: 1.0535 - val_accuracy: 0.5726
Epoch 5/1000
270/270 [==============================] - 0s 86us/step - loss: 0.7899 - accuracy: 0.6704 - val_loss: 1.0287 - val_accuracy: 0.6410
Epoch 6/1000
270/270 [==============================] - 0s 90us/step - loss: 0.7671 - accuracy: 0.6852 - val_loss: 1.0359 - val_accuracy: 0.5470
Epoch 7/1000
270/270 [==============================] - 0s 67us/step - loss: 0.812

Epoch 57/1000
270/270 [==============================] - 0s 58us/step - loss: 0.8033 - accuracy: 0.6593 - val_loss: 1.0235 - val_accuracy: 0.6068
Epoch 58/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7562 - accuracy: 0.6926 - val_loss: 1.0218 - val_accuracy: 0.5641
Epoch 59/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7567 - accuracy: 0.6889 - val_loss: 1.0093 - val_accuracy: 0.5385
Epoch 60/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7578 - accuracy: 0.6963 - val_loss: 1.0017 - val_accuracy: 0.5556
Epoch 61/1000
270/270 [==============================] - 0s 52us/step - loss: 0.7397 - accuracy: 0.6963 - val_loss: 1.0137 - val_accuracy: 0.5556
Epoch 62/1000
270/270 [==============================] - 0s 47us/step - loss: 0.7407 - accuracy: 0.6963 - val_loss: 1.0054 - val_accuracy: 0.6154
Epoch 63/1000
270/270 [==============================] - 0s 67us/step - loss: 0.8043 - accuracy: 0.6741 - val_loss: 1.0541 -

270/270 [==============================] - 0s 45us/step - loss: 0.7449 - accuracy: 0.6889 - val_loss: 1.0076 - val_accuracy: 0.5726
Epoch 114/1000
270/270 [==============================] - 0s 48us/step - loss: 0.7533 - accuracy: 0.7000 - val_loss: 0.9954 - val_accuracy: 0.6154
Epoch 115/1000
270/270 [==============================] - 0s 46us/step - loss: 0.7311 - accuracy: 0.6963 - val_loss: 1.0045 - val_accuracy: 0.5641
Epoch 116/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7390 - accuracy: 0.6889 - val_loss: 0.9936 - val_accuracy: 0.5983
Epoch 117/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7336 - accuracy: 0.6889 - val_loss: 0.9937 - val_accuracy: 0.6154
Epoch 118/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7351 - accuracy: 0.6741 - val_loss: 1.0067 - val_accuracy: 0.5812
Epoch 119/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7302 - accuracy: 0.7037 - val_loss: 1.0022 - val_acc

Epoch 169/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8325 - accuracy: 0.6407 - val_loss: 1.0390 - val_accuracy: 0.5385
Epoch 170/1000
270/270 [==============================] - 0s 56us/step - loss: 0.8299 - accuracy: 0.6704 - val_loss: 1.0603 - val_accuracy: 0.6154
Epoch 171/1000
270/270 [==============================] - 0s 59us/step - loss: 0.8009 - accuracy: 0.6741 - val_loss: 1.0572 - val_accuracy: 0.5470
Epoch 172/1000
270/270 [==============================] - 0s 74us/step - loss: 0.7568 - accuracy: 0.6815 - val_loss: 1.0254 - val_accuracy: 0.6239
Epoch 173/1000
270/270 [==============================] - 0s 71us/step - loss: 0.7662 - accuracy: 0.6667 - val_loss: 1.0221 - val_accuracy: 0.5556
Epoch 174/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7309 - accuracy: 0.7037 - val_loss: 1.0148 - val_accuracy: 0.6239
Epoch 175/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8061 - accuracy: 0.6778 - val_loss: 1

Epoch 280/1000
270/270 [==============================] - 0s 121us/step - loss: 0.7198 - accuracy: 0.6926 - val_loss: 0.9991 - val_accuracy: 0.6239
Epoch 281/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7306 - accuracy: 0.6741 - val_loss: 1.0003 - val_accuracy: 0.6068
Epoch 282/1000
270/270 [==============================] - 0s 219us/step - loss: 0.7268 - accuracy: 0.6889 - val_loss: 1.0067 - val_accuracy: 0.6068
Epoch 283/1000
270/270 [==============================] - 0s 130us/step - loss: 0.7183 - accuracy: 0.6926 - val_loss: 1.0079 - val_accuracy: 0.6239
Epoch 284/1000
270/270 [==============================] - 0s 79us/step - loss: 0.7255 - accuracy: 0.6926 - val_loss: 1.0034 - val_accuracy: 0.6068
Epoch 285/1000
270/270 [==============================] - 0s 83us/step - loss: 0.7179 - accuracy: 0.6889 - val_loss: 1.0015 - val_accuracy: 0.6068
Epoch 286/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7335 - accuracy: 0.6815 - val_loss

...

270/270 [==============================] - 0s 116us/step - loss: 0.7040 - accuracy: 0.7037 - val_loss: 1.0128 - val_accuracy: 0.6410
Epoch 613/1000
270/270 [==============================] - 0s 168us/step - loss: 0.7081 - accuracy: 0.7111 - val_loss: 1.0234 - val_accuracy: 0.5897
Epoch 614/1000
270/270 [==============================] - 0s 198us/step - loss: 0.7011 - accuracy: 0.7074 - val_loss: 1.0402 - val_accuracy: 0.5983
Epoch 615/1000
270/270 [==============================] - 0s 87us/step - loss: 0.7124 - accuracy: 0.7000 - val_loss: 1.0175 - val_accuracy: 0.6325
Epoch 616/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7324 - accuracy: 0.6963 - val_loss: 1.0164 - val_accuracy: 0.6410
Epoch 617/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7155 - accuracy: 0.7000 - val_loss: 1.0154 - val_accuracy: 0.6410
Epoch 618/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7358 - accuracy: 0.7000 - val_loss: 1.0395 - val_

270/270 [==============================] - 0s 63us/step - loss: 0.7053 - accuracy: 0.7074 - val_loss: 1.0360 - val_accuracy: 0.5726
Epoch 723/1000
270/270 [==============================] - 0s 112us/step - loss: 0.7025 - accuracy: 0.6926 - val_loss: 1.0411 - val_accuracy: 0.5812
Epoch 724/1000
270/270 [==============================] - 0s 72us/step - loss: 0.7069 - accuracy: 0.7000 - val_loss: 1.0367 - val_accuracy: 0.5726
Epoch 725/1000
270/270 [==============================] - 0s 130us/step - loss: 0.6952 - accuracy: 0.7000 - val_loss: 1.0175 - val_accuracy: 0.6154
Epoch 726/1000
270/270 [==============================] - 0s 66us/step - loss: 0.7007 - accuracy: 0.7037 - val_loss: 1.0222 - val_accuracy: 0.5812
Epoch 727/1000
270/270 [==============================] - ETA: 0s - loss: 0.6359 - accuracy: 0.75 - 0s 96us/step - loss: 0.7162 - accuracy: 0.6889 - val_loss: 1.0137 - val_accuracy: 0.6581
Epoch 728/1000
270/270 [==============================] - 0s 138us/step - loss: 0.7344 - 

270/270 [==============================] - 0s 100us/step - loss: 0.7080 - accuracy: 0.7111 - val_loss: 1.0215 - val_accuracy: 0.5726
Epoch 833/1000
270/270 [==============================] - 0s 98us/step - loss: 0.6910 - accuracy: 0.7000 - val_loss: 1.0127 - val_accuracy: 0.5812
Epoch 834/1000
270/270 [==============================] - 0s 129us/step - loss: 0.6930 - accuracy: 0.7074 - val_loss: 1.0321 - val_accuracy: 0.5812
Epoch 835/1000
270/270 [==============================] - 0s 180us/step - loss: 0.7555 - accuracy: 0.6852 - val_loss: 1.1423 - val_accuracy: 0.6410
Epoch 836/1000
270/270 [==============================] - 0s 297us/step - loss: 0.8865 - accuracy: 0.6963 - val_loss: 1.0908 - val_accuracy: 0.6581
Epoch 837/1000
270/270 [==============================] - 0s 173us/step - loss: 0.7416 - accuracy: 0.6852 - val_loss: 1.0856 - val_accuracy: 0.5812
Epoch 838/1000
270/270 [==============================] - 0s 202us/step - loss: 0.7329 - accuracy: 0.6926 - val_loss: 1.0522 - v

Epoch 888/1000
270/270 [==============================] - 0s 65us/step - loss: 0.6828 - accuracy: 0.7148 - val_loss: 1.0088 - val_accuracy: 0.5812
Epoch 889/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7014 - accuracy: 0.7000 - val_loss: 1.0347 - val_accuracy: 0.6496
Epoch 890/1000
270/270 [==============================] - 0s 84us/step - loss: 0.7045 - accuracy: 0.6926 - val_loss: 1.0457 - val_accuracy: 0.6410
Epoch 891/1000
270/270 [==============================] - 0s 74us/step - loss: 0.6882 - accuracy: 0.7037 - val_loss: 1.0325 - val_accuracy: 0.5897
Epoch 892/1000
270/270 [==============================] - 0s 76us/step - loss: 0.7009 - accuracy: 0.6926 - val_loss: 1.0106 - val_accuracy: 0.6581
Epoch 893/1000
270/270 [==============================] - 0s 83us/step - loss: 0.7007 - accuracy: 0.7000 - val_loss: 1.0206 - val_accuracy: 0.5983
Epoch 894/1000
270/270 [==============================] - 0s 78us/step - loss: 0.6844 - accuracy: 0.7037 - val_loss: 1

270/270 [==============================] - 0s 134us/step - loss: 0.7010 - accuracy: 0.7000 - val_loss: 1.0439 - val_accuracy: 0.6325
Epoch 999/1000
270/270 [==============================] - 0s 87us/step - loss: 0.7139 - accuracy: 0.6852 - val_loss: 1.0205 - val_accuracy: 0.5812
Epoch 1000/1000
270/270 [==============================] - 0s 88us/step - loss: 0.7012 - accuracy: 0.6926 - val_loss: 1.0252 - val_accuracy: 0.5897


In [129]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 68.87%


In [50]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [51]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [52]:
y_prob4 = df_proba4[df_proba4['phage']=='p0006kpresabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[2.73087800e-01, 5.61901400e-01, 1.65010820e-01],
       [6.12003100e-01, 2.92164980e-01, 9.58318400e-02],
       [7.03594000e-01, 2.86381500e-01, 1.00245120e-02],
       [9.12392860e-04, 2.89244740e-03, 9.96195200e-01],
       [9.01154800e-01, 4.38343960e-02, 5.50107600e-02],
       [5.17113900e-01, 9.89344000e-02, 3.83951700e-01],
       [3.01279630e-01, 1.96160030e-01, 5.02560300e-01],
       [5.88236100e-01, 2.31664760e-01, 1.80099110e-01],
       [3.27735320e-02, 1.49786170e-02, 9.52247800e-01],
       [4.35468400e-01, 4.20245050e-01, 1.44286560e-01],
       [1.98399540e-02, 1.24554420e-01, 8.55605600e-01],
       [2.25273170e-02, 4.97500040e-02, 9.27722700e-01],
       [3.27735320e-02, 1.49786170e-02, 9.52247800e-01],
       [5.17144860e-01, 4.71866430e-01, 1.09887040e-02],
       [2.73087800e-01, 5.61901400e-01, 1.65010820e-01],
       [1.75274690e-02, 1.12088960e-01, 8.70383560e-01],
       [5.66983630e-02, 9.73544400e-02, 8.45947200e-01],
       [2.75113300e-01, 9.11081

In [53]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.7599167214551829

In [54]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.7599167214551829

In [55]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.7753122945430637

In [56]:
np.std(ovos)

0.015787556019189943

In [57]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.7753122945430637

In [58]:
np.std(ovrs)

0.015787556019189943

In [130]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [131]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 62.82%


In [132]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.03226424780028013


In [133]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [134]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 70.38%


In [135]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.04090423


In [46]:
############ Feature selection using lasso ##########

In [17]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [18]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X_over[:,1:], y_over)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [19]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [20]:
X_train_features_over = np.vstack((names, X_over[:,1:]))
X_train_features_over = pd.DataFrame(X_train_features_over)

In [21]:
sel_features = X_train_features_over.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_over.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 20
selected features: 17


In [22]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 0,  1,  2,  4,  5,  6,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18,
        19]])

In [23]:
names_arr = np.array(names)
names_arr[cols]

array(['TTGTTATAGTC', 'TTAATTTAATAGA', 'TTAACATAATAAT', 'TATTATGTTAATG',
       'TACATACCGAT', 'GTGTATCATAAT', 'GCAAACATGCG', 'GAGTCCTGTT',
       'GAGTCCTGTTT',
       'GACAAACATGTATTAGCGTTATGTCGCGAACATCATAACCAGCAACATGCGATTGGCGTTAAGTCGTTTGATGATAAATATCACTTGCATGACTCGTGG',
       'CTTTTTCACCTGT', 'CTTGTGAATTTAG', 'CGCCATTATGTT', 'CAGAAAAGCGT',
       'ACAATTACTATATTT', 'ST', 'CC'], dtype='<U99')

In [24]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [25]:
df_sel['strain'] = X.iloc[:,0]

In [26]:
df_sel

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCAAACATGCG,GAGTCCTGTT,GAGTCCTGTTT,GACAAACATGTATTAGCGTTATGTCGCGAACATCATAACCAGCAACATGCGATTGGCGTTAAGTCGTTTGATGATAAATATCACTTGCATGACTCGTGG,CTTTTTCACCTGT,CTTGTGAATTTAG,CGCCATTATGTT,CAGAAAAGCGT,ACAATTACTATATTT,ST,CC,pheno,strain
0,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,5,5,1,107
1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,8,8,0,109
2,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,5,5,1,115
3,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,5,5,1,120335
4,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,5,5,1,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,5,5,1,SR4152
249,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,3812,5,0,SR4153
250,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,5,5,0,SR4155
251,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,5,5,0,SR4156


In [27]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 18) (253,) (253, 19)


In [28]:
df_sel['pheno'].value_counts()

0    129
1     85
2     39
Name: pheno, dtype: int64

In [29]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 129), (1, 129), (2, 129)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [30]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel_over)

In [31]:
dat5 = pd.DataFrame(X_sel_test_over[:,-1])
dat5['test'] = y_sel_test_over

In [32]:
dat5

,0,test
0,NRS245,1
1,NY439,2
2,CA544,1
3,CA541,2
4,EUH15,1
...,...,...
112,NRS112,0
113,CFBRSa51,2
114,NRS383,1
115,NRS247,0


In [33]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [83]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [84]:
model2_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 490us/step - loss: 12.7994 - accuracy: 0.3074 - val_loss: 13.0135 - val_accuracy: 0.2821
Epoch 2/1000
270/270 [==============================] - 0s 83us/step - loss: 10.3844 - accuracy: 0.3296 - val_loss: 10.4013 - val_accuracy: 0.3846
Epoch 3/1000
270/270 [==============================] - 0s 100us/step - loss: 8.3810 - accuracy: 0.4370 - val_loss: 8.0674 - val_accuracy: 0.3932
Epoch 4/1000
270/270 [==============================] - 0s 111us/step - loss: 6.3687 - accuracy: 0.4407 - val_loss: 5.9722 - val_accuracy: 0.3932
Epoch 5/1000
270/270 [==============================] - 0s 104us/step - loss: 4.6766 - accuracy: 0.4185 - val_loss: 4.2208 - val_accuracy: 0.3590
Epoch 6/1000
270/270 [==============================] - 0s 103us/step - loss: 2.8230 - accuracy: 0.3963 - val_loss: 2.4689 - val_accuracy: 0.3590
Epoch 7/1000
270/270 [==============================] - 0s 106us/step - los

270/270 [==============================] - 0s 79us/step - loss: 0.8962 - accuracy: 0.5778 - val_loss: 1.1656 - val_accuracy: 0.5128
Epoch 112/1000
270/270 [==============================] - 0s 72us/step - loss: 0.9288 - accuracy: 0.5593 - val_loss: 1.1898 - val_accuracy: 0.4957
Epoch 113/1000
270/270 [==============================] - 0s 72us/step - loss: 0.9647 - accuracy: 0.5741 - val_loss: 1.1962 - val_accuracy: 0.5470
Epoch 114/1000
270/270 [==============================] - 0s 67us/step - loss: 1.0155 - accuracy: 0.5370 - val_loss: 1.1661 - val_accuracy: 0.5214
Epoch 115/1000
270/270 [==============================] - 0s 81us/step - loss: 1.0360 - accuracy: 0.5741 - val_loss: 1.1000 - val_accuracy: 0.5385
Epoch 116/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8952 - accuracy: 0.5741 - val_loss: 1.1725 - val_accuracy: 0.5128
Epoch 117/1000
270/270 [==============================] - 0s 83us/step - loss: 0.9394 - accuracy: 0.5370 - val_loss: 1.0626 - val_acc

Epoch 167/1000
270/270 [==============================] - 0s 71us/step - loss: 0.8688 - accuracy: 0.6444 - val_loss: 1.0304 - val_accuracy: 0.5385
Epoch 168/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8721 - accuracy: 0.6185 - val_loss: 1.1514 - val_accuracy: 0.5641
Epoch 169/1000
270/270 [==============================] - 0s 65us/step - loss: 0.9196 - accuracy: 0.6111 - val_loss: 1.1125 - val_accuracy: 0.5556
Epoch 170/1000
270/270 [==============================] - 0s 67us/step - loss: 0.9459 - accuracy: 0.6074 - val_loss: 1.0759 - val_accuracy: 0.5299
Epoch 171/1000
270/270 [==============================] - 0s 69us/step - loss: 0.9327 - accuracy: 0.5963 - val_loss: 1.2786 - val_accuracy: 0.5385
Epoch 172/1000
270/270 [==============================] - 0s 70us/step - loss: 0.9773 - accuracy: 0.5963 - val_loss: 1.3392 - val_accuracy: 0.5299
Epoch 173/1000
270/270 [==============================] - 0s 70us/step - loss: 1.0264 - accuracy: 0.6037 - val_loss: 1

Epoch 278/1000
270/270 [==============================] - 0s 97us/step - loss: 0.7726 - accuracy: 0.6741 - val_loss: 0.9768 - val_accuracy: 0.6068
Epoch 279/1000
270/270 [==============================] - 0s 84us/step - loss: 0.7807 - accuracy: 0.6926 - val_loss: 0.9874 - val_accuracy: 0.6068
Epoch 280/1000
270/270 [==============================] - 0s 76us/step - loss: 0.7778 - accuracy: 0.6852 - val_loss: 0.9950 - val_accuracy: 0.6068
Epoch 281/1000
270/270 [==============================] - 0s 73us/step - loss: 0.7850 - accuracy: 0.6778 - val_loss: 1.0762 - val_accuracy: 0.6239
Epoch 282/1000
270/270 [==============================] - 0s 69us/step - loss: 0.8338 - accuracy: 0.6741 - val_loss: 1.0494 - val_accuracy: 0.5556
Epoch 283/1000
270/270 [==============================] - 0s 67us/step - loss: 0.8162 - accuracy: 0.6667 - val_loss: 1.1120 - val_accuracy: 0.6239
Epoch 284/1000
270/270 [==============================] - 0s 73us/step - loss: 0.9436 - accuracy: 0.6667 - val_loss: 0

Epoch 389/1000
270/270 [==============================] - 0s 106us/step - loss: 0.7505 - accuracy: 0.6889 - val_loss: 0.9648 - val_accuracy: 0.6239
Epoch 390/1000
270/270 [==============================] - 0s 85us/step - loss: 0.7593 - accuracy: 0.6926 - val_loss: 1.0670 - val_accuracy: 0.6410
Epoch 391/1000
270/270 [==============================] - 0s 72us/step - loss: 0.8253 - accuracy: 0.6889 - val_loss: 0.9764 - val_accuracy: 0.6239
Epoch 392/1000
270/270 [==============================] - 0s 72us/step - loss: 1.0690 - accuracy: 0.6741 - val_loss: 1.2571 - val_accuracy: 0.6410
Epoch 393/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8528 - accuracy: 0.6778 - val_loss: 1.0472 - val_accuracy: 0.5385
Epoch 394/1000
270/270 [==============================] - 0s 67us/step - loss: 1.1583 - accuracy: 0.6296 - val_loss: 1.7526 - val_accuracy: 0.4872
Epoch 395/1000
270/270 [==============================] - 0s 69us/step - loss: 0.9032 - accuracy: 0.6185 - val_loss: 

...

Epoch 666/1000
270/270 [==============================] - 0s 71us/step - loss: 0.7260 - accuracy: 0.7074 - val_loss: 0.9949 - val_accuracy: 0.6154
Epoch 667/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7118 - accuracy: 0.7111 - val_loss: 1.0289 - val_accuracy: 0.5812
Epoch 668/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7055 - accuracy: 0.7148 - val_loss: 0.9978 - val_accuracy: 0.6154
Epoch 669/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7021 - accuracy: 0.7037 - val_loss: 1.0036 - val_accuracy: 0.6154
Epoch 670/1000
270/270 [==============================] - 0s 57us/step - loss: 0.7103 - accuracy: 0.7148 - val_loss: 1.0449 - val_accuracy: 0.5897
Epoch 671/1000
270/270 [==============================] - 0s 70us/step - loss: 0.7106 - accuracy: 0.7000 - val_loss: 0.9839 - val_accuracy: 0.6239
Epoch 672/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7421 - accuracy: 0.7000 - val_loss: 1

Epoch 777/1000
270/270 [==============================] - 0s 84us/step - loss: 0.6894 - accuracy: 0.7296 - val_loss: 0.9868 - val_accuracy: 0.6410
Epoch 778/1000
270/270 [==============================] - 0s 90us/step - loss: 0.6965 - accuracy: 0.7259 - val_loss: 1.0094 - val_accuracy: 0.6068
Epoch 779/1000
270/270 [==============================] - 0s 100us/step - loss: 0.7024 - accuracy: 0.7148 - val_loss: 1.0566 - val_accuracy: 0.6154
Epoch 780/1000
270/270 [==============================] - 0s 90us/step - loss: 0.7003 - accuracy: 0.7074 - val_loss: 0.9955 - val_accuracy: 0.6068
Epoch 781/1000
270/270 [==============================] - 0s 96us/step - loss: 0.6898 - accuracy: 0.7148 - val_loss: 0.9903 - val_accuracy: 0.6325
Epoch 782/1000
270/270 [==============================] - 0s 96us/step - loss: 0.7453 - accuracy: 0.7185 - val_loss: 1.6294 - val_accuracy: 0.5812
Epoch 783/1000
270/270 [==============================] - 0s 153us/step - loss: 1.1458 - accuracy: 0.6815 - val_loss:

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.102664). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


270/270 [==============================] - 0s 405us/step - loss: 0.6935 - accuracy: 0.7148 - val_loss: 1.1076 - val_accuracy: 0.6325
Epoch 853/1000
270/270 [==============================] - 0s 314us/step - loss: 0.7166 - accuracy: 0.7000 - val_loss: 1.0725 - val_accuracy: 0.5897
Epoch 854/1000
270/270 [==============================] - 0s 422us/step - loss: 0.7224 - accuracy: 0.7074 - val_loss: 1.0355 - val_accuracy: 0.6154
Epoch 855/1000
270/270 [==============================] - 0s 121us/step - loss: 0.6978 - accuracy: 0.7185 - val_loss: 1.0245 - val_accuracy: 0.6154
Epoch 856/1000
270/270 [==============================] - 0s 316us/step - loss: 0.7188 - accuracy: 0.7037 - val_loss: 1.1583 - val_accuracy: 0.5897
Epoch 857/1000
270/270 [==============================] - 0s 227us/step - loss: 0.7136 - accuracy: 0.7074 - val_loss: 1.0438 - val_accuracy: 0.6239
Epoch 858/1000
270/270 [==============================] - 0s 168us/step - loss: 0.6932 - accuracy: 0.7222 - val_loss: 1.0673 - 

270/270 [==============================] - 0s 185us/step - loss: 0.6894 - accuracy: 0.7111 - val_loss: 0.9876 - val_accuracy: 0.6325
Epoch 963/1000
270/270 [==============================] - 0s 118us/step - loss: 0.6817 - accuracy: 0.7296 - val_loss: 1.0179 - val_accuracy: 0.6239
Epoch 964/1000
270/270 [==============================] - 0s 109us/step - loss: 0.6863 - accuracy: 0.7074 - val_loss: 1.0005 - val_accuracy: 0.6325
Epoch 965/1000
270/270 [==============================] - 0s 121us/step - loss: 0.6871 - accuracy: 0.7111 - val_loss: 1.0812 - val_accuracy: 0.5983
Epoch 966/1000
270/270 [==============================] - 0s 96us/step - loss: 0.6856 - accuracy: 0.7296 - val_loss: 1.0013 - val_accuracy: 0.6410
Epoch 967/1000
270/270 [==============================] - 0s 108us/step - loss: 0.6772 - accuracy: 0.7296 - val_loss: 1.0112 - val_accuracy: 0.6325
Epoch 968/1000
270/270 [==============================] - 0s 95us/step - loss: 0.6786 - accuracy: 0.7148 - val_loss: 1.0369 - va

In [182]:
acc_test2_over = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over*100))

117/117 [==============================] - 0s 70us/step
over-sampling test accuracy: 59.83%


In [86]:
pred5 = model2_over.predict_classes(X_sel_test_over)
pred5

array([2, 2, 1, 1, 1, 0, 0, 1, 0, 1, 0, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2,
       1, 2, 2, 2, 0, 1, 1, 1, 2, 1, 2, 0, 2, 1, 0, 2, 0, 0, 2, 2, 1, 1,
       1, 0, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 1, 1, 1, 2, 1, 0, 0, 0,
       1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 0, 1, 1, 2, 0, 0, 2, 0, 2, 1, 1,
       1, 0, 1, 2, 0, 0, 1, 2, 2, 2, 1, 0, 2, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       2, 1, 0, 1, 0, 1, 1])

In [87]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,NRS245,1,2
1,NY439,2,2
2,CA544,1,1
3,CA541,2,1
4,EUH15,1,1
...,...,...,...
112,NRS112,0,0
113,CFBRSa51,2,1
114,NRS383,1,0
115,NRS247,0,1


In [88]:
proba5 = model2_over.predict_proba(X_sel_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [89]:
dat_proba5

,0,1,2
0,0.136788,0.156677,0.706534
1,0.000259,0.002292,0.997449
2,0.297093,0.563622,0.139284
3,0.297093,0.563622,0.139284
4,0.297093,0.563622,0.139284
...,...,...,...
112,0.701488,0.289887,0.008625
113,0.297093,0.563622,0.139284
114,0.942912,0.056282,0.000806
115,0.408348,0.581091,0.010561


In [90]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [91]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p006ST.csv", index = False,
         header=None)

In [186]:
hist2_over = model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 110us/step - loss: 0.6863 - accuracy: 0.7074 - val_loss: 1.0026 - val_accuracy: 0.6154
Epoch 2/1000
270/270 [==============================] - 0s 93us/step - loss: 0.7281 - accuracy: 0.7000 - val_loss: 1.0597 - val_accuracy: 0.5556
Epoch 3/1000
270/270 [==============================] - 0s 121us/step - loss: 0.7126 - accuracy: 0.7000 - val_loss: 0.9970 - val_accuracy: 0.6325
Epoch 4/1000
270/270 [==============================] - 0s 115us/step - loss: 0.6874 - accuracy: 0.7074 - val_loss: 0.9825 - val_accuracy: 0.6154
Epoch 5/1000
270/270 [==============================] - 0s 96us/step - loss: 0.6885 - accuracy: 0.7148 - val_loss: 0.9594 - val_accuracy: 0.6239
Epoch 6/1000
270/270 [==============================] - 0s 101us/step - loss: 0.6817 - accuracy: 0.7222 - val_loss: 1.0169 - val_accuracy: 0.6239
Epoch 7/1000
270/270 [==============================] - 0s 111us/step - loss: 0.

Epoch 112/1000
270/270 [==============================] - 0s 120us/step - loss: 0.6896 - accuracy: 0.6852 - val_loss: 0.9858 - val_accuracy: 0.5897
Epoch 113/1000
270/270 [==============================] - 0s 87us/step - loss: 0.7144 - accuracy: 0.7074 - val_loss: 0.9685 - val_accuracy: 0.6410
Epoch 114/1000
270/270 [==============================] - 0s 95us/step - loss: 0.6793 - accuracy: 0.7222 - val_loss: 1.0809 - val_accuracy: 0.6154
Epoch 115/1000
270/270 [==============================] - 0s 99us/step - loss: 0.7227 - accuracy: 0.7000 - val_loss: 1.0976 - val_accuracy: 0.5641
Epoch 116/1000
270/270 [==============================] - 0s 98us/step - loss: 0.7246 - accuracy: 0.7037 - val_loss: 1.0062 - val_accuracy: 0.6154
Epoch 117/1000
270/270 [==============================] - 0s 99us/step - loss: 0.6758 - accuracy: 0.7074 - val_loss: 0.9779 - val_accuracy: 0.6239
Epoch 118/1000
270/270 [==============================] - 0s 87us/step - loss: 0.7212 - accuracy: 0.7111 - val_loss: 

270/270 [==============================] - 0s 119us/step - loss: 0.6930 - accuracy: 0.7148 - val_loss: 1.0778 - val_accuracy: 0.6239
Epoch 223/1000
270/270 [==============================] - 0s 109us/step - loss: 0.7572 - accuracy: 0.6926 - val_loss: 0.9682 - val_accuracy: 0.6239
Epoch 224/1000
270/270 [==============================] - 0s 133us/step - loss: 0.7366 - accuracy: 0.7074 - val_loss: 0.9925 - val_accuracy: 0.6068
Epoch 225/1000
270/270 [==============================] - 0s 96us/step - loss: 0.7028 - accuracy: 0.6926 - val_loss: 1.0290 - val_accuracy: 0.6239
Epoch 226/1000
270/270 [==============================] - 0s 119us/step - loss: 0.7175 - accuracy: 0.6852 - val_loss: 0.9785 - val_accuracy: 0.5983
Epoch 227/1000
270/270 [==============================] - 0s 103us/step - loss: 0.6675 - accuracy: 0.7074 - val_loss: 1.0860 - val_accuracy: 0.5812
Epoch 228/1000
270/270 [==============================] - 0s 151us/step - loss: 0.7214 - accuracy: 0.7000 - val_loss: 1.0538 - v

Epoch 333/1000
270/270 [==============================] - 0s 94us/step - loss: 0.7543 - accuracy: 0.6926 - val_loss: 1.0244 - val_accuracy: 0.6154
Epoch 334/1000
270/270 [==============================] - 0s 89us/step - loss: 0.6948 - accuracy: 0.7148 - val_loss: 0.9871 - val_accuracy: 0.5983
Epoch 335/1000
270/270 [==============================] - 0s 82us/step - loss: 0.6863 - accuracy: 0.6889 - val_loss: 0.9764 - val_accuracy: 0.5983
Epoch 336/1000
270/270 [==============================] - 0s 87us/step - loss: 0.6718 - accuracy: 0.7111 - val_loss: 0.9748 - val_accuracy: 0.6154
Epoch 337/1000
270/270 [==============================] - 0s 85us/step - loss: 0.6668 - accuracy: 0.7037 - val_loss: 0.9962 - val_accuracy: 0.5641
Epoch 338/1000
270/270 [==============================] - 0s 96us/step - loss: 0.6986 - accuracy: 0.7111 - val_loss: 1.0033 - val_accuracy: 0.6154
Epoch 339/1000
270/270 [==============================] - 0s 83us/step - loss: 0.6972 - accuracy: 0.6926 - val_loss: 1

Epoch 444/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8313 - accuracy: 0.6815 - val_loss: 1.1975 - val_accuracy: 0.6154
Epoch 445/1000
270/270 [==============================] - 0s 93us/step - loss: 0.8021 - accuracy: 0.7222 - val_loss: 0.9954 - val_accuracy: 0.6068
Epoch 446/1000
270/270 [==============================] - 0s 95us/step - loss: 0.7568 - accuracy: 0.6963 - val_loss: 1.1132 - val_accuracy: 0.6068
Epoch 447/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8181 - accuracy: 0.6926 - val_loss: 1.0148 - val_accuracy: 0.6325
Epoch 448/1000
270/270 [==============================] - 0s 91us/step - loss: 0.7775 - accuracy: 0.7148 - val_loss: 1.1411 - val_accuracy: 0.6410
Epoch 449/1000
270/270 [==============================] - 0s 95us/step - loss: 0.7403 - accuracy: 0.7037 - val_loss: 1.0394 - val_accuracy: 0.5726
Epoch 450/1000
270/270 [==============================] - 0s 85us/step - loss: 0.6936 - accuracy: 0.7148 - val_loss: 1

...

Epoch 668/1000
270/270 [==============================] - 0s 82us/step - loss: 0.7490 - accuracy: 0.7037 - val_loss: 1.0556 - val_accuracy: 0.6325
Epoch 669/1000
270/270 [==============================] - 0s 90us/step - loss: 0.7283 - accuracy: 0.7185 - val_loss: 1.0136 - val_accuracy: 0.5897
Epoch 670/1000
270/270 [==============================] - 0s 82us/step - loss: 0.7033 - accuracy: 0.6889 - val_loss: 1.0060 - val_accuracy: 0.6154
Epoch 671/1000
270/270 [==============================] - 0s 83us/step - loss: 0.7025 - accuracy: 0.7000 - val_loss: 0.9731 - val_accuracy: 0.6154
Epoch 672/1000
270/270 [==============================] - 0s 94us/step - loss: 0.6967 - accuracy: 0.7148 - val_loss: 0.9585 - val_accuracy: 0.6239
Epoch 673/1000
270/270 [==============================] - 0s 90us/step - loss: 0.6687 - accuracy: 0.7148 - val_loss: 0.9818 - val_accuracy: 0.6239
Epoch 674/1000
270/270 [==============================] - 0s 91us/step - loss: 0.6759 - accuracy: 0.7000 - val_loss: 0

Epoch 724/1000
270/270 [==============================] - 0s 90us/step - loss: 0.6563 - accuracy: 0.7259 - val_loss: 0.9624 - val_accuracy: 0.6325
Epoch 725/1000
270/270 [==============================] - 0s 81us/step - loss: 0.6557 - accuracy: 0.7259 - val_loss: 0.9571 - val_accuracy: 0.6239
Epoch 726/1000
270/270 [==============================] - 0s 87us/step - loss: 0.6589 - accuracy: 0.7296 - val_loss: 0.9812 - val_accuracy: 0.6068
Epoch 727/1000
270/270 [==============================] - 0s 81us/step - loss: 0.6670 - accuracy: 0.7148 - val_loss: 0.9806 - val_accuracy: 0.5983
Epoch 728/1000
270/270 [==============================] - 0s 84us/step - loss: 0.6826 - accuracy: 0.6778 - val_loss: 1.0977 - val_accuracy: 0.6239
Epoch 729/1000
270/270 [==============================] - 0s 90us/step - loss: 0.7265 - accuracy: 0.7074 - val_loss: 0.9709 - val_accuracy: 0.6154
Epoch 730/1000
270/270 [==============================] - 0s 91us/step - loss: 0.7152 - accuracy: 0.6889 - val_loss: 0

Epoch 780/1000
270/270 [==============================] - 0s 135us/step - loss: 0.6686 - accuracy: 0.7333 - val_loss: 1.0241 - val_accuracy: 0.5983
Epoch 781/1000
270/270 [==============================] - 0s 119us/step - loss: 0.6861 - accuracy: 0.7111 - val_loss: 0.9977 - val_accuracy: 0.6068
Epoch 782/1000
270/270 [==============================] - 0s 109us/step - loss: 0.6673 - accuracy: 0.7222 - val_loss: 0.9770 - val_accuracy: 0.6068
Epoch 783/1000
270/270 [==============================] - 0s 124us/step - loss: 0.6713 - accuracy: 0.7074 - val_loss: 1.0025 - val_accuracy: 0.6068
Epoch 784/1000
270/270 [==============================] - 0s 115us/step - loss: 0.6626 - accuracy: 0.7111 - val_loss: 0.9789 - val_accuracy: 0.6410
Epoch 785/1000
270/270 [==============================] - 0s 124us/step - loss: 0.6808 - accuracy: 0.7148 - val_loss: 1.0240 - val_accuracy: 0.5983
Epoch 786/1000
270/270 [==============================] - 0s 143us/step - loss: 0.7035 - accuracy: 0.7074 - val_

Epoch 891/1000
270/270 [==============================] - 0s 88us/step - loss: 0.6579 - accuracy: 0.7111 - val_loss: 0.9680 - val_accuracy: 0.5897
Epoch 892/1000
270/270 [==============================] - 0s 91us/step - loss: 0.6532 - accuracy: 0.7259 - val_loss: 0.9685 - val_accuracy: 0.6154
Epoch 893/1000
270/270 [==============================] - 0s 82us/step - loss: 0.6556 - accuracy: 0.7444 - val_loss: 0.9780 - val_accuracy: 0.6496
Epoch 894/1000
270/270 [==============================] - 0s 90us/step - loss: 0.6544 - accuracy: 0.7259 - val_loss: 0.9627 - val_accuracy: 0.6325
Epoch 895/1000
270/270 [==============================] - 0s 81us/step - loss: 0.6583 - accuracy: 0.7222 - val_loss: 0.9694 - val_accuracy: 0.6068
Epoch 896/1000
270/270 [==============================] - 0s 87us/step - loss: 0.6683 - accuracy: 0.7148 - val_loss: 0.9622 - val_accuracy: 0.6325
Epoch 897/1000
270/270 [==============================] - 0s 80us/step - loss: 0.7001 - accuracy: 0.7259 - val_loss: 1

Epoch 947/1000
270/270 [==============================] - 0s 98us/step - loss: 1.3572 - accuracy: 0.6704 - val_loss: 2.1860 - val_accuracy: 0.5897
Epoch 948/1000
270/270 [==============================] - 0s 88us/step - loss: 1.6198 - accuracy: 0.6889 - val_loss: 1.8257 - val_accuracy: 0.6068
Epoch 949/1000
270/270 [==============================] - 0s 88us/step - loss: 1.2316 - accuracy: 0.6815 - val_loss: 1.0899 - val_accuracy: 0.5897
Epoch 950/1000
270/270 [==============================] - 0s 98us/step - loss: 0.7310 - accuracy: 0.6963 - val_loss: 1.0917 - val_accuracy: 0.5726
Epoch 951/1000
270/270 [==============================] - 0s 79us/step - loss: 0.7452 - accuracy: 0.7074 - val_loss: 1.1257 - val_accuracy: 0.6239
Epoch 952/1000
270/270 [==============================] - 0s 91us/step - loss: 0.6873 - accuracy: 0.7259 - val_loss: 1.0667 - val_accuracy: 0.5385
Epoch 953/1000
270/270 [==============================] - 0s 81us/step - loss: 0.6833 - accuracy: 0.7037 - val_loss: 1

In [187]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over.history['accuracy'])*100))

over-sampling train accuracy: 71.04%


In [34]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [35]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS245,1,2,1.345807e-02,2.164788e-01,7.700630e-01
1,p0006kpresabs_qual,NY439,2,2,2.674153e-02,9.294230e-04,9.723290e-01
2,p0006kpresabs_qual,CA544,1,0,4.147484e-01,3.626331e-01,2.226184e-01
3,p0006kpresabs_qual,CA541,2,0,4.147484e-01,3.626331e-01,2.226184e-01
4,p0006kpresabs_qual,EUH15,1,0,4.147484e-01,3.626331e-01,2.226184e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CA541,1,1,3.723218e-01,6.276781e-01,1.945911e-08
985,p0017Skpresabs_qual,SR4152,1,0,7.372800e-01,2.627200e-01,4.197748e-08
986,p0017Skpresabs_qual,NRS110,2,2,4.194510e-08,7.508231e-09,1.000000e+00
987,p0017Skpresabs_qual,CFBRSa70,0,0,7.372800e-01,2.627200e-01,4.197748e-08


In [36]:
y_prob5 = df_proba5[df_proba5['phage']=='p0006kpresabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[1.36788460e-01, 1.56677300e-01, 7.06534270e-01],
       [2.58510400e-04, 2.29230870e-03, 9.97449100e-01],
       [2.97093330e-01, 5.63622240e-01, 1.39284450e-01],
       [2.97093330e-01, 5.63622240e-01, 1.39284450e-01],
       [2.97093330e-01, 5.63622240e-01, 1.39284450e-01],
       [4.87148970e-01, 1.45918260e-01, 3.66932720e-01],
       [8.08099800e-01, 1.57598400e-01, 3.43017840e-02],
       [5.32192850e-02, 9.09106400e-01, 3.76744050e-02],
       [4.87148970e-01, 1.45918260e-01, 3.66932720e-01],
       [7.14723840e-02, 6.93003700e-01, 2.35523980e-01],
       [4.87148970e-01, 1.45918260e-01, 3.66932720e-01],
       [2.36267240e-02, 9.51616240e-02, 8.81211640e-01],
       [1.18916640e-02, 5.16688500e-03, 9.82941450e-01],
       [5.24063600e-01, 4.48216530e-01, 2.77199020e-02],
       [3.85296640e-01, 3.30152570e-04, 6.14373200e-01],
       [1.17519476e-01, 2.02449370e-01, 6.80031240e-01],
       [1.87709470e-01, 1.86848540e-01, 6.25441970e-01],
       [2.07614730e-01, 2.25955

In [37]:
ovo5 = rocauc_ovo(y_sel_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.7400832785448169

In [38]:
ovr5 = rocauc_ovr(y_sel_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.7400832785448169

In [39]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel_over)

In [40]:
dat6 = pd.DataFrame(X_sel_test_over[:,-1])
dat6['test'] = y_sel_test_over

In [41]:
dat6

,0,test
0,NRS249,2
1,NRS188,1
2,NRS232,2
3,NY439,2
4,GA27,2
...,...,...
112,SR3569,0
113,NRS204,0
114,NRS203,0
115,CFBRSa25,1


In [42]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [43]:
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [97]:
model2_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 436us/step - loss: 17.0411 - accuracy: 0.3593 - val_loss: 8.0796 - val_accuracy: 0.3504
Epoch 2/1000
270/270 [==============================] - 0s 62us/step - loss: 13.7429 - accuracy: 0.3481 - val_loss: 5.9167 - val_accuracy: 0.3077
Epoch 3/1000
270/270 [==============================] - 0s 87us/step - loss: 10.6352 - accuracy: 0.3259 - val_loss: 4.6846 - val_accuracy: 0.2991
Epoch 4/1000
270/270 [==============================] - 0s 85us/step - loss: 9.2271 - accuracy: 0.3667 - val_loss: 3.8483 - val_accuracy: 0.3761
Epoch 5/1000
270/270 [==============================] - 0s 95us/step - loss: 7.2309 - accuracy: 0.3556 - val_loss: 2.9778 - val_accuracy: 0.3932
Epoch 6/1000
270/270 [==============================] - 0s 116us/step - loss: 5.2570 - accuracy: 0.4222 - val_loss: 2.4321 - val_accuracy: 0.4017
Epoch 7/1000
270/270 [==============================] - 0s 94us/step - loss: 3.

270/270 [==============================] - 0s 68us/step - loss: 0.8691 - accuracy: 0.5741 - val_loss: 1.0162 - val_accuracy: 0.5128
Epoch 113/1000
270/270 [==============================] - 0s 87us/step - loss: 0.8651 - accuracy: 0.5852 - val_loss: 0.9838 - val_accuracy: 0.5128
Epoch 114/1000
270/270 [==============================] - 0s 78us/step - loss: 0.8642 - accuracy: 0.5704 - val_loss: 0.9812 - val_accuracy: 0.5128
Epoch 115/1000
270/270 [==============================] - 0s 88us/step - loss: 0.8879 - accuracy: 0.5667 - val_loss: 1.0324 - val_accuracy: 0.5043
Epoch 116/1000
270/270 [==============================] - 0s 100us/step - loss: 0.8922 - accuracy: 0.5889 - val_loss: 1.1744 - val_accuracy: 0.5128
Epoch 117/1000
270/270 [==============================] - 0s 93us/step - loss: 0.9162 - accuracy: 0.5926 - val_loss: 1.0504 - val_accuracy: 0.5043
Epoch 118/1000
270/270 [==============================] - 0s 109us/step - loss: 0.8631 - accuracy: 0.5630 - val_loss: 0.9979 - val_a

Epoch 168/1000
270/270 [==============================] - 0s 74us/step - loss: 0.8634 - accuracy: 0.6037 - val_loss: 1.0310 - val_accuracy: 0.5128
Epoch 169/1000
270/270 [==============================] - 0s 76us/step - loss: 0.8504 - accuracy: 0.5963 - val_loss: 1.0406 - val_accuracy: 0.5043
Epoch 170/1000
270/270 [==============================] - 0s 66us/step - loss: 0.8371 - accuracy: 0.5926 - val_loss: 0.9762 - val_accuracy: 0.4957
Epoch 171/1000
270/270 [==============================] - 0s 65us/step - loss: 0.8203 - accuracy: 0.5963 - val_loss: 0.9871 - val_accuracy: 0.4957
Epoch 172/1000
270/270 [==============================] - 0s 84us/step - loss: 0.8249 - accuracy: 0.5889 - val_loss: 1.0049 - val_accuracy: 0.4872
Epoch 173/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8214 - accuracy: 0.5963 - val_loss: 0.9748 - val_accuracy: 0.4957
Epoch 174/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8168 - accuracy: 0.5852 - val_loss: 0

Epoch 224/1000
270/270 [==============================] - 0s 52us/step - loss: 0.9153 - accuracy: 0.6222 - val_loss: 1.3270 - val_accuracy: 0.5043
Epoch 225/1000
270/270 [==============================] - 0s 55us/step - loss: 0.9483 - accuracy: 0.5852 - val_loss: 1.1572 - val_accuracy: 0.5299
Epoch 226/1000
270/270 [==============================] - 0s 48us/step - loss: 0.8232 - accuracy: 0.6148 - val_loss: 1.0640 - val_accuracy: 0.4530
Epoch 227/1000
270/270 [==============================] - 0s 52us/step - loss: 0.8992 - accuracy: 0.5815 - val_loss: 1.1689 - val_accuracy: 0.5470
Epoch 228/1000
270/270 [==============================] - 0s 51us/step - loss: 0.8702 - accuracy: 0.6222 - val_loss: 1.0946 - val_accuracy: 0.5043
Epoch 229/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8422 - accuracy: 0.5926 - val_loss: 0.9820 - val_accuracy: 0.5214
Epoch 230/1000
270/270 [==============================] - 0s 68us/step - loss: 0.7952 - accuracy: 0.6259 - val_loss: 0

Epoch 280/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7855 - accuracy: 0.6481 - val_loss: 0.9667 - val_accuracy: 0.5556
Epoch 281/1000
270/270 [==============================] - 0s 47us/step - loss: 0.7904 - accuracy: 0.6222 - val_loss: 1.0273 - val_accuracy: 0.5897
Epoch 282/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8187 - accuracy: 0.6481 - val_loss: 1.0610 - val_accuracy: 0.5983
Epoch 283/1000
270/270 [==============================] - 0s 56us/step - loss: 0.7922 - accuracy: 0.6519 - val_loss: 0.9685 - val_accuracy: 0.5556
Epoch 284/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7956 - accuracy: 0.6222 - val_loss: 1.0059 - val_accuracy: 0.5556
Epoch 285/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8036 - accuracy: 0.6519 - val_loss: 1.0178 - val_accuracy: 0.5556
Epoch 286/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7800 - accuracy: 0.6889 - val_loss: 0

...

Epoch 614/1000
270/270 [==============================] - 0s 48us/step - loss: 0.7162 - accuracy: 0.7000 - val_loss: 0.9738 - val_accuracy: 0.5983
Epoch 615/1000
270/270 [==============================] - 0s 47us/step - loss: 0.7222 - accuracy: 0.6852 - val_loss: 0.9713 - val_accuracy: 0.6068
Epoch 616/1000
270/270 [==============================] - 0s 49us/step - loss: 0.7268 - accuracy: 0.7037 - val_loss: 0.9973 - val_accuracy: 0.6239
Epoch 617/1000
270/270 [==============================] - 0s 44us/step - loss: 0.7361 - accuracy: 0.7111 - val_loss: 1.0295 - val_accuracy: 0.6239
Epoch 618/1000
270/270 [==============================] - 0s 41us/step - loss: 0.7196 - accuracy: 0.7037 - val_loss: 0.9799 - val_accuracy: 0.6239
Epoch 619/1000
270/270 [==============================] - 0s 43us/step - loss: 0.7235 - accuracy: 0.6778 - val_loss: 1.0286 - val_accuracy: 0.5983
Epoch 620/1000
270/270 [==============================] - 0s 34us/step - loss: 0.7282 - accuracy: 0.6815 - val_loss: 0

Epoch 670/1000
270/270 [==============================] - 0s 54us/step - loss: 0.7207 - accuracy: 0.6963 - val_loss: 0.9918 - val_accuracy: 0.6325
Epoch 671/1000
270/270 [==============================] - 0s 44us/step - loss: 0.7138 - accuracy: 0.6963 - val_loss: 0.9765 - val_accuracy: 0.6239
Epoch 672/1000
270/270 [==============================] - 0s 54us/step - loss: 0.7149 - accuracy: 0.7037 - val_loss: 0.9736 - val_accuracy: 0.6154
Epoch 673/1000
270/270 [==============================] - 0s 55us/step - loss: 0.7162 - accuracy: 0.7037 - val_loss: 0.9921 - val_accuracy: 0.6325
Epoch 674/1000
270/270 [==============================] - 0s 49us/step - loss: 0.7130 - accuracy: 0.6963 - val_loss: 0.9762 - val_accuracy: 0.6068
Epoch 675/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7106 - accuracy: 0.6852 - val_loss: 0.9949 - val_accuracy: 0.6325
Epoch 676/1000
270/270 [==============================] - 0s 80us/step - loss: 0.7163 - accuracy: 0.6963 - val_loss: 0

Epoch 781/1000
270/270 [==============================] - 0s 50us/step - loss: 0.7070 - accuracy: 0.7000 - val_loss: 0.9797 - val_accuracy: 0.6154
Epoch 782/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7039 - accuracy: 0.7000 - val_loss: 0.9804 - val_accuracy: 0.6325
Epoch 783/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7099 - accuracy: 0.6852 - val_loss: 0.9969 - val_accuracy: 0.6410
Epoch 784/1000
270/270 [==============================] - 0s 56us/step - loss: 0.7158 - accuracy: 0.7000 - val_loss: 0.9931 - val_accuracy: 0.6410
Epoch 785/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7097 - accuracy: 0.7037 - val_loss: 0.9688 - val_accuracy: 0.6239
Epoch 786/1000
270/270 [==============================] - 0s 77us/step - loss: 0.7105 - accuracy: 0.7000 - val_loss: 0.9871 - val_accuracy: 0.6325
Epoch 787/1000
270/270 [==============================] - 0s 53us/step - loss: 0.7116 - accuracy: 0.6889 - val_loss: 0

Epoch 837/1000
270/270 [==============================] - 0s 56us/step - loss: 0.7174 - accuracy: 0.6889 - val_loss: 0.9875 - val_accuracy: 0.5897
Epoch 838/1000
270/270 [==============================] - 0s 63us/step - loss: 0.6978 - accuracy: 0.7111 - val_loss: 1.0564 - val_accuracy: 0.6239
Epoch 839/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7409 - accuracy: 0.7000 - val_loss: 1.0462 - val_accuracy: 0.6325
Epoch 840/1000
270/270 [==============================] - 0s 50us/step - loss: 0.7124 - accuracy: 0.7037 - val_loss: 0.9923 - val_accuracy: 0.5897
Epoch 841/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7112 - accuracy: 0.6889 - val_loss: 1.0203 - val_accuracy: 0.6325
Epoch 842/1000
270/270 [==============================] - 0s 98us/step - loss: 0.7222 - accuracy: 0.7074 - val_loss: 1.0345 - val_accuracy: 0.6325
Epoch 843/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7144 - accuracy: 0.7037 - val_loss: 0

Epoch 893/1000
270/270 [==============================] - 0s 58us/step - loss: 0.7042 - accuracy: 0.6963 - val_loss: 0.9869 - val_accuracy: 0.6154
Epoch 894/1000
270/270 [==============================] - 0s 63us/step - loss: 0.6985 - accuracy: 0.7000 - val_loss: 0.9864 - val_accuracy: 0.6325
Epoch 895/1000
270/270 [==============================] - 0s 51us/step - loss: 0.6947 - accuracy: 0.7148 - val_loss: 0.9793 - val_accuracy: 0.6410
Epoch 896/1000
270/270 [==============================] - 0s 54us/step - loss: 0.6954 - accuracy: 0.7148 - val_loss: 0.9806 - val_accuracy: 0.6239
Epoch 897/1000
270/270 [==============================] - 0s 67us/step - loss: 0.6989 - accuracy: 0.7000 - val_loss: 0.9868 - val_accuracy: 0.6325
Epoch 898/1000
270/270 [==============================] - 0s 59us/step - loss: 0.6984 - accuracy: 0.7000 - val_loss: 0.9950 - val_accuracy: 0.6410
Epoch 899/1000
270/270 [==============================] - 0s 66us/step - loss: 0.6996 - accuracy: 0.7037 - val_loss: 0

In [215]:
acc_test2_over2 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over2*100))

117/117 [==============================] - 0s 89us/step
over-sampling test accuracy: 58.12%


In [99]:
pred6 = model2_over2.predict_classes(X_sel_test_over)
pred6

array([2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 0, 1, 2, 1, 0, 2, 0, 1, 1,
       1, 1, 2, 0, 2, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 2, 0, 0,
       0, 1, 0, 2, 2, 2, 1, 2, 2, 2, 1, 1, 0, 0, 1, 1, 1, 0, 1, 2, 1, 0,
       0, 1, 0, 1, 0, 2, 1, 2, 0, 2, 2, 1, 0, 1, 2, 2, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 2, 0, 1, 0, 2, 2, 0, 2, 2, 0, 1, 1, 1, 2,
       0, 0, 0, 0, 0, 1, 2])

In [100]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,NRS249,2,2
1,NRS188,1,0
2,NRS232,2,2
3,NY439,2,2
4,GA27,2,0
...,...,...,...
112,SR3569,0,0
113,NRS204,0,0
114,NRS203,0,0
115,CFBRSa25,1,1


In [101]:
proba6 = model2_over2.predict_proba(X_sel_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [102]:
dat_proba6

,0,1,2
0,0.287155,0.095654,0.617191
1,0.867139,0.086886,0.045975
2,0.402464,0.030584,0.566951
3,0.004070,0.013978,0.981951
4,0.692081,0.158598,0.149321
...,...,...,...
112,0.407915,0.333097,0.258988
113,0.861852,0.138041,0.000107
114,0.527203,0.437141,0.035656
115,0.347299,0.557314,0.095387


In [103]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [104]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p006ST.csv", index = False,
         header=None)

In [219]:
hist2_over2 = model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 76us/step - loss: 0.7064 - accuracy: 0.7111 - val_loss: 1.0693 - val_accuracy: 0.5983
Epoch 2/1000
270/270 [==============================] - 0s 59us/step - loss: 0.6952 - accuracy: 0.6963 - val_loss: 1.0539 - val_accuracy: 0.5641
Epoch 3/1000
270/270 [==============================] - 0s 72us/step - loss: 0.6988 - accuracy: 0.7037 - val_loss: 1.0925 - val_accuracy: 0.6154
Epoch 4/1000
270/270 [==============================] - 0s 100us/step - loss: 0.7138 - accuracy: 0.7185 - val_loss: 1.2496 - val_accuracy: 0.6239
Epoch 5/1000
270/270 [==============================] - 0s 124us/step - loss: 0.7450 - accuracy: 0.7185 - val_loss: 1.0480 - val_accuracy: 0.6154
Epoch 6/1000
270/270 [==============================] - 0s 83us/step - loss: 0.7010 - accuracy: 0.7074 - val_loss: 1.0435 - val_accuracy: 0.5812
Epoch 7/1000
270/270 [==============================] - 0s 87us/step - loss: 0.688

Epoch 57/1000
270/270 [==============================] - 0s 64us/step - loss: 1.0955 - accuracy: 0.6852 - val_loss: 1.6722 - val_accuracy: 0.5983
Epoch 58/1000
270/270 [==============================] - 0s 77us/step - loss: 0.9683 - accuracy: 0.6704 - val_loss: 1.3269 - val_accuracy: 0.5983
Epoch 59/1000
270/270 [==============================] - 0s 70us/step - loss: 0.8304 - accuracy: 0.6815 - val_loss: 1.2275 - val_accuracy: 0.4701
Epoch 60/1000
270/270 [==============================] - 0s 79us/step - loss: 0.8021 - accuracy: 0.6630 - val_loss: 1.2389 - val_accuracy: 0.5726
Epoch 61/1000
270/270 [==============================] - 0s 72us/step - loss: 0.9943 - accuracy: 0.6852 - val_loss: 2.0198 - val_accuracy: 0.5556
Epoch 62/1000
270/270 [==============================] - 0s 88us/step - loss: 1.1136 - accuracy: 0.6296 - val_loss: 1.1504 - val_accuracy: 0.5641
Epoch 63/1000
270/270 [==============================] - 0s 120us/step - loss: 1.0102 - accuracy: 0.6815 - val_loss: 1.6194 

Epoch 113/1000
270/270 [==============================] - 0s 74us/step - loss: 0.6849 - accuracy: 0.7148 - val_loss: 1.1068 - val_accuracy: 0.6239
Epoch 114/1000
270/270 [==============================] - 0s 74us/step - loss: 0.7206 - accuracy: 0.7185 - val_loss: 1.1981 - val_accuracy: 0.6068
Epoch 115/1000
270/270 [==============================] - 0s 85us/step - loss: 0.7061 - accuracy: 0.7222 - val_loss: 1.0570 - val_accuracy: 0.5556
Epoch 116/1000
270/270 [==============================] - 0s 81us/step - loss: 0.6959 - accuracy: 0.7074 - val_loss: 1.0560 - val_accuracy: 0.5726
Epoch 117/1000
270/270 [==============================] - 0s 66us/step - loss: 0.6901 - accuracy: 0.7111 - val_loss: 1.0752 - val_accuracy: 0.6154
Epoch 118/1000
270/270 [==============================] - 0s 56us/step - loss: 0.6862 - accuracy: 0.7296 - val_loss: 1.1392 - val_accuracy: 0.6068
Epoch 119/1000
270/270 [==============================] - 0s 58us/step - loss: 0.7059 - accuracy: 0.7296 - val_loss: 1

Epoch 169/1000
270/270 [==============================] - 0s 80us/step - loss: 0.6891 - accuracy: 0.7148 - val_loss: 1.0814 - val_accuracy: 0.6154
Epoch 170/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6822 - accuracy: 0.7222 - val_loss: 1.0750 - val_accuracy: 0.6239
Epoch 171/1000
270/270 [==============================] - 0s 77us/step - loss: 0.6826 - accuracy: 0.7185 - val_loss: 1.0533 - val_accuracy: 0.5983
Epoch 172/1000
270/270 [==============================] - 0s 72us/step - loss: 0.6866 - accuracy: 0.7222 - val_loss: 1.0504 - val_accuracy: 0.6068
Epoch 173/1000
270/270 [==============================] - 0s 104us/step - loss: 0.6866 - accuracy: 0.7185 - val_loss: 1.1006 - val_accuracy: 0.6154
Epoch 174/1000
270/270 [==============================] - 0s 67us/step - loss: 0.6931 - accuracy: 0.7185 - val_loss: 1.0443 - val_accuracy: 0.5983
Epoch 175/1000
270/270 [==============================] - 0s 76us/step - loss: 0.6861 - accuracy: 0.7185 - val_loss: 

Epoch 225/1000
270/270 [==============================] - 0s 70us/step - loss: 0.7548 - accuracy: 0.7000 - val_loss: 1.3404 - val_accuracy: 0.5983
Epoch 226/1000
270/270 [==============================] - 0s 70us/step - loss: 0.7515 - accuracy: 0.7000 - val_loss: 1.0732 - val_accuracy: 0.5726
Epoch 227/1000
270/270 [==============================] - 0s 78us/step - loss: 0.7857 - accuracy: 0.6963 - val_loss: 1.1124 - val_accuracy: 0.5897
Epoch 228/1000
270/270 [==============================] - 0s 76us/step - loss: 0.7470 - accuracy: 0.7148 - val_loss: 1.4582 - val_accuracy: 0.6325
Epoch 229/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8870 - accuracy: 0.7000 - val_loss: 1.5282 - val_accuracy: 0.6325
Epoch 230/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8082 - accuracy: 0.7111 - val_loss: 1.1354 - val_accuracy: 0.5812
Epoch 231/1000
270/270 [==============================] - 0s 79us/step - loss: 0.7253 - accuracy: 0.7074 - val_loss: 1

...

Epoch 558/1000
270/270 [==============================] - 0s 70us/step - loss: 0.6767 - accuracy: 0.7148 - val_loss: 1.0527 - val_accuracy: 0.6239
Epoch 559/1000
270/270 [==============================] - 0s 245us/step - loss: 0.6783 - accuracy: 0.7148 - val_loss: 1.1094 - val_accuracy: 0.6239
Epoch 560/1000
270/270 [==============================] - 0s 488us/step - loss: 0.6722 - accuracy: 0.7185 - val_loss: 1.0390 - val_accuracy: 0.5726
Epoch 561/1000
270/270 [==============================] - 0s 270us/step - loss: 0.6715 - accuracy: 0.7037 - val_loss: 1.0617 - val_accuracy: 0.6154
Epoch 562/1000
270/270 [==============================] - 0s 75us/step - loss: 0.6672 - accuracy: 0.7222 - val_loss: 1.0418 - val_accuracy: 0.6154
Epoch 563/1000
270/270 [==============================] - 0s 76us/step - loss: 0.6888 - accuracy: 0.7000 - val_loss: 1.1062 - val_accuracy: 0.6239
Epoch 564/1000
270/270 [==============================] - 0s 67us/step - loss: 0.6947 - accuracy: 0.7222 - val_loss

270/270 [==============================] - 0s 84us/step - loss: 0.6689 - accuracy: 0.7259 - val_loss: 1.0513 - val_accuracy: 0.5812
Epoch 669/1000
270/270 [==============================] - 0s 83us/step - loss: 0.6848 - accuracy: 0.7037 - val_loss: 1.0553 - val_accuracy: 0.6154
Epoch 670/1000
270/270 [==============================] - 0s 116us/step - loss: 0.7005 - accuracy: 0.7222 - val_loss: 1.1494 - val_accuracy: 0.6239
Epoch 671/1000
270/270 [==============================] - 0s 95us/step - loss: 0.6827 - accuracy: 0.7074 - val_loss: 1.0665 - val_accuracy: 0.5470
Epoch 672/1000
270/270 [==============================] - 0s 122us/step - loss: 0.6766 - accuracy: 0.7222 - val_loss: 1.1984 - val_accuracy: 0.6068
Epoch 673/1000
270/270 [==============================] - 0s 72us/step - loss: 0.7020 - accuracy: 0.7222 - val_loss: 1.0371 - val_accuracy: 0.5641
Epoch 674/1000
270/270 [==============================] - 0s 94us/step - loss: 0.7416 - accuracy: 0.7000 - val_loss: 1.0410 - val_a

270/270 [==============================] - 0s 65us/step - loss: 0.6890 - accuracy: 0.7111 - val_loss: 1.2204 - val_accuracy: 0.5812
Epoch 779/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7690 - accuracy: 0.7111 - val_loss: 1.2332 - val_accuracy: 0.5641
Epoch 780/1000
270/270 [==============================] - 0s 155us/step - loss: 0.6994 - accuracy: 0.7000 - val_loss: 1.0664 - val_accuracy: 0.5726
Epoch 781/1000
270/270 [==============================] - 0s 244us/step - loss: 0.6973 - accuracy: 0.6926 - val_loss: 1.2064 - val_accuracy: 0.5983
Epoch 782/1000
270/270 [==============================] - 0s 74us/step - loss: 0.7285 - accuracy: 0.7185 - val_loss: 1.1642 - val_accuracy: 0.6239
Epoch 783/1000
270/270 [==============================] - 0s 142us/step - loss: 0.6777 - accuracy: 0.7111 - val_loss: 1.0695 - val_accuracy: 0.5897
Epoch 784/1000
270/270 [==============================] - 0s 145us/step - loss: 0.7163 - accuracy: 0.6815 - val_loss: 1.4240 - val

270/270 [==============================] - 0s 135us/step - loss: 0.6636 - accuracy: 0.7111 - val_loss: 1.1343 - val_accuracy: 0.6154
Epoch 889/1000
270/270 [==============================] - 0s 75us/step - loss: 0.6777 - accuracy: 0.7296 - val_loss: 1.0687 - val_accuracy: 0.5983
Epoch 890/1000
270/270 [==============================] - 0s 58us/step - loss: 0.6699 - accuracy: 0.7148 - val_loss: 1.1239 - val_accuracy: 0.5983
Epoch 891/1000
270/270 [==============================] - 0s 84us/step - loss: 0.6683 - accuracy: 0.7370 - val_loss: 1.0626 - val_accuracy: 0.5726
Epoch 892/1000
270/270 [==============================] - 0s 81us/step - loss: 0.6718 - accuracy: 0.7148 - val_loss: 1.0787 - val_accuracy: 0.5726
Epoch 893/1000
270/270 [==============================] - 0s 102us/step - loss: 0.6628 - accuracy: 0.7148 - val_loss: 1.0806 - val_accuracy: 0.6068
Epoch 894/1000
270/270 [==============================] - 0s 99us/step - loss: 0.6659 - accuracy: 0.7185 - val_loss: 1.0480 - val_a

Epoch 944/1000
270/270 [==============================] - 0s 94us/step - loss: 0.6583 - accuracy: 0.7296 - val_loss: 1.0547 - val_accuracy: 0.5812
Epoch 945/1000
270/270 [==============================] - 0s 97us/step - loss: 0.6618 - accuracy: 0.7259 - val_loss: 1.0697 - val_accuracy: 0.6154
Epoch 946/1000
270/270 [==============================] - 0s 104us/step - loss: 0.6581 - accuracy: 0.7296 - val_loss: 1.0607 - val_accuracy: 0.5726
Epoch 947/1000
270/270 [==============================] - 0s 142us/step - loss: 0.6597 - accuracy: 0.7222 - val_loss: 1.0937 - val_accuracy: 0.5983
Epoch 948/1000
270/270 [==============================] - 0s 114us/step - loss: 0.6819 - accuracy: 0.7185 - val_loss: 1.1505 - val_accuracy: 0.6154
Epoch 949/1000
270/270 [==============================] - 0s 94us/step - loss: 0.6667 - accuracy: 0.7259 - val_loss: 1.0557 - val_accuracy: 0.5641
Epoch 950/1000
270/270 [==============================] - 0s 117us/step - loss: 0.6714 - accuracy: 0.6926 - val_los

In [220]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over2.history['accuracy'])*100))

over-sampling train accuracy: 71.37%


In [44]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [45]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS249,2,1,1.888869e-01,5.108038e-01,3.003094e-01
1,p0006kpresabs_qual,NRS188,1,1,1.888869e-01,5.108038e-01,3.003094e-01
2,p0006kpresabs_qual,NRS232,2,2,4.222906e-01,7.029924e-02,5.074101e-01
3,p0006kpresabs_qual,NY439,2,2,3.558408e-04,2.976018e-04,9.993465e-01
4,p0006kpresabs_qual,GA27,2,1,3.940971e-01,4.184215e-01,1.874814e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS252,0,0,7.239556e-01,2.760444e-01,1.176030e-09
985,p0017Skpresabs_qual,SR2852,1,1,1.052276e-07,9.999999e-01,1.101559e-28
986,p0017Skpresabs_qual,NRS108,1,1,1.540350e-17,1.000000e+00,9.011977e-16
987,p0017Skpresabs_qual,NRS202,0,0,6.888959e-01,3.111042e-01,2.228958e-09


In [46]:
y_prob6 = df_proba6[df_proba6['phage']=='p0006kpresabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[2.87154600e-01, 9.56542800e-02, 6.17191140e-01],
       [8.67139100e-01, 8.68863760e-02, 4.59745750e-02],
       [4.02464360e-01, 3.05844680e-02, 5.66951200e-01],
       [4.07016800e-03, 1.39784740e-02, 9.81951360e-01],
       [6.92080700e-01, 1.58598290e-01, 1.49321110e-01],
       [2.87154600e-01, 9.56542800e-02, 6.17191140e-01],
       [6.92080700e-01, 1.58598290e-01, 1.49321110e-01],
       [5.79746900e-01, 3.15918860e-01, 1.04334330e-01],
       [6.92080700e-01, 1.58598290e-01, 1.49321110e-01],
       [4.07016800e-03, 1.39784740e-02, 9.81951360e-01],
       [4.27348500e-02, 1.08565520e-01, 8.48699600e-01],
       [1.32498760e-01, 2.02864860e-01, 6.64636400e-01],
       [1.84088360e-01, 2.97449180e-02, 7.86166700e-01],
       [8.81933900e-01, 4.00584400e-02, 7.80076600e-02],
       [1.25085180e-01, 5.62238160e-01, 3.12676700e-01],
       [6.41439760e-03, 6.84890400e-04, 9.92900700e-01],
       [3.47299000e-01, 5.57314000e-01, 9.53869700e-02],
       [6.92080700e-01, 1.58598

In [47]:
ovo6 = rocauc_ovo(y_sel_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.7286872671488056

In [48]:
ovr6 = rocauc_ovr(y_sel_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.7286872671488056

In [49]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel_over)

In [50]:
dat7 = pd.DataFrame(X_sel_test_over[:,-1])
dat7['test'] = y_sel_test_over

In [51]:
dat7

,0,test
0,NRS210,0
1,NRS205,2
2,312,2
3,GA15,2
4,SR4035,0
...,...,...
112,NRS265,2
113,CFBREBSa108,1
114,NY224,1
115,NRS386,2


In [52]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [53]:
model2_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [54]:
model2_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 472us/step - loss: 17.6857 - accuracy: 0.3333 - val_loss: 21.3819 - val_accuracy: 0.3333
Epoch 2/1000
270/270 [==============================] - 0s 142us/step - loss: 12.5583 - accuracy: 0.3111 - val_loss: 15.3782 - val_accuracy: 0.3248
Epoch 3/1000
270/270 [==============================] - 0s 124us/step - loss: 9.4249 - accuracy: 0.2704 - val_loss: 12.7497 - val_accuracy: 0.2991
Epoch 4/1000
270/270 [==============================] - 0s 201us/step - loss: 7.6449 - accuracy: 0.2852 - val_loss: 10.1050 - val_accuracy: 0.2735
Epoch 5/1000
270/270 [==============================] - 0s 270us/step - loss: 5.7325 - accuracy: 0.2519 - val_loss: 7.8803 - val_accuracy: 0.2906
Epoch 6/1000
270/270 [==============================] - 0s 76us/step - loss: 4.6877 - accuracy: 0.3000 - val_loss: 5.8561 - val_accuracy: 0.3248
Epoch 7/1000
270/270 [==============================] - 0s 69us/step - lo

Epoch 112/1000
270/270 [==============================] - 0s 124us/step - loss: 0.9225 - accuracy: 0.6111 - val_loss: 0.9342 - val_accuracy: 0.6068
Epoch 113/1000
270/270 [==============================] - 0s 85us/step - loss: 0.9225 - accuracy: 0.6148 - val_loss: 0.9952 - val_accuracy: 0.5897
Epoch 114/1000
270/270 [==============================] - 0s 88us/step - loss: 1.0887 - accuracy: 0.6370 - val_loss: 0.9742 - val_accuracy: 0.5556
Epoch 115/1000
270/270 [==============================] - 0s 105us/step - loss: 0.8811 - accuracy: 0.6222 - val_loss: 0.9274 - val_accuracy: 0.5983
Epoch 116/1000
270/270 [==============================] - 0s 97us/step - loss: 1.1196 - accuracy: 0.6519 - val_loss: 1.1413 - val_accuracy: 0.5641
Epoch 117/1000
270/270 [==============================] - 0s 90us/step - loss: 1.1813 - accuracy: 0.5815 - val_loss: 1.1548 - val_accuracy: 0.5043
Epoch 118/1000
270/270 [==============================] - 0s 126us/step - loss: 0.9324 - accuracy: 0.6185 - val_loss

Epoch 167/1000
270/270 [==============================] - 0s 125us/step - loss: 0.9068 - accuracy: 0.6296 - val_loss: 0.9048 - val_accuracy: 0.5897
Epoch 168/1000
270/270 [==============================] - 0s 133us/step - loss: 1.1805 - accuracy: 0.6222 - val_loss: 1.5372 - val_accuracy: 0.5470
Epoch 169/1000
270/270 [==============================] - 0s 120us/step - loss: 1.1411 - accuracy: 0.6185 - val_loss: 1.0043 - val_accuracy: 0.5897
Epoch 170/1000
270/270 [==============================] - 0s 116us/step - loss: 0.9489 - accuracy: 0.6296 - val_loss: 0.8981 - val_accuracy: 0.6154
Epoch 171/1000
270/270 [==============================] - 0s 121us/step - loss: 0.8757 - accuracy: 0.6481 - val_loss: 0.9819 - val_accuracy: 0.5897
Epoch 172/1000
270/270 [==============================] - 0s 106us/step - loss: 0.9426 - accuracy: 0.6407 - val_loss: 0.8866 - val_accuracy: 0.6154
Epoch 173/1000
270/270 [==============================] - 0s 126us/step - loss: 0.8069 - accuracy: 0.6667 - val_

270/270 [==============================] - 0s 121us/step - loss: 0.8246 - accuracy: 0.6667 - val_loss: 0.9006 - val_accuracy: 0.6154
Epoch 278/1000
270/270 [==============================] - 0s 131us/step - loss: 0.8235 - accuracy: 0.6815 - val_loss: 0.9336 - val_accuracy: 0.5726
Epoch 279/1000
270/270 [==============================] - 0s 116us/step - loss: 0.7766 - accuracy: 0.6852 - val_loss: 0.9227 - val_accuracy: 0.6239
Epoch 280/1000
270/270 [==============================] - 0s 96us/step - loss: 0.7835 - accuracy: 0.6815 - val_loss: 1.0378 - val_accuracy: 0.5812
Epoch 281/1000
270/270 [==============================] - 0s 96us/step - loss: 0.8093 - accuracy: 0.6630 - val_loss: 0.8614 - val_accuracy: 0.6154
Epoch 282/1000
270/270 [==============================] - 0s 126us/step - loss: 0.7594 - accuracy: 0.6963 - val_loss: 0.8630 - val_accuracy: 0.6410
Epoch 283/1000
270/270 [==============================] - 0s 130us/step - loss: 0.7673 - accuracy: 0.6815 - val_loss: 0.8800 - va

Epoch 388/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7539 - accuracy: 0.6926 - val_loss: 0.8912 - val_accuracy: 0.5983
Epoch 389/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7446 - accuracy: 0.6926 - val_loss: 0.8667 - val_accuracy: 0.6154
Epoch 390/1000
270/270 [==============================] - 0s 69us/step - loss: 0.7400 - accuracy: 0.7074 - val_loss: 0.9358 - val_accuracy: 0.5983
Epoch 391/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7657 - accuracy: 0.6852 - val_loss: 0.8703 - val_accuracy: 0.6068
Epoch 392/1000
270/270 [==============================] - 0s 63us/step - loss: 0.7663 - accuracy: 0.6852 - val_loss: 0.8976 - val_accuracy: 0.6239
Epoch 393/1000
270/270 [==============================] - 0s 58us/step - loss: 0.7952 - accuracy: 0.6593 - val_loss: 1.0296 - val_accuracy: 0.5812
Epoch 394/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8392 - accuracy: 0.6815 - val_loss: 1

...

Epoch 721/1000
270/270 [==============================] - 0s 95us/step - loss: 0.7131 - accuracy: 0.6926 - val_loss: 0.8604 - val_accuracy: 0.6325
Epoch 722/1000
270/270 [==============================] - 0s 94us/step - loss: 0.7129 - accuracy: 0.6852 - val_loss: 0.9056 - val_accuracy: 0.5983
Epoch 723/1000
270/270 [==============================] - 0s 85us/step - loss: 0.7123 - accuracy: 0.7037 - val_loss: 0.8826 - val_accuracy: 0.6325
Epoch 724/1000
270/270 [==============================] - 0s 86us/step - loss: 0.7473 - accuracy: 0.6741 - val_loss: 0.8595 - val_accuracy: 0.6325
Epoch 725/1000
270/270 [==============================] - 0s 86us/step - loss: 0.7222 - accuracy: 0.6963 - val_loss: 0.8672 - val_accuracy: 0.6325
Epoch 726/1000
270/270 [==============================] - 0s 95us/step - loss: 0.7309 - accuracy: 0.7000 - val_loss: 0.8737 - val_accuracy: 0.6410
Epoch 727/1000
270/270 [==============================] - 0s 93us/step - loss: 0.7129 - accuracy: 0.6963 - val_loss: 0

Epoch 777/1000
270/270 [==============================] - 0s 85us/step - loss: 0.7441 - accuracy: 0.6593 - val_loss: 0.8661 - val_accuracy: 0.6325
Epoch 778/1000
270/270 [==============================] - 0s 86us/step - loss: 0.7416 - accuracy: 0.6815 - val_loss: 0.9036 - val_accuracy: 0.6239
Epoch 779/1000
270/270 [==============================] - 0s 99us/step - loss: 0.9376 - accuracy: 0.6741 - val_loss: 1.0143 - val_accuracy: 0.6239
Epoch 780/1000
270/270 [==============================] - 0s 84us/step - loss: 0.7782 - accuracy: 0.7074 - val_loss: 0.9837 - val_accuracy: 0.6068
Epoch 781/1000
270/270 [==============================] - 0s 85us/step - loss: 0.7154 - accuracy: 0.7148 - val_loss: 0.8941 - val_accuracy: 0.6239
Epoch 782/1000
270/270 [==============================] - 0s 85us/step - loss: 0.7261 - accuracy: 0.6963 - val_loss: 0.9688 - val_accuracy: 0.5812
Epoch 783/1000
270/270 [==============================] - 0s 80us/step - loss: 0.7702 - accuracy: 0.6815 - val_loss: 1

Epoch 888/1000
270/270 [==============================] - 0s 81us/step - loss: 0.7386 - accuracy: 0.7074 - val_loss: 0.8945 - val_accuracy: 0.6154
Epoch 889/1000
270/270 [==============================] - 0s 91us/step - loss: 0.7520 - accuracy: 0.6704 - val_loss: 0.8892 - val_accuracy: 0.6325
Epoch 890/1000
270/270 [==============================] - 0s 126us/step - loss: 0.7172 - accuracy: 0.6852 - val_loss: 0.8883 - val_accuracy: 0.6154
Epoch 891/1000
270/270 [==============================] - 0s 179us/step - loss: 0.7202 - accuracy: 0.6963 - val_loss: 0.9011 - val_accuracy: 0.6239
Epoch 892/1000
270/270 [==============================] - 0s 189us/step - loss: 0.7131 - accuracy: 0.7037 - val_loss: 0.8887 - val_accuracy: 0.6325
Epoch 893/1000
270/270 [==============================] - 0s 188us/step - loss: 0.7158 - accuracy: 0.6963 - val_loss: 0.9839 - val_accuracy: 0.5897
Epoch 894/1000
270/270 [==============================] - 0s 127us/step - loss: 0.7365 - accuracy: 0.6852 - val_lo

Epoch 944/1000
270/270 [==============================] - 0s 88us/step - loss: 0.7563 - accuracy: 0.6778 - val_loss: 1.1338 - val_accuracy: 0.5983
Epoch 945/1000
270/270 [==============================] - 0s 83us/step - loss: 0.8348 - accuracy: 0.6926 - val_loss: 1.2089 - val_accuracy: 0.6154
Epoch 946/1000
270/270 [==============================] - 0s 85us/step - loss: 1.2247 - accuracy: 0.6852 - val_loss: 0.9734 - val_accuracy: 0.5983
Epoch 947/1000
270/270 [==============================] - 0s 84us/step - loss: 0.7848 - accuracy: 0.7000 - val_loss: 0.8997 - val_accuracy: 0.6325
Epoch 948/1000
270/270 [==============================] - 0s 82us/step - loss: 0.7662 - accuracy: 0.6963 - val_loss: 0.9325 - val_accuracy: 0.5812
Epoch 949/1000
270/270 [==============================] - 0s 80us/step - loss: 0.7277 - accuracy: 0.6926 - val_loss: 0.8882 - val_accuracy: 0.6239
Epoch 950/1000
270/270 [==============================] - 0s 81us/step - loss: 0.7229 - accuracy: 0.6741 - val_loss: 0

Epoch 1000/1000
270/270 [==============================] - 0s 87us/step - loss: 0.7394 - accuracy: 0.7000 - val_loss: 0.9679 - val_accuracy: 0.6410


In [237]:
acc_test2_over3 = model2_over3.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over3*100))

117/117 [==============================] - 0s 87us/step
over-sampling test accuracy: 67.52%


In [112]:
pred7 = model2_over3.predict_classes(X_sel_test_over)
pred7

array([0, 2, 2, 0, 0, 2, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 2, 0, 2, 2, 0, 0,
       0, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 2, 0, 1, 2, 0,
       1, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 1, 2, 2, 1, 2, 0, 2, 1,
       2, 0, 0, 2, 1, 1, 2, 0, 2, 1, 0, 0, 2, 2, 2, 0, 1, 0, 0, 0, 0, 1,
       2, 1, 2, 0, 2, 2, 0, 2, 1, 1, 0, 2, 0, 1, 1, 0, 0, 2, 2, 0, 2, 0,
       0, 1, 2, 0, 1, 2, 2])

In [113]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS210,0,0
1,NRS205,2,2
2,312,2,2
3,GA15,2,0
4,SR4035,0,0
...,...,...,...
112,NRS265,2,2
113,CFBREBSa108,1,0
114,NY224,1,1
115,NRS386,2,2


In [114]:
proba7 = model2_over3.predict_proba(X_sel_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [115]:
dat_proba7

,0,1,2
0,0.767829,0.230110,0.002061
1,0.069677,0.000197,0.930126
2,0.021005,0.016618,0.962377
3,0.580774,0.222390,0.196836
4,0.628776,0.370517,0.000707
...,...,...,...
112,0.294361,0.066527,0.639111
113,0.968010,0.016245,0.015745
114,0.244494,0.710529,0.044977
115,0.087958,0.190353,0.721689


In [116]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [117]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p006ST.csv", index = False,
         header=None)

In [241]:
hist2_over3 = model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 121us/step - loss: 0.6618 - accuracy: 0.7111 - val_loss: 0.8234 - val_accuracy: 0.6667
Epoch 2/1000
270/270 [==============================] - 0s 127us/step - loss: 0.6797 - accuracy: 0.7148 - val_loss: 0.8956 - val_accuracy: 0.6496
Epoch 3/1000
270/270 [==============================] - 0s 169us/step - loss: 0.7890 - accuracy: 0.6778 - val_loss: 0.9446 - val_accuracy: 0.6239
Epoch 4/1000
270/270 [==============================] - 0s 141us/step - loss: 0.7020 - accuracy: 0.7148 - val_loss: 0.9373 - val_accuracy: 0.5983
Epoch 5/1000
270/270 [==============================] - 0s 137us/step - loss: 0.6856 - accuracy: 0.6963 - val_loss: 0.8586 - val_accuracy: 0.6581
Epoch 6/1000
270/270 [==============================] - 0s 120us/step - loss: 0.7312 - accuracy: 0.6963 - val_loss: 0.8686 - val_accuracy: 0.6410
Epoch 7/1000
270/270 [==============================] - 0s 117us/step - loss: 

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.124900). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


270/270 [==============================] - 0s 238us/step - loss: 0.6632 - accuracy: 0.7111 - val_loss: 0.8408 - val_accuracy: 0.6496
Epoch 92/1000
270/270 [==============================] - 0s 316us/step - loss: 0.6609 - accuracy: 0.6963 - val_loss: 0.8271 - val_accuracy: 0.6496
Epoch 93/1000
270/270 [==============================] - 0s 151us/step - loss: 0.6531 - accuracy: 0.7111 - val_loss: 0.8266 - val_accuracy: 0.6581
Epoch 94/1000
270/270 [==============================] - 0s 103us/step - loss: 0.7020 - accuracy: 0.7111 - val_loss: 0.8641 - val_accuracy: 0.6581
Epoch 95/1000
270/270 [==============================] - 0s 120us/step - loss: 0.6818 - accuracy: 0.7148 - val_loss: 0.8371 - val_accuracy: 0.6752
Epoch 96/1000
270/270 [==============================] - 0s 230us/step - loss: 0.6784 - accuracy: 0.7148 - val_loss: 0.8403 - val_accuracy: 0.6496
Epoch 97/1000
270/270 [==============================] - 0s 129us/step - loss: 0.6958 - accuracy: 0.6926 - val_loss: 0.8277 - val_ac

270/270 [==============================] - 0s 312us/step - loss: 0.6515 - accuracy: 0.7148 - val_loss: 0.8455 - val_accuracy: 0.6667
Epoch 202/1000
270/270 [==============================] - 0s 109us/step - loss: 0.7062 - accuracy: 0.7074 - val_loss: 0.9525 - val_accuracy: 0.6581
Epoch 203/1000
270/270 [==============================] - 0s 146us/step - loss: 0.7330 - accuracy: 0.7185 - val_loss: 0.8698 - val_accuracy: 0.6154
Epoch 204/1000
270/270 [==============================] - 0s 206us/step - loss: 0.7234 - accuracy: 0.6963 - val_loss: 0.8397 - val_accuracy: 0.6752
Epoch 205/1000
270/270 [==============================] - 0s 213us/step - loss: 0.6769 - accuracy: 0.7148 - val_loss: 0.8752 - val_accuracy: 0.6325
Epoch 206/1000
270/270 [==============================] - 0s 137us/step - loss: 0.6817 - accuracy: 0.7148 - val_loss: 0.8258 - val_accuracy: 0.6581
Epoch 207/1000
270/270 [==============================] - 0s 127us/step - loss: 0.6620 - accuracy: 0.7074 - val_loss: 0.8578 - 

270/270 [==============================] - 0s 325us/step - loss: 0.6364 - accuracy: 0.7296 - val_loss: 0.8367 - val_accuracy: 0.6496
Epoch 312/1000
270/270 [==============================] - 0s 178us/step - loss: 0.6459 - accuracy: 0.7296 - val_loss: 0.8127 - val_accuracy: 0.6752
Epoch 313/1000
270/270 [==============================] - 0s 455us/step - loss: 0.6393 - accuracy: 0.7222 - val_loss: 0.8443 - val_accuracy: 0.6410
Epoch 314/1000
270/270 [==============================] - 0s 120us/step - loss: 0.6373 - accuracy: 0.7222 - val_loss: 0.8144 - val_accuracy: 0.6838
Epoch 315/1000
270/270 [==============================] - 0s 102us/step - loss: 0.6448 - accuracy: 0.7259 - val_loss: 0.8289 - val_accuracy: 0.6581
Epoch 316/1000
270/270 [==============================] - 0s 122us/step - loss: 0.6330 - accuracy: 0.7370 - val_loss: 0.8123 - val_accuracy: 0.6923
Epoch 317/1000
270/270 [==============================] - 0s 509us/step - loss: 0.6337 - accuracy: 0.7407 - val_loss: 0.8125 - 

...

270/270 [==============================] - 0s 204us/step - loss: 0.6205 - accuracy: 0.7444 - val_loss: 0.8292 - val_accuracy: 0.6667
Epoch 532/1000
270/270 [==============================] - 0s 162us/step - loss: 0.6386 - accuracy: 0.7148 - val_loss: 0.8350 - val_accuracy: 0.6838
Epoch 533/1000
270/270 [==============================] - 0s 113us/step - loss: 0.6310 - accuracy: 0.7296 - val_loss: 0.8089 - val_accuracy: 0.6923
Epoch 534/1000
270/270 [==============================] - 0s 194us/step - loss: 0.6309 - accuracy: 0.7259 - val_loss: 0.8108 - val_accuracy: 0.6838
Epoch 535/1000
270/270 [==============================] - 0s 293us/step - loss: 0.6368 - accuracy: 0.7185 - val_loss: 0.8083 - val_accuracy: 0.6667
Epoch 536/1000
270/270 [==============================] - 0s 122us/step - loss: 0.6209 - accuracy: 0.7444 - val_loss: 0.8072 - val_accuracy: 0.6838
Epoch 537/1000
270/270 [==============================] - 0s 117us/step - loss: 0.6148 - accuracy: 0.7444 - val_loss: 0.8073 - 

270/270 [==============================] - 0s 159us/step - loss: 0.6572 - accuracy: 0.7370 - val_loss: 0.8582 - val_accuracy: 0.6581
Epoch 642/1000
270/270 [==============================] - 0s 182us/step - loss: 0.6164 - accuracy: 0.7333 - val_loss: 0.8079 - val_accuracy: 0.6667
Epoch 643/1000
270/270 [==============================] - 0s 166us/step - loss: 0.6076 - accuracy: 0.7333 - val_loss: 0.8350 - val_accuracy: 0.6410
Epoch 644/1000
270/270 [==============================] - 0s 111us/step - loss: 0.6206 - accuracy: 0.7259 - val_loss: 0.8066 - val_accuracy: 0.6752
Epoch 645/1000
270/270 [==============================] - 0s 216us/step - loss: 0.6047 - accuracy: 0.7519 - val_loss: 0.8070 - val_accuracy: 0.6923
Epoch 646/1000
270/270 [==============================] - 0s 155us/step - loss: 0.6025 - accuracy: 0.7519 - val_loss: 0.8044 - val_accuracy: 0.6838
Epoch 647/1000
270/270 [==============================] - 0s 107us/step - loss: 0.6234 - accuracy: 0.7370 - val_loss: 0.8462 - 

270/270 [==============================] - 0s 64us/step - loss: 0.6082 - accuracy: 0.7407 - val_loss: 0.8079 - val_accuracy: 0.6838
Epoch 752/1000
270/270 [==============================] - 0s 393us/step - loss: 0.6012 - accuracy: 0.7444 - val_loss: 0.8103 - val_accuracy: 0.6752
Epoch 753/1000
270/270 [==============================] - 0s 378us/step - loss: 0.6042 - accuracy: 0.7444 - val_loss: 0.8120 - val_accuracy: 0.6752
Epoch 754/1000
270/270 [==============================] - 0s 269us/step - loss: 0.6105 - accuracy: 0.7370 - val_loss: 0.8177 - val_accuracy: 0.6752
Epoch 755/1000
270/270 [==============================] - 0s 321us/step - loss: 0.6177 - accuracy: 0.7444 - val_loss: 0.8102 - val_accuracy: 0.6752
Epoch 756/1000
270/270 [==============================] - 0s 337us/step - loss: 0.6086 - accuracy: 0.7407 - val_loss: 0.8083 - val_accuracy: 0.6752
Epoch 757/1000
270/270 [==============================] - 0s 128us/step - loss: 0.6019 - accuracy: 0.7407 - val_loss: 0.8142 - v

270/270 [==============================] - 0s 108us/step - loss: 0.6040 - accuracy: 0.7333 - val_loss: 0.8270 - val_accuracy: 0.6667
Epoch 862/1000
270/270 [==============================] - 0s 133us/step - loss: 0.6016 - accuracy: 0.7444 - val_loss: 0.8222 - val_accuracy: 0.6752
Epoch 863/1000
270/270 [==============================] - 0s 105us/step - loss: 0.5986 - accuracy: 0.7444 - val_loss: 0.8171 - val_accuracy: 0.6667
Epoch 864/1000
270/270 [==============================] - 0s 93us/step - loss: 0.5993 - accuracy: 0.7370 - val_loss: 0.8259 - val_accuracy: 0.6667
Epoch 865/1000
270/270 [==============================] - 0s 119us/step - loss: 0.5921 - accuracy: 0.7444 - val_loss: 0.8201 - val_accuracy: 0.6667
Epoch 866/1000
270/270 [==============================] - 0s 116us/step - loss: 0.6037 - accuracy: 0.7444 - val_loss: 0.8219 - val_accuracy: 0.6752
Epoch 867/1000
270/270 [==============================] - 0s 113us/step - loss: 0.6059 - accuracy: 0.7296 - val_loss: 0.8243 - v

270/270 [==============================] - 0s 106us/step - loss: 0.6275 - accuracy: 0.7444 - val_loss: 1.0817 - val_accuracy: 0.6325
Epoch 972/1000
270/270 [==============================] - 0s 99us/step - loss: 0.7105 - accuracy: 0.7259 - val_loss: 0.8871 - val_accuracy: 0.6581
Epoch 973/1000
270/270 [==============================] - 0s 107us/step - loss: 0.6660 - accuracy: 0.7296 - val_loss: 0.8671 - val_accuracy: 0.6581
Epoch 974/1000
270/270 [==============================] - 0s 96us/step - loss: 0.5992 - accuracy: 0.7481 - val_loss: 0.8168 - val_accuracy: 0.6838
Epoch 975/1000
270/270 [==============================] - 0s 107us/step - loss: 0.6355 - accuracy: 0.7222 - val_loss: 0.8418 - val_accuracy: 0.6667
Epoch 976/1000
270/270 [==============================] - 0s 128us/step - loss: 0.6393 - accuracy: 0.7333 - val_loss: 0.8526 - val_accuracy: 0.6667
Epoch 977/1000
270/270 [==============================] - 0s 104us/step - loss: 0.6294 - accuracy: 0.7296 - val_loss: 0.8428 - va

In [242]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over3.history['accuracy'])*100))

over-sampling train accuracy: 72.55%


In [55]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [56]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS210,0,0,6.132076e-01,2.812180e-01,1.055744e-01
1,p0006kpresabs_qual,NRS205,2,2,1.993202e-04,6.834937e-07,9.998000e-01
2,p0006kpresabs_qual,312,2,1,3.589463e-01,3.982787e-01,2.427750e-01
3,p0006kpresabs_qual,GA15,2,1,3.589463e-01,3.982787e-01,2.427750e-01
4,p0006kpresabs_qual,SR4035,0,1,3.589463e-01,3.982787e-01,2.427750e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS383,1,0,5.477194e-01,4.522807e-01,1.761374e-08
985,p0017Skpresabs_qual,NRS218,1,1,6.953657e-05,9.999305e-01,3.132419e-10
986,p0017Skpresabs_qual,NRS209,2,2,2.713214e-09,6.656316e-09,1.000000e+00
987,p0017Skpresabs_qual,SR2852,1,1,9.956684e-12,1.000000e+00,7.441288e-26


In [57]:
y_prob7 = df_proba7[df_proba7['phage']=='p0006kpresabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[7.67828900e-01, 2.30110440e-01, 2.06067500e-03],
       [6.96772800e-02, 1.96709080e-04, 9.30126100e-01],
       [2.10048200e-02, 1.66178460e-02, 9.62377300e-01],
       [5.80773600e-01, 2.22390470e-01, 1.96835900e-01],
       [6.28776250e-01, 3.70516660e-01, 7.07122300e-04],
       [4.51863560e-02, 2.77519600e-02, 9.27061740e-01],
       [3.35063100e-01, 6.64909500e-01, 2.73770000e-05],
       [2.70308000e-01, 3.80303600e-01, 3.49388450e-01],
       [5.80773600e-01, 2.22390470e-01, 1.96835900e-01],
       [9.72553300e-01, 2.74464530e-02, 1.92193720e-07],
       [2.46529220e-01, 6.02341650e-01, 1.51129140e-01],
       [8.85721100e-01, 1.06108680e-01, 8.17020000e-03],
       [1.66191280e-01, 6.49790940e-01, 1.84017780e-01],
       [7.99674030e-01, 7.64478600e-02, 1.23878084e-01],
       [2.46529220e-01, 6.02341650e-01, 1.51129140e-01],
       [2.46529220e-01, 6.02341650e-01, 1.51129140e-01],
       [3.14755740e-04, 1.22953540e-01, 8.76731750e-01],
       [6.97452900e-01, 1.02297

In [58]:
ovo7 = rocauc_ovo(y_sel_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.7888450580758274

In [59]:
ovr7 = rocauc_ovr(y_sel_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.7888450580758274

In [60]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel_over)

In [61]:
dat8 = pd.DataFrame(X_sel_test_over[:,-1])
dat8['test'] = y_sel_test_over

In [62]:
dat8

,0,test
0,NRS236,1
1,NRS113,2
2,CFBRSa23,0
3,NRS249,2
4,107,1
...,...,...
112,NY439,2
113,NRS106,0
114,221,0
115,NRS386,2


In [63]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [64]:
model2_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [123]:
model2_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [124]:
model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 603us/step - loss: 14.5086 - accuracy: 0.3852 - val_loss: 11.2833 - val_accuracy: 0.4786
Epoch 2/1000
270/270 [==============================] - 0s 71us/step - loss: 8.4163 - accuracy: 0.4667 - val_loss: 4.9167 - val_accuracy: 0.4444
Epoch 3/1000
270/270 [==============================] - 0s 88us/step - loss: 3.3838 - accuracy: 0.4333 - val_loss: 2.7282 - val_accuracy: 0.3675
Epoch 4/1000
270/270 [==============================] - 0s 123us/step - loss: 2.8056 - accuracy: 0.3630 - val_loss: 2.7566 - val_accuracy: 0.3761
Epoch 5/1000
270/270 [==============================] - 0s 91us/step - loss: 2.6665 - accuracy: 0.3519 - val_loss: 2.6052 - val_accuracy: 0.3590
Epoch 6/1000
270/270 [==============================] - 0s 102us/step - loss: 2.5098 - accuracy: 0.3593 - val_loss: 1.9119 - val_accuracy: 0.4017
Epoch 7/1000
270/270 [==============================] - 0s 148us/step - loss: 1

Epoch 57/1000
270/270 [==============================] - 0s 65us/step - loss: 1.0729 - accuracy: 0.5333 - val_loss: 0.9908 - val_accuracy: 0.4957
Epoch 58/1000
270/270 [==============================] - 0s 57us/step - loss: 0.9849 - accuracy: 0.5296 - val_loss: 1.0150 - val_accuracy: 0.4615
Epoch 59/1000
270/270 [==============================] - 0s 92us/step - loss: 0.9701 - accuracy: 0.5333 - val_loss: 1.0035 - val_accuracy: 0.4957
Epoch 60/1000
270/270 [==============================] - 0s 88us/step - loss: 1.0692 - accuracy: 0.5333 - val_loss: 1.1864 - val_accuracy: 0.4530
Epoch 61/1000
270/270 [==============================] - 0s 91us/step - loss: 1.1125 - accuracy: 0.5407 - val_loss: 1.0122 - val_accuracy: 0.5043
Epoch 62/1000
270/270 [==============================] - 0s 140us/step - loss: 1.0253 - accuracy: 0.5444 - val_loss: 1.0248 - val_accuracy: 0.5214
Epoch 63/1000
270/270 [==============================] - 0s 200us/step - loss: 1.0509 - accuracy: 0.5370 - val_loss: 1.1468

Epoch 168/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8225 - accuracy: 0.6259 - val_loss: 0.9367 - val_accuracy: 0.6068
Epoch 169/1000
270/270 [==============================] - 0s 101us/step - loss: 0.8279 - accuracy: 0.6481 - val_loss: 0.9401 - val_accuracy: 0.5812
Epoch 170/1000
270/270 [==============================] - 0s 64us/step - loss: 0.8088 - accuracy: 0.6259 - val_loss: 0.9798 - val_accuracy: 0.5983
Epoch 171/1000
270/270 [==============================] - 0s 75us/step - loss: 0.8830 - accuracy: 0.6074 - val_loss: 0.9518 - val_accuracy: 0.5812
Epoch 172/1000
270/270 [==============================] - 0s 80us/step - loss: 0.8479 - accuracy: 0.6111 - val_loss: 0.9880 - val_accuracy: 0.6325
Epoch 173/1000
270/270 [==============================] - 0s 85us/step - loss: 0.8917 - accuracy: 0.6185 - val_loss: 1.0703 - val_accuracy: 0.5812
Epoch 174/1000
270/270 [==============================] - 0s 61us/step - loss: 0.8769 - accuracy: 0.5815 - val_loss: 

Epoch 224/1000
270/270 [==============================] - 0s 58us/step - loss: 0.7822 - accuracy: 0.6444 - val_loss: 0.9851 - val_accuracy: 0.5897
Epoch 225/1000
270/270 [==============================] - 0s 57us/step - loss: 0.8208 - accuracy: 0.6519 - val_loss: 0.9216 - val_accuracy: 0.5641
Epoch 226/1000
270/270 [==============================] - 0s 50us/step - loss: 0.7910 - accuracy: 0.6296 - val_loss: 0.9270 - val_accuracy: 0.5812
Epoch 227/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7965 - accuracy: 0.6593 - val_loss: 0.9269 - val_accuracy: 0.6068
Epoch 228/1000
270/270 [==============================] - 0s 78us/step - loss: 0.7997 - accuracy: 0.6333 - val_loss: 0.9214 - val_accuracy: 0.4444
Epoch 229/1000
270/270 [==============================] - 0s 57us/step - loss: 0.7877 - accuracy: 0.6148 - val_loss: 0.9444 - val_accuracy: 0.5812
Epoch 230/1000
270/270 [==============================] - 0s 53us/step - loss: 0.7892 - accuracy: 0.6519 - val_loss: 0

Epoch 280/1000
270/270 [==============================] - 0s 139us/step - loss: 0.8979 - accuracy: 0.6333 - val_loss: 0.9182 - val_accuracy: 0.5897
Epoch 281/1000
270/270 [==============================] - 0s 66us/step - loss: 0.8306 - accuracy: 0.6444 - val_loss: 1.0346 - val_accuracy: 0.6068
Epoch 282/1000
270/270 [==============================] - 0s 72us/step - loss: 0.9098 - accuracy: 0.6519 - val_loss: 1.0345 - val_accuracy: 0.5897
Epoch 283/1000
270/270 [==============================] - 0s 90us/step - loss: 0.8483 - accuracy: 0.6074 - val_loss: 0.9714 - val_accuracy: 0.5897
Epoch 284/1000
270/270 [==============================] - 0s 93us/step - loss: 0.9499 - accuracy: 0.6519 - val_loss: 1.2666 - val_accuracy: 0.5983
Epoch 285/1000
270/270 [==============================] - 0s 57us/step - loss: 0.9556 - accuracy: 0.6556 - val_loss: 1.0429 - val_accuracy: 0.5641
Epoch 286/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8941 - accuracy: 0.5815 - val_loss: 

...

Epoch 725/1000
270/270 [==============================] - 0s 68us/step - loss: 0.7063 - accuracy: 0.6926 - val_loss: 0.9769 - val_accuracy: 0.6239
Epoch 726/1000
270/270 [==============================] - 0s 64us/step - loss: 0.6968 - accuracy: 0.6889 - val_loss: 0.9526 - val_accuracy: 0.6325
Epoch 727/1000
270/270 [==============================] - 0s 71us/step - loss: 0.7040 - accuracy: 0.6778 - val_loss: 0.9813 - val_accuracy: 0.6239
Epoch 728/1000
270/270 [==============================] - 0s 73us/step - loss: 0.7287 - accuracy: 0.6889 - val_loss: 0.9864 - val_accuracy: 0.6325
Epoch 729/1000
270/270 [==============================] - 0s 60us/step - loss: 0.7046 - accuracy: 0.6926 - val_loss: 1.0329 - val_accuracy: 0.6239
Epoch 730/1000
270/270 [==============================] - 0s 71us/step - loss: 0.7329 - accuracy: 0.6704 - val_loss: 1.0052 - val_accuracy: 0.6410
Epoch 731/1000
270/270 [==============================] - 0s 67us/step - loss: 0.7317 - accuracy: 0.6963 - val_loss: 0

Epoch 781/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7240 - accuracy: 0.6889 - val_loss: 0.9743 - val_accuracy: 0.6325
Epoch 782/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7062 - accuracy: 0.6889 - val_loss: 0.9363 - val_accuracy: 0.6410
Epoch 783/1000
270/270 [==============================] - 0s 65us/step - loss: 0.6930 - accuracy: 0.6926 - val_loss: 0.9733 - val_accuracy: 0.6325
Epoch 784/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7023 - accuracy: 0.6926 - val_loss: 0.9504 - val_accuracy: 0.6410
Epoch 785/1000
270/270 [==============================] - 0s 59us/step - loss: 0.7849 - accuracy: 0.6630 - val_loss: 1.0354 - val_accuracy: 0.6325
Epoch 786/1000
270/270 [==============================] - 0s 62us/step - loss: 0.8517 - accuracy: 0.6741 - val_loss: 1.4609 - val_accuracy: 0.6325
Epoch 787/1000
270/270 [==============================] - 0s 63us/step - loss: 1.0151 - accuracy: 0.6815 - val_loss: 1

Epoch 837/1000
270/270 [==============================] - 0s 80us/step - loss: 0.7153 - accuracy: 0.6889 - val_loss: 0.9909 - val_accuracy: 0.6496
Epoch 838/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7102 - accuracy: 0.6963 - val_loss: 1.0013 - val_accuracy: 0.6239
Epoch 839/1000
270/270 [==============================] - 0s 72us/step - loss: 0.7280 - accuracy: 0.6889 - val_loss: 0.9742 - val_accuracy: 0.6239
Epoch 840/1000
270/270 [==============================] - 0s 70us/step - loss: 0.7268 - accuracy: 0.6778 - val_loss: 1.0288 - val_accuracy: 0.6410
Epoch 841/1000
270/270 [==============================] - 0s 74us/step - loss: 0.7020 - accuracy: 0.6963 - val_loss: 0.9837 - val_accuracy: 0.6325
Epoch 842/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7088 - accuracy: 0.6889 - val_loss: 0.9638 - val_accuracy: 0.6325
Epoch 843/1000
270/270 [==============================] - 0s 69us/step - loss: 0.7015 - accuracy: 0.6889 - val_loss: 0

Epoch 893/1000
270/270 [==============================] - 0s 66us/step - loss: 0.7159 - accuracy: 0.6852 - val_loss: 0.9483 - val_accuracy: 0.6496
Epoch 894/1000
270/270 [==============================] - 0s 65us/step - loss: 0.7148 - accuracy: 0.6741 - val_loss: 1.0606 - val_accuracy: 0.6154
Epoch 895/1000
270/270 [==============================] - 0s 81us/step - loss: 0.7079 - accuracy: 0.6593 - val_loss: 1.0373 - val_accuracy: 0.6325
Epoch 896/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7098 - accuracy: 0.6926 - val_loss: 0.9483 - val_accuracy: 0.6239
Epoch 897/1000
270/270 [==============================] - 0s 60us/step - loss: 0.7181 - accuracy: 0.6852 - val_loss: 0.9851 - val_accuracy: 0.6154
Epoch 898/1000
270/270 [==============================] - 0s 73us/step - loss: 0.7178 - accuracy: 0.6926 - val_loss: 1.0078 - val_accuracy: 0.6239
Epoch 899/1000
270/270 [==============================] - 0s 68us/step - loss: 0.7093 - accuracy: 0.6741 - val_loss: 0

Epoch 949/1000
270/270 [==============================] - 0s 49us/step - loss: 0.6817 - accuracy: 0.6815 - val_loss: 0.9704 - val_accuracy: 0.6667
Epoch 950/1000
270/270 [==============================] - 0s 56us/step - loss: 0.6804 - accuracy: 0.6963 - val_loss: 0.9986 - val_accuracy: 0.6068
Epoch 951/1000
270/270 [==============================] - 0s 53us/step - loss: 0.6936 - accuracy: 0.6815 - val_loss: 0.9766 - val_accuracy: 0.6581
Epoch 952/1000
270/270 [==============================] - 0s 73us/step - loss: 0.6783 - accuracy: 0.7000 - val_loss: 1.0092 - val_accuracy: 0.6239
Epoch 953/1000
270/270 [==============================] - 0s 60us/step - loss: 0.7187 - accuracy: 0.6963 - val_loss: 1.0501 - val_accuracy: 0.6410
Epoch 954/1000
270/270 [==============================] - 0s 64us/step - loss: 0.7472 - accuracy: 0.6963 - val_loss: 1.0519 - val_accuracy: 0.6410
Epoch 955/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6982 - accuracy: 0.6926 - val_loss: 1

In [261]:
acc_test2_over4 = model2_over4.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over4*100))

117/117 [==============================] - 0s 84us/step
over-sampling test accuracy: 64.10%


In [125]:
pred8 = model2_over4.predict_classes(X_sel_test_over)
pred8

array([2, 2, 0, 2, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 0, 2, 2, 1, 2, 2, 1, 0,
       0, 0, 1, 1, 1, 0, 2, 1, 0, 2, 2, 0, 1, 1, 0, 2, 0, 1, 1, 2, 2, 2,
       2, 2, 0, 2, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2,
       2, 1, 1, 2, 0, 1, 2, 1, 1, 0, 2, 2, 1, 0, 0, 2, 1, 0, 1, 0, 2, 1,
       1, 0, 1, 1, 0, 0, 2, 1, 0, 2, 1, 2, 1, 2, 1, 1, 1, 0, 2, 2, 1, 1,
       1, 1, 2, 0, 1, 2, 1])

In [126]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,NRS236,1,2
1,NRS113,2,2
2,CFBRSa23,0,0
3,NRS249,2,2
4,107,1,1
...,...,...,...
112,NY439,2,2
113,NRS106,0,0
114,221,0,1
115,NRS386,2,2


In [127]:
proba8 = model2_over4.predict_proba(X_sel_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [128]:
dat_proba8

,0,1,2
0,0.006798,0.055683,0.937519
1,0.112860,0.056992,0.830148
2,0.480284,0.383319,0.136397
3,0.212836,0.003320,0.783843
4,0.314489,0.507028,0.178483
...,...,...,...
112,0.002477,0.016737,0.980786
113,0.593088,0.224250,0.182662
114,0.314489,0.507028,0.178483
115,0.056708,0.293001,0.650292


In [129]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [130]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p006ST.csv", index = False,
         header=None)

In [265]:
hist2_over4 = model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 270 samples, validate on 117 samples
Epoch 1/1000
270/270 [==============================] - 0s 72us/step - loss: 0.6900 - accuracy: 0.6889 - val_loss: 1.0304 - val_accuracy: 0.6325
Epoch 2/1000
270/270 [==============================] - 0s 66us/step - loss: 0.6966 - accuracy: 0.7000 - val_loss: 1.1327 - val_accuracy: 0.6154
Epoch 3/1000
270/270 [==============================] - 0s 66us/step - loss: 0.7060 - accuracy: 0.7037 - val_loss: 1.0305 - val_accuracy: 0.6154
Epoch 4/1000
270/270 [==============================] - 0s 84us/step - loss: 0.6902 - accuracy: 0.6741 - val_loss: 1.0337 - val_accuracy: 0.6068
Epoch 5/1000
270/270 [==============================] - 0s 89us/step - loss: 0.7010 - accuracy: 0.7000 - val_loss: 0.9978 - val_accuracy: 0.5983
Epoch 6/1000
270/270 [==============================] - 0s 80us/step - loss: 0.6762 - accuracy: 0.6963 - val_loss: 0.9932 - val_accuracy: 0.6239
Epoch 7/1000
270/270 [==============================] - 0s 70us/step - loss: 0.6653 

Epoch 57/1000
270/270 [==============================] - 0s 65us/step - loss: 0.6880 - accuracy: 0.6963 - val_loss: 1.0701 - val_accuracy: 0.5812
Epoch 58/1000
270/270 [==============================] - 0s 96us/step - loss: 0.6780 - accuracy: 0.6926 - val_loss: 1.0312 - val_accuracy: 0.5812
Epoch 59/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6641 - accuracy: 0.7074 - val_loss: 0.9837 - val_accuracy: 0.6154
Epoch 60/1000
270/270 [==============================] - 0s 70us/step - loss: 0.6692 - accuracy: 0.7000 - val_loss: 1.0162 - val_accuracy: 0.6239
Epoch 61/1000
270/270 [==============================] - 0s 98us/step - loss: 0.6858 - accuracy: 0.6926 - val_loss: 1.0632 - val_accuracy: 0.6239
Epoch 62/1000
270/270 [==============================] - 0s 61us/step - loss: 0.7418 - accuracy: 0.6926 - val_loss: 1.0517 - val_accuracy: 0.6496
Epoch 63/1000
270/270 [==============================] - 0s 66us/step - loss: 0.6978 - accuracy: 0.6852 - val_loss: 1.0196 -

Epoch 113/1000
270/270 [==============================] - 0s 55us/step - loss: 0.6814 - accuracy: 0.7000 - val_loss: 1.0110 - val_accuracy: 0.6068
Epoch 114/1000
270/270 [==============================] - 0s 65us/step - loss: 0.6752 - accuracy: 0.6889 - val_loss: 1.0007 - val_accuracy: 0.5897
Epoch 115/1000
270/270 [==============================] - 0s 58us/step - loss: 0.6677 - accuracy: 0.6926 - val_loss: 1.1392 - val_accuracy: 0.6325
Epoch 116/1000
270/270 [==============================] - 0s 66us/step - loss: 1.0138 - accuracy: 0.7000 - val_loss: 1.6902 - val_accuracy: 0.6239
Epoch 117/1000
270/270 [==============================] - 0s 70us/step - loss: 1.1225 - accuracy: 0.7037 - val_loss: 1.2624 - val_accuracy: 0.6154
Epoch 118/1000
270/270 [==============================] - 0s 62us/step - loss: 0.7504 - accuracy: 0.6926 - val_loss: 1.1774 - val_accuracy: 0.5726
Epoch 119/1000
270/270 [==============================] - 0s 65us/step - loss: 0.8676 - accuracy: 0.6333 - val_loss: 1

270/270 [==============================] - 0s 62us/step - loss: 0.7034 - accuracy: 0.6963 - val_loss: 1.0409 - val_accuracy: 0.6410
Epoch 224/1000
270/270 [==============================] - 0s 78us/step - loss: 0.6817 - accuracy: 0.7111 - val_loss: 1.0274 - val_accuracy: 0.6068
Epoch 225/1000
270/270 [==============================] - 0s 66us/step - loss: 0.6691 - accuracy: 0.6815 - val_loss: 0.9995 - val_accuracy: 0.5983
Epoch 226/1000
270/270 [==============================] - 0s 68us/step - loss: 0.6793 - accuracy: 0.6852 - val_loss: 1.0278 - val_accuracy: 0.6154
Epoch 227/1000
270/270 [==============================] - 0s 75us/step - loss: 0.6715 - accuracy: 0.7000 - val_loss: 1.0074 - val_accuracy: 0.5897
Epoch 228/1000
270/270 [==============================] - 0s 74us/step - loss: 0.6726 - accuracy: 0.6889 - val_loss: 1.0229 - val_accuracy: 0.5983
Epoch 229/1000
270/270 [==============================] - 0s 96us/step - loss: 0.6771 - accuracy: 0.7037 - val_loss: 1.0391 - val_acc

Epoch 334/1000
270/270 [==============================] - 0s 62us/step - loss: 0.6533 - accuracy: 0.7037 - val_loss: 1.0001 - val_accuracy: 0.6581
Epoch 335/1000
270/270 [==============================] - 0s 87us/step - loss: 0.6419 - accuracy: 0.7074 - val_loss: 1.0127 - val_accuracy: 0.6239
Epoch 336/1000
270/270 [==============================] - 0s 68us/step - loss: 0.6377 - accuracy: 0.7074 - val_loss: 0.9864 - val_accuracy: 0.6154
Epoch 337/1000
270/270 [==============================] - 0s 62us/step - loss: 0.6452 - accuracy: 0.6926 - val_loss: 1.0261 - val_accuracy: 0.6325
Epoch 338/1000
270/270 [==============================] - 0s 61us/step - loss: 0.6818 - accuracy: 0.6963 - val_loss: 1.0240 - val_accuracy: 0.6068
Epoch 339/1000
270/270 [==============================] - 0s 75us/step - loss: 0.6650 - accuracy: 0.6852 - val_loss: 1.0018 - val_accuracy: 0.5897
Epoch 340/1000
270/270 [==============================] - 0s 53us/step - loss: 0.6414 - accuracy: 0.6926 - val_loss: 1

...

270/270 [==============================] - 0s 57us/step - loss: 0.6923 - accuracy: 0.6963 - val_loss: 1.0207 - val_accuracy: 0.5897
Epoch 612/1000
270/270 [==============================] - 0s 70us/step - loss: 0.6843 - accuracy: 0.6963 - val_loss: 1.0129 - val_accuracy: 0.6068
Epoch 613/1000
270/270 [==============================] - 0s 63us/step - loss: 0.6499 - accuracy: 0.7000 - val_loss: 1.0148 - val_accuracy: 0.6410
Epoch 614/1000
270/270 [==============================] - 0s 66us/step - loss: 0.6601 - accuracy: 0.7000 - val_loss: 1.0445 - val_accuracy: 0.6325
Epoch 615/1000
270/270 [==============================] - 0s 75us/step - loss: 0.6364 - accuracy: 0.7111 - val_loss: 1.1002 - val_accuracy: 0.6068
Epoch 616/1000
270/270 [==============================] - 0s 81us/step - loss: 0.6965 - accuracy: 0.6741 - val_loss: 1.0526 - val_accuracy: 0.6239
Epoch 617/1000
270/270 [==============================] - 0s 65us/step - loss: 0.6624 - accuracy: 0.7000 - val_loss: 1.0001 - val_acc

Epoch 667/1000
270/270 [==============================] - 0s 193us/step - loss: 0.6445 - accuracy: 0.7074 - val_loss: 1.0007 - val_accuracy: 0.6325
Epoch 668/1000
270/270 [==============================] - 0s 192us/step - loss: 0.6510 - accuracy: 0.6963 - val_loss: 1.0078 - val_accuracy: 0.6154
Epoch 669/1000
270/270 [==============================] - 0s 145us/step - loss: 0.6398 - accuracy: 0.7000 - val_loss: 1.0445 - val_accuracy: 0.6239
Epoch 670/1000
270/270 [==============================] - 0s 253us/step - loss: 0.6340 - accuracy: 0.7111 - val_loss: 1.0767 - val_accuracy: 0.5983
Epoch 671/1000
270/270 [==============================] - 0s 200us/step - loss: 0.6774 - accuracy: 0.6889 - val_loss: 1.0571 - val_accuracy: 0.6068
Epoch 672/1000
270/270 [==============================] - 0s 172us/step - loss: 0.6448 - accuracy: 0.7000 - val_loss: 1.0102 - val_accuracy: 0.6068
Epoch 673/1000
270/270 [==============================] - 0s 143us/step - loss: 0.6412 - accuracy: 0.6926 - val_

270/270 [==============================] - 0s 106us/step - loss: 0.6280 - accuracy: 0.7148 - val_loss: 1.0542 - val_accuracy: 0.6154
Epoch 778/1000
270/270 [==============================] - 0s 88us/step - loss: 0.6511 - accuracy: 0.6852 - val_loss: 1.0851 - val_accuracy: 0.6154
Epoch 779/1000
270/270 [==============================] - 0s 65us/step - loss: 0.6586 - accuracy: 0.6926 - val_loss: 1.0625 - val_accuracy: 0.6410
Epoch 780/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6426 - accuracy: 0.6963 - val_loss: 1.0194 - val_accuracy: 0.6325
Epoch 781/1000
270/270 [==============================] - 0s 65us/step - loss: 0.6274 - accuracy: 0.7111 - val_loss: 1.0053 - val_accuracy: 0.6496
Epoch 782/1000
270/270 [==============================] - 0s 66us/step - loss: 0.6274 - accuracy: 0.7074 - val_loss: 1.0410 - val_accuracy: 0.6239
Epoch 783/1000
270/270 [==============================] - 0s 92us/step - loss: 0.6230 - accuracy: 0.7074 - val_loss: 1.0284 - val_ac

Epoch 833/1000
270/270 [==============================] - 0s 100us/step - loss: 0.6304 - accuracy: 0.6963 - val_loss: 1.0313 - val_accuracy: 0.6239
Epoch 834/1000
270/270 [==============================] - 0s 73us/step - loss: 0.6327 - accuracy: 0.6963 - val_loss: 1.0360 - val_accuracy: 0.6239
Epoch 835/1000
270/270 [==============================] - 0s 87us/step - loss: 0.6346 - accuracy: 0.6852 - val_loss: 1.0259 - val_accuracy: 0.6581
Epoch 836/1000
270/270 [==============================] - 0s 102us/step - loss: 0.6329 - accuracy: 0.6963 - val_loss: 1.0102 - val_accuracy: 0.6325
Epoch 837/1000
270/270 [==============================] - 0s 86us/step - loss: 0.6305 - accuracy: 0.6963 - val_loss: 1.0048 - val_accuracy: 0.6581
Epoch 838/1000
270/270 [==============================] - 0s 78us/step - loss: 0.6466 - accuracy: 0.6889 - val_loss: 1.0445 - val_accuracy: 0.6325
Epoch 839/1000
270/270 [==============================] - 0s 71us/step - loss: 0.6370 - accuracy: 0.7148 - val_loss:

Epoch 943/1000
270/270 [==============================] - 0s 107us/step - loss: 0.6321 - accuracy: 0.6963 - val_loss: 1.0373 - val_accuracy: 0.6239
Epoch 944/1000
270/270 [==============================] - 0s 80us/step - loss: 0.6291 - accuracy: 0.7037 - val_loss: 1.0500 - val_accuracy: 0.5983
Epoch 945/1000
270/270 [==============================] - 0s 106us/step - loss: 0.6256 - accuracy: 0.6926 - val_loss: 1.0536 - val_accuracy: 0.6068
Epoch 946/1000
270/270 [==============================] - 0s 86us/step - loss: 0.6164 - accuracy: 0.7074 - val_loss: 1.0468 - val_accuracy: 0.6068
Epoch 947/1000
270/270 [==============================] - 0s 84us/step - loss: 0.6282 - accuracy: 0.6926 - val_loss: 1.0328 - val_accuracy: 0.5726
Epoch 948/1000
270/270 [==============================] - 0s 142us/step - loss: 0.6411 - accuracy: 0.6889 - val_loss: 1.0473 - val_accuracy: 0.6239
Epoch 949/1000
270/270 [==============================] - 0s 167us/step - loss: 0.6215 - accuracy: 0.7000 - val_los

In [266]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over4.history['accuracy'])*100))

over-sampling train accuracy: 69.56%


In [65]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [66]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS236,1,2,1.321970e-02,2.446264e-01,7.421539e-01
1,p0006kpresabs_qual,NRS113,2,2,3.478230e-02,2.806685e-01,6.845492e-01
2,p0006kpresabs_qual,CFBRSa23,0,0,4.090251e-01,3.405008e-01,2.504741e-01
3,p0006kpresabs_qual,NRS249,2,1,1.987907e-01,5.331044e-01,2.681049e-01
4,p0006kpresabs_qual,107,1,0,4.090251e-01,3.405008e-01,2.504741e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CFBRSa30,0,0,7.207667e-01,2.792331e-01,2.571588e-07
985,p0017Skpresabs_qual,NRS383,1,0,6.129044e-01,3.870795e-01,1.601290e-05
986,p0017Skpresabs_qual,NRS110,2,2,3.260306e-07,7.910664e-07,9.999989e-01
987,p0017Skpresabs_qual,NRS209,2,2,3.604249e-12,2.698129e-07,9.999998e-01


In [67]:
y_prob8 = df_proba8[df_proba8['phage']=='p0006kpresabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[6.79776700e-03, 5.56828380e-02, 9.37519430e-01],
       [1.12860054e-01, 5.69916070e-02, 8.30148340e-01],
       [4.80284450e-01, 3.83318600e-01, 1.36396960e-01],
       [2.12836350e-01, 3.32044410e-03, 7.83843160e-01],
       [3.14489450e-01, 5.07027570e-01, 1.78483000e-01],
       [5.49065770e-02, 2.23908190e-01, 7.21185200e-01],
       [3.14489450e-01, 5.07027570e-01, 1.78483000e-01],
       [6.49095650e-01, 3.41353980e-01, 9.55032300e-03],
       [3.14489450e-01, 5.07027570e-01, 1.78483000e-01],
       [5.35238000e-01, 3.76735000e-01, 8.80270150e-02],
       [3.14489450e-01, 5.07027570e-01, 1.78483000e-01],
       [2.57582960e-01, 1.81236270e-01, 5.61180700e-01],
       [3.14489450e-01, 5.07027570e-01, 1.78483000e-01],
       [5.81124570e-02, 3.08436660e-01, 6.33450900e-01],
       [6.86385630e-01, 3.08748930e-01, 4.86536930e-03],
       [6.34144400e-05, 2.58747780e-04, 9.99677900e-01],
       [1.04569540e-01, 3.83047200e-01, 5.12383200e-01],
       [3.14489450e-01, 5.07027

In [68]:
ovo8 = rocauc_ovo(y_sel_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.7501643655489808

In [69]:
ovr8 = rocauc_ovr(y_sel_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.7501643655489808

In [70]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.7519449923296077

In [71]:
np.std(ovos2)

0.022618617985551374

In [72]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.7519449923296077

In [73]:
np.std(ovrs2)

0.022618617985551374

In [267]:
accs_l_over = [acc_test2_over, acc_test2_over2, acc_test2_over3, acc_test2_over4]

In [268]:
mean_l_over = np.mean(accs_l_over)
print('over-sampling test accuracy mean after lasso: %.2f%%' % (mean_l_over*100))

over-sampling test accuracy mean after lasso: 62.39%


In [269]:
std_l_over = np.std(accs_l_over)
print('over-sampling test accuracy standard deviation after lasso:', std_l_over)

over-sampling test accuracy standard deviation after lasso: 0.03676207159959011


In [270]:
accs_train_l_over = [np.mean(hist2_over.history['accuracy']), np.mean(hist2_over2.history['accuracy']), np.mean(hist2_over3.history['accuracy']),
             np.mean(hist2_over4.history['accuracy'])]

In [271]:
mean_train_l_over = np.mean(accs_train_l_over)
print('over-sampling train accuracy mean after lasso: %.2f%%' % (mean_train_l_over*100))

over-sampling train accuracy mean after lasso: 71.13%


In [272]:
std_train_l_over = np.std(accs_train_l_over)
print('over-sampling train accuracy standard deviation after lasso:', std_train_l_over)

over-sampling train accuracy standard deviation after lasso: 0.010667921
